In [1]:
import os
import sys
import requests

import time

# need to see processing speed
from time import process_time

# need for date functions
from pandas.tseries.offsets import DateOffset


from pandas import DataFrame
import pandas as pd

import numpy as np

import re

# bold text
from IPython.display import Markdown, display
def printmd(string):
    display(Markdown(string))

# styles pandas dataframes
from IPython.core.display import HTML
css = open('style-table.css').read() + open('style-notebook.css').read()
HTML('<style>{}</style>'.format(css))

# need for my date functions
from datetime import date


import pandas as pd
import numpy as np
import chart_studio.plotly as py
import cufflinks as cf
import seaborn as sns   #used for the datasets provided by seaborn
import plotly.express as px
%matplotlib inline
import plotly.graph_objects as go



from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
cf.go_offline()

In [2]:
holidays2020 = ['2020-01-01', '2020-01-20', '2020-02-17', '2020-04-10', '2020-05-25', '2020-07-03', '2020-09-07']
holidays2021 = ['2021-01-01', '2021-01-18', '2021-02-15', '2021-04-02', '2021-05-31', '2021-07-05', '2021-09-06', '2021-11-25', '2021-12-24']
holidays2022 = ['2022-01-17', '2022-02-21', '2022-04-15', '2022-05-30', '2022-06-20', '2022-07-04', '2022-09-05', '2022-12-26']

weekdays_year = pd.date_range(start='2022/1/1', end='2022/12/31', freq='B')

# Complete NYSE & Nasdaq Ticker List
tickerList = """ 'NXTG  First Trust Index NextG ETF', 'PH  Parker Hannifin', 'GNOM  Global X Genomics and Biotechnology', 'DRI  Darden', 'FORA  Forian', 'PDM  Piedmont Office Realty', 'GPC  Genuine Parts', 'AGRO  Adecoagro', 'BAX  Baxter', 'HNDL  Strategy Shares Nasdaq 7HANDL Index', 'ACMR  ACM Research', 'PAYO  Payoneer Global', 'BLOK  Amplify Transformational Data Sharing ETF', 'PGNY  Progyny', 'AROC  Archrock', 'ARHS  Arhaus Inc', 'BMI  Badger Meter', 'LTPZ  PIMCO 15 Plus Year', 'AXDX  Accelerate Diagnostics', 'MASI  Masimo', 'CNP  CenterPoint Energy', '.VIX  CBOE Volatility Index', 'BJ  BJs Wholesale Club', 'KEMQ  KraneShares Emerging Mkts Consumer Technology Idx ETF', 'CNSL  Consolidated Communications', 'PLXS  Plexus', 'AKA  AKA Brands', 'GLOV  GLOV ETF', 'KIE  SPDR S&P Insurance', 'HNI  HNI Corporation', 'MARA  Marathon Patent Group', 'FND  Floor & Decor', 'AMKR  Amkor Technology', 'JJT  iPath Bloomberg Tin ETN', 'NTRA  Natera', 'KFYP  KraneShares CSI China Five Year Plan', 'DXJS  WisdomTree Japan Equity', 'BFAM  Bright Horizons', 'FCN  FTI Consulting', 'FMTX  Forma Therapeutics Holdings', 'PAYX  Paychex', 'BLKB  Blackbaud', 'MPAA  Motorcar Parts', 'BDN  Brandywine Realty', 'FMAT  MSCI Materials ETF', 'GBIL  Goldman Sachs Treasuryaccess 0 1 Year ETF', 'DIG  Proshares Oil & Gas', 'COLL  Collegium Pharmaceutical', 'CARG  CarGurus', 'HERO  Global X Video Games and Esports ETF', 'EWBC  East West Bancorp', 'GGR  Poema Global Holdings Corp', 'CERE  Cerevel Therapeutics', 'ALLE  Allegion', 'CBT  Cabot', '.RLV   Russell 1000 Value Index', 'BHIL  Benson Hill', 'BCTX  BriaCell Therapeutics', 'HOFV  Hall of Fame Resort', 'CHII  Global X China Industrials', 'BWA  BorgWarner', 'ENFN  Enfusion Inc', 'EYPT  EyePoint Pharmaceuticals', 'PLOW  Douglas Dynamics', 'DOYU  DouYu International', 'NMIH  NMI Holdings', 'BC  Brunswick', 'MIDD  Middleby', 'FOXA  21st Century Fox', 'BNDX  Vanguard International', 'BCBP  BCB Bancorp', 'FUV  Arcimoto', 'ITT  ITT Corp.', 'MVV  ProShares Ultra MidCap400', 'PARAA  Paramount Global A', 'LW  Lamb Weston', 'CRSR  Corsair Gaming', 'DGLY  Digital Ally', 'HAE  Haemonetics', 'EPZM  Epizyme', 'EQIX  Equinix', 'GLAD  Gladstone Capital', 'IGLB  iShares LongTerm Corporate Bond ETF', 'CARS  Cars.com', 'AVIR  Atea Pharmaceuticals ', 'CAE  Cae', 'DNUT  Krispy Kreme', 'MMM  3M', 'GM  General Motors', 'MFGP  Micro Focus International', 'LAZ  Lazard', 'CSV  Carriage Services', 'EMAN  eMagin', 'MYY  ProShares Short MidCap400', 'PFFA  Virtus Infracap US Preferred Stock ETF', 'NUGT  Direxion Daily Gold Miners Bull 3X Shares', 'ATRO  Astronics', 'MS  Morgan Stanley', 'CNHI  CNH Industrial', 'IVZ  Invesco', 'IEMG  iShares Core MSCI', 'EXPO  Exponent', 'OIS  Oil States', 'AMSWA  American Software', 'NFG  National Fuel', 'IEUR  iShares Core MSCI Europe', 'AXL  American Axle', 'NZRO  NZRO ETF', 'CODI  Compass Diversified', 'KCE  SPDR S&P Capital Markets', 'BARK  Original BARK Company', 'FPAC  Far Peak Acquisition Corp', 'CCK  Crown Holdings', 'FRST  Primis Financial', 'JOET  Virtus Terranova US Quality Momentum ETF', 'LUMN  Lumen Technologies', 'AMP  Ameriprise', 'AEE  Ameren', 'INDS  Pacer Benchmark Ind. Real Estate SCTR ETF', 'FQAL  Fidelity Quality Factor ETF', 'AZUL  Azul', 'DPST  Direxion Daily Regional Banks Bull 3X Shares', 'KFS  Kingsway Financial', 'HONE  HarborOne Bancorp', 'EMBC  Embecta Corp', 'EL  Estee Lauder', 'DGRW  WisdomTree Dividend', 'AVDX  AvidXchange Holdings', 'IXN  iShares Global Tech', 'MBWM  Mercantile Bank', 'BEKE  KE Holdings', 'BVN  Buenaventura', 'BRC  Brady', 'FMX  Fomento Mexicano', 'LVS  Las Vegas Sands', 'LINC  Lincoln Educational', 'CRH  CRH', 'BTTX  Better Therapeutics', 'ASHX  DBX CSI 300 China A Shares Hedged Equity ETF', 'OVLH  Overlay Shares Hedged Large Cap Equity ETF', 'GSPY  Gotham Enhanced 500 ETF', 'GTLS  Chart Industries', 'ANY  Sphere 3D', 'FMNB  Farmers National Banc', 'CHUY  Chuy s', 'BAC  Bank of America', 'PEI  Pennsylvania REIT', 'CCF  Chase Corporation', 'AKBA  Akebia', 'LTCH  Latch', 'DWAS  PowerShares DWA', 'EBS  Emergent BioSolutions', 'GLOB  Globant', 'NNOX  Nano X Imaging', 'BEST  BEST Inc', 'FRME  First Merchants', 'JJS  iPath Series B Softs ETN', 'IONS  Ionis Pharmaceuticals', 'HP  Helmerich & Payne', 'MAN  Manpower', 'HOFT  Hooker Furniture', 'FTRP  Field Trip Health', 'GCMG  GCM Grosvenor', 'CRUS  Cirrus Logic', 'CDAY  Ceridian HCM Holding', 'BJK  Market Vectors Gaming', 'CIEN  Ciena', 'LD  iPath Bloomberg Lead Subindex Total Return ETN', 'MELI  MercadoLibre', 'LIFE  aTyr Pharma', 'PFS  Provident Financial', 'IPSC  Century Therapeutics Inc', 'EBF  Ennis', 'AIEQ  AI Powered Equity ETF', 'INTF  iShares FactorSelect MSCI International ETF', 'EVGO  EVgo', 'FNLC  The First Bancorp', 'IEIH  iShares Evolved US Innovative Healthcare ETF', 'NXRT  NexPoint Residential', 'MOH  Molina Healthcare', 'LAZR  Luminar Technologies', 'EHTH  eHealth', 'CCB  Coastal Financial', 'LSI  Life Storage', 'CROX  Crocs', '.XDS  PHLX Swiss Franc Index', 'GTES  Gates Industrial Corp', 'ELS  Equity Lifestyle', 'JNPR  Juniper Networks', 'NWSA  News Corp.', 'JUST  Just Us Large Cap Equity ETF', 'ADSK  Autodesk', 'IMLP  iPath S&P', '.NANOS  Cboe NANOS Index', 'DVOL  First Trust Dorsey Wright Momentum and Low Volatility ETF', 'FWRG  First Watch Restaurant Group Inc', 'FWONK  Liberty Formula One', 'DRIP  Direxion Oil & Gas 3x Bear ETF', 'EAR  Eargo', 'JAAA  Janus Henderson AAA CLO ETF', 'BZ  Kanzhun Ltd', 'PLYA  Playa Hotels & Resorts', 'DFAC  Dimensional US Core Equity 2 E', 'BL  BlackLine Inc', 'IMO  Imperial Oil', 'MTVR  Fount Metaverse ETF', 'ALTG  Alta Equipment', 'COUR  Coursera', 'JD  JD.com', 'CMG  Chipotle', 'LAZY  Lazydays Holdings', 'MLPX  Global X MLP Energy', 'HRTX  Heron Therapeutics', 'GCC  GCC', 'MRTN  Marten Transport', 'DOG  ProShares Dow30', 'BEN  Franklin Resources', 'MORN  Morningstar', 'CPF  Central Pacific', 'EVEX  Eve Holding Inc', 'FRI  First Trust REIT Index', 'ESSA  ESSA Bancorp', 'FBP  First Bancorp', 'GPRE  Green Plains', 'NS  NuStar Energy', 'BBD  Banco Bradesco', 'LEU  Centrus Energy', 'IVAC  Intevac', 'CGNX  Cognex', 'ESS  Essex Property Trust', 'GSAT  Globalstar', 'MYMD  MyMD Pharmaceuticals', 'KPLT  Katapult Holdings', 'HRB  H&R Block', 'PLG  Platinum Group', 'EPV  ProShares MSCI Europe', 'OPTT  Ocean Power', 'EWCO  S&P 500 Equal Wt Comm Services ETF', 'NSTB  Northern Star Investment Corp II', 'INTA  Intapp Inc', 'ELAN  Elanco Animal Health', 'FNF  Fidelity National', 'EXR  Extra Space Storage', 'KBUY  Kraneshares Cicc China Consumer Leaders Index ETF', 'HIVE  HIVE Blockchain', 'MNRL  Brigham Minerals', 'LFMD  LifeMD', 'BLTS  Bright Lights Acquisition Corp', 'AWH  Aspiro Womans Health', 'JXI  Global Utilities', 'BKKT  Bakkt Holdings', 'ALIT  Alight', 'DBO  PowerShares Oil', 'HTZ  Hertz Global Holdings', 'KBA  KraneShares Bosera MSCI China A', 'BOTZ  Global X Robotics and Artificial Intelligence Thematic', 'LHCG  LHC Group', 'AMCR  Amcor', 'GPN  Global Payments', 'ITOS  iTeos Therapeutics', 'NTLA  Intellia Therapeutics', 'LOVE  Lovesac', 'GBDC  Golub Capital', 'CRBN  iShares MSCI ACWI Low Carbon Target ETF', 'DAR  Darling Ingredients', 'GRID  First Trust NASDAQ Clean Edge Smart Grid Infrastructure Index Fd', 'PHUN  Phunware', 'JPUS  JPMorgan Diversified Return US Equity ETF', 'FOUR  Shift4 Payments', 'KBH  KB Home', 'CF  CF Industries', 'CLPT  ClearPoint Neuro', 'CBD  Companhia Brasileira de Distribuicao', 'ARMK  Aramark', 'FXZ  AlphaDEX Fund', 'CLDL  Direxion Daily Cloud Computing Bull 2x', 'CAG  ConAgra', 'ALL  Allstate', 'EBND  SPDR Emeging Markets', '.MXEF  MSCI Emerging Markets Index', 'HGTY  Aldel Financial', 'FFIN  First Financial', 'HPP  Hudson Pacific', 'EMB  iShares Emerging Markets Bond ETF', 'KBWR  PowerShares KBW Regional Banking Portfolio', 'ATNX  Athenex', 'CORR  CorEnergy', 'INBX  Inhibrx', 'MPX  Marine Products', 'GINN  Goldman Sachs Innovate ETF', 'FXD  AlphaDEX Consumer Discretionary', 'NTUS  Natus Medical', 'CRTX  Cortexyme', 'CNXC  Concentrix', '.RLG   Russell 1000 Growth Index', 'HVT  Haverty Furniture', 'HIBS  Direxion Daily S&P 500 High Beta Bear 3x Shares', 'BIRD  Allbirds', 'D  Dominion', 'JFIN  Jiayin Group', 'CTRA  Coterra Energy', 'AYI  Acuity Brands', 'METC  Ramaco Resources', 'DGRO  iShares Core Dividend Growth', 'GOLF  Acushnet', 'ATXS  Astria Therapeutics', 'DUSL  Direxion Daily Industrials Bull 3x Shares ETF', 'HPE  Hewlett Packard Enterprise', 'LIVN  LivaNova', 'GSLC  Goldman Sachs ActiveBeta US Large Cap Equity ETF', 'HEXO  Hexo Corp', 'LTRY  Lottery.com', 'KEUA  Kraneshares European Carbon Allowance ETF', 'EEMX  SPDR MSCI Emerging Mkt Fossil Fuel Reserves Free ETF', 'CSWI  CSW Industrials', 'EWN   iShares Netherlands', 'DXYN  Dixie Group', 'HOPE  Hope Bancorp', 'MXI  iShares Global Materials', 'OGS  ONE Gas', 'ECPG  Encore Capital', 'FENY  Fidelity MSCI Energy Index', 'HSTM  HealthStream', 'NHWK  Heat Biologics Inc', 'LBAI  Lakeland Bancorp', 'AWAY  ETFMG Travel Tech ETF', 'DFAS  Dimensional US Small Cap ETF', 'PETZ  TDH Holdings', 'FAN  First Trust ISE Global Wind Energy Index Fund', 'BLD  TopBuild', 'FCUV  Focus Universal', 'IWC  iShares Russell Microcap Index Fund', 'BOH  Bank of Hawaii', 'HMST  HomeStreet', 'KW  Kennedy Wilson', 'GPI  Group 1 Automotive', 'CHIU  Global X MSCI China Utilities ETF', 'BRFS  BRF S.A.', 'DPZ  Dominos', 'CMCSA  Comcast', 'FBIZ  First Business Financial', 'AKRO  Akero Therapeutics', 'DES  WIsdomTree Trust', 'IBTX  Independent Bank', 'CGAU  Centerra Gold', 'ALB  Albemarle', 'INDY  iShares S&P India Nifty 50 Index Fund', 'PBFX  PBF Logistics', 'FBND  Fidelity Total Bond ETF', 'AMPE  Ampio', 'COMB  Graiteshares Bloomberg Comm ETF', 'DYN  Dyne Therapeutics', 'EA  Electronic Arts', 'NOVN  Novan', 'NOMD  Nomad Foods', 'KXIN  Kaixin Auto', 'DKS  Dicks', 'CION  Cion Investment Corp', 'AFK  Market Vectors Africa', 'CTRN  Citi Trends', 'DHT  DHT Holdings', 'AEZS  Aeterna Zentaris', 'FOX  Twenty First Century Fox', 'BFLY  Butterfly Network', 'AVNS  Avanos Medical', 'KXI  iShares Global Consumer', 'EFAX  SPDR MSCI Eafe Fossil Fuel Reserves Free ETF', 'EBET  Esports Technologies', 'CDZI  Cadiz', 'MNDT  Mandiant', 'IDCC  InterDigital', 'FUTY  Fidelity MSCI Utilities Index', 'FTSL  First Trust Senior Loan Fund', 'BYND  Beyond Meat', 'ATHM  Autohome', 'KEY  KeyCorp', 'CNXN  Connection', 'KO  Coca Cola', 'BODY  Beachbody', 'LYRA  Lyra Therapeutics Inc', 'AXU  Alexco Resource', 'IJJ  iShares S&P MidCap 400 Value Index Fund', 'CL  Colgate Palmolive', 'PAR  PAR Technology', 'JG  Aurora Mobile', 'FBC  Flagstar Bancorp', 'NKTX  Nkarta', 'HLX  Helix Energy', 'LOCL  Leo Holdings III Corp', 'IYY  iShares Dow Jones US', 'CLXT  Calyxt', 'FSV  FirstService', 'AMAM  Ambrx Biopharma Inc', 'ATR  AptarGroup', 'MMP  Magellan Midstream', 'PCG  PG&E', 'CYB  WisdomTree Chinese Yuan', 'BSM  Black Stone Minerals', 'IEDI  iShares Evolved US Discretionary Spending ETF', 'LABU  Direxion S&P Biotech Bull 3X', 'MTN  Vail Resorts', 'CAMP  CalAmp', 'FENG  Phoenix New Media', 'KALL  Kraneshares Msci All China Index ETF', 'FFIC  Flushing financial', 'BCD  Aberdeen Commodity Longer Dated ETF', 'AEG  AEGON', 'ELOX  Eloxx Pharmaceuticals', 'FID  First Trust ETF VI First Trust S&P International Div Aristocrats', 'IEF  iShares Barclays 7 to 10 Year Treasury Bond Fund', 'ACRE  Ares Commercial', 'ASHR  db X trackers China', 'DNN  Denison Mines', 'FWONA  Liberty Formula One', 'FIVE  Five Below', 'MLKN  Herman Miller', 'FLEX  Flex', 'AVNW  Aviat Networks', 'DOW  Dow Chemical', 'NCLH  Norwegian Cruise Lines', 'ATAI  ATAI Life Sciences', 'LEJU  Leju', 'BAMR  Brookfield Asset Management Reinsurance Partners Ltd (Class A Stock)', 'HPQ  HP Inc.', 'NVS  Novartis', 'EVRG  Evergy', 'BIZD  Market Vectors BDC Income', 'BEP  Brookfield Renewable', 'MRO  Marathon Oil', 'ANF  Abercrombie & Fitch', 'DCTH  Delcath Systems Inc', 'PAGP  Plains GP Holdings', 'ONEV  SPDR Russell 1000 Low Volatility', 'OPEN  Opendoor Technologies', 'NEPT  Neptune Technologies', 'BYD  Boyd Gaming', 'BZFD  Buzzfeed', 'CPB  Campbell Soup', 'ACB  Aurora Cannabis', 'CLMT  Calumet Specialty Products', 'BOOM  Dynamic Materials', 'OLLI  Ollies Bargain Outlet', 'EGRX  Eagle Pharmaceuticals', 'BBVA  Banco Bilbao', 'IXG  iShares Global Financials', 'AMK  AssetMark Financial', 'CGGR  CGGR ETF', 'NVMI  Nova Measuring', 'COHU  Cohu', 'AZO  AutoZone', 'MPC  Marathon Petroleum', 'LOCO  El Pollo Loco', 'KRBN  KraneShares Global Carbon ETF', 'DELL  Dell Technologies', 'AMBC  Ambac Financial', 'MRAI  Marpai', 'ASPU  Aspen Group', 'COOP  Mr Cooper Group', 'FNX  First Trust Mid Cap Core AlphaDEX Fund', 'CADE  Cadence Bancorp', 'PDCO  Patterson Companies', 'ISRG  Intuitive Surgical', 'EFG  iShares MSCI Growth', 'HLIO  Helios Technologies', 'BLNK  Blink Charging', 'OTRK  Ontrak', 'PLL  Piedmont Lithium', 'NNVC  NanoViricides', 'AMEH  Apollo Medical Holdings', 'FLWS  1 800 Flowers', 'BEDZ  Advisorshares Hotel ETF', 'CRAK  Market Vectors Oil Refiners ETF', 'FNHC  Federated National', 'IJH  iShares S&P MidCap 400 Index Fund', 'CUZ  Cousins Properties', 'GLL  ProShares UltraShort Gold', 'KEX  Kirby', 'CMRE  Costamare', 'GEOS  Geospace', 'AGNC  AGNC Investment Corp.', 'IDLV  PowerShares S&P International Developed Low Volatility Portfolio', 'NSSC  NAPCO Security', 'EWJ  iShares Japan', 'HHC  Howard Hughes', 'FRPT  Freshpet', 'OSIS  OSI Systems', 'GDDY  GoDaddy', 'MNKKQ  Mallinckrodt', 'HDEF  DBX trackers MSCI EAFE High Dividend Yield Hedged', 'FNCL  Fidelity MSCI Financials Index', 'MDYG  SPDR S&P 400 Mid Cap Growth', 'LODE  Comstock Mining', 'AUR  Reinvent Technology Partners', 'AGNG  Global X Aging Population ETF', 'AMN  AMN Healthcare', 'BIOC  Biocept', 'IJS  iShares S&P SmallCap 600 Values Index Fund', 'FTAG  First Trust II Global Agriculture', 'NJR  New Jersey Resources', 'PBJ  Power Shares Food And Beverage', 'DESP  Despegar.com', 'BIV  Vanguard Intermediate Bond', 'BF/A  Brown Forman', 'ICE  Intercontinental Exchange', 'HAUS  Home Appreciation REIT ETF', 'OXY  Occidental Petroleum', 'NZAC  NZAC ETF', 'ALXO  ALX Oncology', 'IQLT  iShares Edge MSCI Intl Quality Factor ETF', 'DGRS  WisdomTree Trust WisdomTree US SmallCap Quality Div Grwth Fund', 'GWRE  GuideWire', 'CNS  Cohen & Steers', 'NLSN  Nielsen', 'DUOL  Duolingo', 'BBAX  JPMorgan BetaBuilders Developed Asiaex Japan ETF', 'CTBI  Community Trust', 'OC  Owens Corning', 'PID  PowerShares International', 'ONLN  Proshares Online Retail ETF', 'HAIL  SPDR S&P Kensho Smart Mobility ETF', '.XDB  PHLX British Pound Index', 'BREZ  Breeze Holdings Acquisition Corp', 'FDVV  Fidelity Core Dividend ETF', 'CVNA  Carvana', 'DASH  DoorDash', 'DLN  WisdomTree Trust', 'GEM  Goldman Sachs ActiveBeta Emerging Markets Equity ETF', 'ATEN  A10 Networks', 'AVXL  Anavex', 'HYGH  iShares Interest Rate Hedged High Yield Bond', 'CSD  Guggenheim Spin', 'OB  Outbrain Inc', 'JAMF  Jamf Holding Corp', 'BITF  Bitfarms', 'DD  DuPont de Nemours', 'ARAV  Aravive', 'DOOO  BRP', 'LYFT  Lyft', 'ACXP  Acurx Pharmaceuticals', '.SIXB  S&P Materials Sector Index', 'APPH  AppHarvest', 'FXU  Utilities AlphaDEX', 'MSTR  MicroStrategy', 'BMA  Banco Macro', 'NVTA  Invitae', 'CE  Celanese', 'BMTX  BM Technologies', 'DFAU  Dimensional US Core Equity Market ETF', 'PICK  iShares MSCI Select Metals', 'MILN  Global X Millennials Thematic', 'MSCI  MSCI', 'EPM  Evolution Petroleum', 'HRI  Herc Holdings', 'OFS  OFS Capital', 'GYLD  Arrow Dow Jones Global Yield', 'GLBE  Global E Online', 'DBP  PowersShares Metals', 'GOGO  Gogo', 'MRIN  Marin Software', 'OLPX  Olaplex Holdings', 'MAT  Mattel', 'EB  Eventbrite', 'AJRD  Aerojet Rocketdyne', 'FCPT  Four Corners Property Trust', 'IVOO  Vanguard S&P Mid Cap 400', 'FSS  Federal Signal', 'BATRK  Liberty Media Series C', 'PIN  PowerShares India Portfolio', 'KOLD  ProShares UltraShort DJ-UBS Natural Gas', 'CLIR  ClearSign Combustion', 'APPF  AppFolio', 'FLCH  Franklin FTSE China ETF', 'IDYA  IDEAYA Biosciences', 'FNA  Paragon 28 Inc', 'OGIG  Oshares Global Internet Giants ETF', 'FTXO  First Trust Nasdaq Bank', 'ARBE  Autobot Holdings', 'ENS  Enersys', 'MOG/A  Moog Inc', 'AFG  American Financial Group', 'JETS  US Global Jets ETF', 'GLDD  Great Lakes Dredge', 'DWAC  Digital World Acquisition Corp', 'CAAP  Corporacion America Airports', 'CHD  Church & Dwight', 'NUVA  Nuvasive', 'ANDE  Andersons', 'ELVT  Elevate', 'NVGS  Navigator Holdings', 'LMAT  LeMaitre', 'KFY  Korn Ferry', 'MGPI  MGP Ingredients', 'FTEK  Fuel Tech', 'NCTY  The 9 Limited', 'FNDA  Schwab Fundamental US Small Company Index', 'HTGC  Hercules Capital', 'GNUS  Genius Brands', 'COW  iPath Livestock ETN', 'MN  Manning & Napier', 'AMD  Advanced Micro', 'MLI  Mueller Industries', 'NMG  Nouveau Monde Graphite', '.SIXM  S&P Financial Sector Index', 'DBVT  DBV Technologies', 'APH  Amphenol', 'EZA    iShares South Africa', 'GEF  Greif', 'MFA  MFA Financial', 'KWEB  KraneShares China Internet', 'ELMSQ  Electric Last Mile Solutions Inc', 'HALO  Halozyme', 'LAKE  Lakeland Industries', 'CRIT  CRIT ETF', 'GBT  Global Blood Therapeutics', 'PLX  Protalix', 'IYT  iShares Transportation Average', 'HEDJ  Wizdom Tree Europe', 'PGEN  Precigen', 'IJR  IShares S&P SmallCap 600 Index Fund', 'EDU  New Oriental Education', 'EXPE  Expedia', 'CVY  Guggenheim Multi Asset', 'DOMO  Domo', 'HOMB  Home BancShares', 'KLIC  Kulicke & Soffa', 'GOOGL  Alphabet Class A', 'HTA  Healthcare Trust', 'NRDS  Nerdwallet', 'DM  Desktop Metal', 'CWB  SPDR Convertible', 'AMED  Amedisys', 'AIG  AIG', 'FFIE  Faraday Future Intelligent Electric', 'PHM  PulteGroup', 'OCFT  OneConnect FinTech', 'OAS  Oasis Petroleum ', 'MSI  Motorola Solutions', 'MEXX  Direxion Msci Mexico Bull 3x', 'ACCO  Acco Brands', 'CEMB  iShares Emerging Markets Corporate Bond ETF', 'NWPX  Northwest Pipe', 'CKPT  Checkpoint Therapeutics', 'HRMY  Harmony Biosciences', 'GAIN  Gladstone Investment', 'DCI  Donaldson', 'MCRI  Monarch Casino', 'IBB  iShares Nasdaq Biotechnology Index Fund', 'HLNE  Hamilton Lane', "BCO  Brink's", 'NI  Nisource', 'CHB  China Biotech Innovations ETF', 'JPEM  JPMorgan Diversified Return Emerging Mkts Equ ETF', 'CRDF  Cardiff Oncology', 'PGRE  Paramount Group', 'GRTS  Gritstone Oncology', 'FTCH  Farfetch Limited', 'MEI  Methode Electronics', 'CLDT  Chatham Lodging', 'AGL  Agilon Health', 'NVDA  NVIDIA', 'DOV  Dover', 'KSCD  KFA Small Cap Quality Dividend Index ETF', 'MXL  Maxlinear', 'MOD  Modine Manufacturing', 'MSEX  Middlesex Water', '.XDA  PHLX Australian Dollar Index', 'HCI  Homeowners Choice', 'CLX  Clorox', 'CAL  Caleres', 'AVNT  Avient', 'FREY  FREYR Battery', 'NTCO  Natura and Co', 'GIFI  Gulf Island Fabrication', 'NGM  NGM Biopharmaceuticals', 'BBU  Brookfield Business Partners', 'IFF  International Flavors', 'CSII  Cardiovascular Systems', 'HMTV  Hemisphere Media', 'INSW  International Seaways', 'PI  Impinj', 'ERJ  Embraer', 'PCSB  PCSB Financial', 'IOVA  Iovance Biotherapeutics', 'BB  BlackBerry', 'IBN  Icici Bank', 'DDG  ProShares Short Oil and Gas', 'FCG  ISE Natural Gas Index', 'PESI  PermaFix Environmental Services', 'BCC  Boise Cascade', 'FXC  Canadian Dollar Trust', 'CZR  Caesars', 'FAST  Fastenal', 'NUSI  Nationwide Risk managed Income ETF', 'HUN  Huntsman', 'MDIV  Multi-Asset Diversified Income ETF', 'LMNL  Liminal BioSciences', 'IMAX  IMAX', 'AMPH  Amphastar', 'CURV  Torrid Holdings', 'DG  Dollar General', 'FOCS  Focus Financial Partners', 'DMLP  Dorchester Minerals', 'CPLP  Capital Product', 'G  Genpact', 'NWE  NorthWestern', 'CVX  Chevron', 'GILT  Gilat Satellite', 'FBZ  First Trust Brazil AlphaDEX Fund', 'AVTR  Avantor', 'CCMP  Cabot Microelectronics', 'EURN  Euronav NV', 'GE  General Electric', 'GAME  Engine Media Holdings', 'EMBD  Global X Emerging Markets Bond ETF', 'DOC  Physicians Realty Trust', 'NL  NL Industries', 'IAU  iShares Gold Trust', 'MTTR  Matterport', 'LEGH  Legacy Housing Corp', 'BOWL  Isos Acquisition Corp', 'LPLA  LPL Financial', 'CACI  CACI', 'ADI  Analog Devices', 'ESGU  iShares Trust iShares ESG MSCI USA', 'OGN  Organon and Co', 'MKL  Markel', 'NSTG  NanoString', 'AMNB  American National Bankshares', 'BAND  Bandwidth Inc', 'AGO  Assured Guaranty', 'MTDR  Matador', 'AXP  American Express', 'IAUF  iShares Gold Strategy ETF', 'COWN  Cowen Group', 'CPRT  Copart', 'AVA  Avista', 'IWS  iShares Russell Midcap Value Index Fund', 'NABL  N able', 'JJN  iPath Bloomberg Nickel ETN', 'EWZ  EWZ Brazil ETF', 'BTI  British American Tobacco', 'EXFY  Expensify Inc', 'CGDV  CGDV ETF', 'HTHT  China Lodging', 'ENVA  Enova International', '.XDE  PHLX Euro Index', 'ARE  Alexandria Real Estate', 'PKG  Packaging Corp.', 'BZQ  ProShares UltraShort', 'CSBR  Champions Oncology', 'GO  Grocery Outlet', 'MTBC  Medical Transcription Billing', 'PAY  Paymentus Holdings', 'NOAH  Noah Holdings', 'GOCO  GoHealth', 'DJP  iPath UBS Index', 'FOA  Finance of America Companies', 'LGIH  LGI HOmes', 'GDRX  GoodRx Holdings ', 'IXUS  iShares MSCI', 'GPMT  Granite Point Mortgage', 'INNV  InnovAge Holding Corp', 'CLGN  CollPlant Biotechnologies', 'NX  Quanex', 'CTRE  CareTrust REIT', 'FNDE  Schwab Fundamental Emerging Markets Fund', 'KURA  Kura Oncology', 'HEZU  iShares MSCI EMU', 'FTHM  Fathom Holdings', 'AZPN  Aspen Technology', 'FTFT  Future FinTech Group', 'GTY  Getty Realty', 'HHH  HHH ETF', 'NU  Nu Holdings', 'COP  ConocoPhillips', '.XND  Nasdaq 100 Micro Index', 'FITE  SPDR S&P Kensho Future Security ETF', 'NERD  Roundhill Digital Entertainment ETF', 'MTOR  Meritor', 'JAZZ  Jazz Pharmaceuticals', 'MMX  Maverix Metals', 'CRNT  Ceragon Networks', 'LAW  CS Disco', 'CARE  Carter Bankshares', 'BNTC  Benitec Biopharma', 'LCID  Lucid Group', 'KRTX  Karuna Therapeutics', 'CPRX  Catalyst Pharmaceuticals', 'INFA  Informatica', 'EWGS  iShares MSCI Germany Small Cap ETF', 'PLNT  Planet Fitness', 'NETZ  NETZ ETF', 'CRSP  Crispr Therapeutics', 'CENTA  Central Garden & Pet', 'CATY  Cathay General', 'HLTH  Cue Health', 'CLNE  Clean Energy', 'IRNT  IronNet', 'HCOM  HCOM Stock', 'OCX  OncoCyte', 'NUE  Nucor', 'AFMD  Affimed NV', 'IHI  iShares US Medical Devices', 'IGRO  iShares International Dividend Growth ETF', 'MOXC  Moxian Inc', 'LMBS  First Trust Low Duration Mortgage Opportunities', 'HD  Home Depot', 'CRIS  Curis', 'AGEN  Agenus', 'OI  Owens-Illinois', 'DGII  Digi International', 'ALTU  Altitude Acquisition Corp', 'EWL  iShares Switzerland', 'GDOT  Green Dot', 'NEGG  Newegg Commerce', 'GORO  Gold Resource', 'ACAD  Acadia', 'AVB  AvalonBay', 'GENC  Gencor Industries', 'GAU  Galiano Gold', 'FDT  First Trust Developed Markets ex US AlphaDEX Fund', 'NIFE  Direxion Fallen Knives ETF', 'ABMD  Abiomed', 'GALT  Galectin Therapeutics', 'MYPS  PLAYSTUDIOS', 'AIP  Arteris Inc', 'DSGX  Descartes Systems', 'ASLN  ASLAN Pharmaceuticals', 'EDAP  EDAP TMS', 'GOSS  Gossamer Bio', 'ERF  Enerplus', 'PAG  Penske Automotive', 'AEVA  Aeva Technologies', 'MIC  Macquarie Infrastructure', 'CQQQ  Guggenheim China', 'NFRA  FlexShares STOXX Global Broad Infrastructure Index Fund', 'BUZZ  Vaneck Vectors Social Sentiment ETF', 'GIGE  BCD Aberdeen All Commodity ETF', 'IWP  iShares Russell Midcap Growth Index Fund', 'OCG  Oriental Culture Holdings', 'AWRE  Aware', 'ATI  Allegheny Technologies', 'HIBB  Hibbett Sports', 'MWA  Mueller Water', 'NILE  Ault Global Holdings', 'FCOM  Fidelity MSCI Telecommunication Services', 'GLIN  VanEck India Growth ETF', 'MAXN  Maxeon Solar Technologies', 'CXM  Sprinklr Inc', 'IMVT  Immunovant', 'CSGP  CoStar Group', 'INTZ  Intrusion Inc', 'ESTC  Elastic NV', 'ARAY  Accuray', 'GASS  Stealth Gas', 'FWRD  Forward Air', 'HOTL  HOTL ETF', 'AMBP  Ardagh Metal Packaging', 'ENLC  Enlink Midstream', 'HUBG  Hub Group', 'COCO  Vita Coco', 'CRVS  Corvus Pharmaceuticals', 'AKR  Acadia Realty Trust', 'DNA  Ginkgo Bioworks', 'CYCC  Cyclacel Pharmaceuticals', 'HOOK  HOOKIPA Pharma', 'FORR  Forrester', 'DVAX  Dynavax', 'GLMD  Galmed', 'NOA  North American Energy', 'CASS  Cass Information Systems', 'AVEO  AVEO Oncology', 'DFAT  Dimensional US Targeted Value ETF', '.MRUT  Mini Russell 2000', 'KRO  Kronos Worldwide', 'MUSA  Murphy USA', 'KBNT  Kubient', 'GSG  iShares GSCI Commodity Indexed Trust', 'DIVO  Amplify CWP Enhanced Div Income ETF', 'CALX  Calix', 'IMAB  I MAB (ADS)', 'JBT  John Bean Technologies', 'ARVL  Arrival', 'ASA  ASA Gold', 'JPST  JPMorgan Ultra-Short Income ETF', 'FVD  First Trust Value Line Dividend Fund', 'NE  Noble Corp', 'MIXT  MiX Telematics', 'EMTY  ProShares Decline of the Retail Store ETF', 'EWI  iShares Italy', 'EIDO  iShares Indonesia', 'ACRX  AcelRx', 'CO  China Cord Blood', 'EVOP  EVO Payments', 'DXD  Proshares Dow30', 'MKTX  MarketAxess', 'MIR  Mirion Technologies', 'FATH  Fathom Digital Manufacturing', 'BRP  BRP Group', 'ASND  Ascendis Pharma', 'JKS  JinkoSolar', 'ADES  Advanced Emissions', 'NLY  Annaly Capital', 'DEI  Douglas Emmett', 'GOOD  Gladstone', 'ARKK  ARK Innovation ETF', 'ORTX  Orchard Therapeutics', 'CLW  ClearWater Paper', 'CCV  Churchill Capital Corp V', 'EPAC  Actuant', 'FANH  Fanhua', 'FSTX  F star Therapeutics', 'OSCR  Oscar Health', 'CMP  Compass Minerals', 'LTH  Life Time Group Holdings Inc', 'CUBI  Customers Bancorp', 'BRZE  Braze Inc', 'KZIA  Kazia Therapeutics', 'IMUX  Immunic', 'ANSS  ANSYS', '.VOLQ  .VOLQ Index', 'PAGS  PagSeguro Digital', 'ADT  ADT', 'BROS  Dutch Bros', 'IT  Gartner', 'IAS  Intergral Ad Science Holding Corporation', 'BA  Boeing', 'LXFR  Luxfer', 'CBL  CBL &amp; Associates Properties, Inc.', 'CCOI  Cogent', 'IEO  iShares US Oil and Gas Exploration and Production', 'CVS  CVS Health', 'ARKO  ARKO', 'BIB  ProShares Ultra', '.MXEA  MSCI EAFE Index', 'NOG  Northern Oil and Gas', 'CTLT  Catalent', 'AQUA  Evoqua Water Technologies', 'FSMD  Fidelity Small Mid Multifactor ETF', 'ORPHY  Orphazyme A/S (ADR)', 'CXW  CoreCivic', 'FITB  Fifth Third', 'ANNX  Annexon', 'BUSE  First Busey', 'ALSN  Allison Transmission', 'EWH  iShares Hong Kong', 'ANIX  Anixa Biosciences', 'NIC  Nicolet Bankshares Inc', 'IMCR  Immunocore Holdings', 'ISUN  iSun', 'JJM  iPath Bloomberg Industrial Metals ETN', 'LUNG  Pulmonx Corp', 'APTV  Aptiv', 'LEA  Lear', 'FRBA  First Bank', 'CRUZ  Defiance Hotel Airline Cruise ETF', 'NEWYY  Puxin Ltd', 'HZON  Horizon Acquisition Corp', 'HMN  Horace Mann', 'AYRO  AYRO', 'FNCB  FNCB Bancorp', 'NRP  Natural Resource Partners', 'KOP  Koppers Holdings', 'ESE  ESCO Technologies', 'FORG  ForgeRock', 'IYM  iShares DJ US Basic Materials Sector Index Fund', 'OVT  Overlay Shares Short Term Bond ETF', 'HDB  HDFC Bank', 'ACWX  iShares MSCI ACWI', 'EARN  Ellington Residential', 'HLIT  Harmonic', 'GLRE  Greenlight Capital Re', 'CRON  Cronos Group', 'ONEW  OneWater Marine', 'NTP  Nam Tai Property', 'BALL  Ball Corporation', 'FNY  First Trust Mid Cap Growth AlphaDEX Fund', 'HEPA  Hepion Pharmaceuticals', 'NEE  NextEra Energy', 'GOOS  Canada Goose', 'MOMO  Momo', 'IWFH  iShares Virtual Work and Life Multisector ETF', 'BX  Blackstone', 'OGE  OGE Energy', 'LQDI  iShares Inflation Hedged Corporate Bond ETF', 'KLDO  Kaleido Biosciences', 'CSX  CSX', 'PFFV  Global X Variable Rate Preferred ETF', 'FXN  AlphaDEX First Energy', 'ACEV  ACE Convergence Acquisition Corp', 'LSTR  Landstar System', 'CVT  Dragoneer Growth Opportunities', 'ALHC  Alignment Healthcare', 'GNW  Genworth', 'BKR  Baker Hughes', 'GBOX  GreenBox POS', 'IZRL  Ark Israel Innovative Technology ETF', 'PG  Procter & Gamble', 'HESM  Hess Midstream Partners', 'ARKG  ARK Genomic Revolution Multi Sector ETF', 'EWUS  iShares MSCI United Kingdom Small Cap ETF', 'IWM  iShares Russell 2000 ETF', 'BKCC  BlackRock Capital Investment', 'KROS  Keros Therapeutics', 'OZON  Ozon Holdings', 'CRDL  Cardiol Therapeutics', 'NP  Neenah Paper', '.SPIKE  SPIKES Volatility Index', 'ATC  Atotech', 'HCC  Warrior Met Coal', 'DO  Diamond Offshore Drilling Inc', 'DMTK  DermTech', 'ATNF  180 Life Sciences', 'CLAR  Clarus', 'ATHA  Athira Pharma', 'OLMA  Olema Pharmaceuticals', 'PEG  Public Service Enterprise Group', 'FIS  Fidelity National Information Services', 'DY  Dycom', 'BCI  Aberdeen Commodity ETF', 'GIS  General Mills', 'MAXR  Maxar Technologies', 'CINT  Ci&amp;T Inc', 'FENC  Fennec Pharmaceuticals', 'NG  NovaGold', 'DMRC  Digimarc', 'CBAT  CBAK Energy Technology', 'CLOV  Clover Health', 'AXTA  Axalta Coating', 'HYXU  iShares International High Yield Bond ETF', 'AA  Alcoa', 'AMSC  AMSC', 'DT  Dynatrace', 'LULU  Lululemon', 'GSIE  Goldman Sachs ActiveBeta International Equity ETF', 'ATCO  Atlas Corp', 'JXN  Jackson Financial', 'HBM  Hudbay Minerals', "DENN  Denny's", 'CENT  Central Garden & Pet', 'AVT  Avnet', 'INDL  Direxion Daily India Shares', 'IGC  India Globalization Capital', 'ONL  Orion Office REIT', 'OLK  Olink Holding AB', 'MBT  Mobile Telesystems', 'ARKW  ARK ETF Trust', 'ARWR  Arrowhead', 'FTAI  Fortress Transportation', 'IEV  iShares S&P Europe 350 Index Fund', 'CDXS  Codexis', 'EXI  iShares Global Industrials', 'CAR  Avis Budget', 'JSTC  Adasina Social Justice All Cap Global ETF', 'AKUS  Akouos', 'JJE  iPath Series B Energy ETN', 'BRX  Brixmor', 'AMAT  Applied Materials', 'BMO  Bank of Montreal', 'APOG  Apogee Enterprises', 'EWW  EWW', 'COMM  CommScope', 'GTEK  GTEK ETF', 'IFRA  iShares US Infrastructure ETF', 'CHIX  Global X China Financials', 'BIG  Big Lots', 'HAL  Halliburton', 'CND  Concord Acquisition Corp', 'BLU  BELLUS Health', 'BHE  Benchmark Electronics', 'GRN  iPath Series B Carbon ETN', 'MESA  Mesa Air', 'EPHY  Epiphany Technology Acquisition Corp', 'LJPC  La Jolla', 'KPTI  Karyopharm', 'GSHD  Goosehead Insurance', 'BFIX  BFIX ETF', 'IPAR  Inter Parfums', 'MGI  MoneyGram', 'OPK  Opko Health', 'DEM  Wisdom Tree  Markets', 'AVAV  AeroVironment', 'NTNX  Nutanix', 'CYBR  CyberArk', 'CHKP  Check Point', 'ILF  iShares Latin America 40 Index Fund', 'PATH  UiPath', 'E  ENI Spa', 'IHF  iShares DJ US Health Care Providers Index Fund', 'MT  ArcelorMittal', '.HGX  PHLX Housing Sector Index', 'AFTY  CSOP FTSE China A50 ETF', 'LBTYK  Liberty Global', '.SIXI  S&P Industrials Sector Index', 'LFLY  Leafly Holdings', 'JDST  Direxion Junior Gold', 'AMH  American Homes 4 Rent', 'LRCX  Lam Research', 'MTD  METTLER TOLEDO', 'MIRO  MiroMatrix Medical Inc', 'AEM  Agnico Eagle', 'PDSB  PDS Biotechnology Corp', 'APEI  American Public Education', 'IXC  iShares S&P Global Energy Sector Index', 'PKB  PowerShares Dynamic Building and Construction Portfolio', 'MIND  Mitcham Industries', 'IAI  iShares DJ US Broker Dealers Index Fund', 'PCAR  PACCAR', 'DOLE  Dole plc', 'BGRY  Berkshire Grey', 'CHK  Chesapeake Energy', 'AG  First Majestic Silver', 'ATOM  Atomera', 'IDRA  Idera Pharmaceuticals', 'PKE  Park Electrochemical', 'KBWD  PowerShares KBW High Dividend Yield Financial Portfolio', 'KARS  Kraneshares Electric Vehicles and Future Mobility ETF', 'EXAS  EXACT Sciences', 'AUS  Austerlitz Acquisition Corp', 'ENSC  Ensysce Biosciences', 'FFIV  F5 Networks', 'CMA  Comerica', 'KB  KB Financial Group', 'FBIO  Fortress Biotech', 'ONEQ  Fidelity Nasdaq Composite Index', 'ITA  iShares US Aerospace and Defense', 'AYX  Alteryx', 'DOCS  Doximity', 'ATRC  AtriCure', 'DHR  Danaher', 'LPX  Louisiana Pacific', 'IEHS  iShares Evolved US Healthcare Staples ETF', 'BHVN  Biohaven', 'INDA  iShares MSCI India', 'GOEX  Global X Gold Explorers', 'PIPP  Pine Island Acquisition Corp', 'ECNS  iShares China Small Cap', 'FDLO  Fidelity Low Volatility Factor ETF', 'CDAK  Codiak BioSciences', 'OTEX  OpenText', 'CNI  Canadian National', 'CSGS  CSG Systems', 'DLPN  Dolphin Entertainment', 'CMAX  CareMax Inc (Class A Stock)', 'NOTV  Inotiv', 'CG  Carlyle Group', 'ICFI  ICF International', 'CNR  Cornerstone Building', 'FEAM  5E Advanced Materials Inc', 'CEQP  Crestwood Equity', 'PCOM  Points International', 'LVO  LiveOne', 'HAYW  Hayward Holdings', 'CSTM  Constellium', 'HWKN  Hawkins Inc.', 'HIL  Hill International', 'OKTA  Okta', 'IBM  IBM', 'NESR  National Energy Services Reunited', 'BSET  Bassett Furniture', 'LUV  Southwest', 'CEI  Camber Energy', 'KKR  KKR', 'ICD  Independence Contract Drilling', 'CRNC  Cerence', 'FIZZ  National Beverage', 'TUP  Tupperware Brands', 'FRGE  Motive Capital Corp', 'LBRDK  Liberty Broadband', 'NTWK  NetSol Technologies', 'JACK  Jack in the Box', 'ATOS  Atossa Genetics', 'CCRV  iShares Commodity Curve Carry Strategy ETF', 'UE  Urban Edge', 'EBIZ  Global X E commerce ETF', 'DFE  WisdomTree Europe', 'ERII  Energy Recovery', 'NOW  ServiceNow', 'HYDR  Global X Hydrogen ETF', 'PGF  PowerShares Financial Preferred Portfolio', 'FCF  First Commonwealth', 'BNDD  Quadratic Deflation ETF', 'LOPE  Grand Canyon', 'NAUT  Nautilus Biotechnology', 'FL  Foot Locker', 'SPTL  SPDR Portfolio Long Term Treasury ETF', 'SPTM  SPDR Portfolio Total Stock Market ETF', 'IGT  International Game', 'IWN  iShares Russell 2000 Value Index Fund', 'VRRM  Verra Mobility', 'GVIP  Goldman Sachs Hedge Industry Vip ETF', 'NLTX  Neoleukin Therapeutics', 'CERS  Cerus', 'CHDN  Churchill Downs', 'VMI  Valmont', 'SPMD  SPDR S&P 400 Mid Cap ETF', 'EEMS  iShares MSCI Emerging Markets Small Cap', 'SKYW  SkyWest', 'SCHG  Schwab US Large Cap Growth', 'HEFA  iShares Currency Hedged MSCI EAFE', 'CNBS  Amplify Seymour Cannabis ETF', 'ALLO  Allogene Therapeutics', 'AMRK  A-Mark Precious Metals', 'GEHI  Gravitas Education Holdings Inc - ADR', 'RGA  Reinsurance Group', 'GOLD  Barrick Gold', 'RKT  Rocket Companies', 'BASE  Couchbase Inc', 'FOLD  Amicus', 'EEM  iShares Emerging Markets ETF', 'XSPA  XpresSpa', 'FROG  JFrog Ltd', 'BBW  Build A Bear Workshop', 'LICY  Li Cycle Holdings Corp', 'PLXP  PLx Pharma', 'DLTR  Dollar Tree', 'JELD  JELD-WEN Holding', 'IQDF  FlexShares International Quality Dividend Index Fund', 'TCX  Tucows', 'BRK/B  Berkshire Hathaway', 'VTEB  Vanguard Tax Exempt Bond ETF', 'SCHX  Schwab Large Cap', 'R  Ryder', 'FTXH  First Trust Nasdaq Pharmaceuticals', 'FIDU  Fidelity MSCI Industrials Index', 'MNTX  Manitex', 'SPHD  PowerShares S&P 500 High Dividend Low Volatility Portfolio', 'ESNT  Essent Group', 'YY  YY', 'SCM  Stellus Capital', 'XOMA  XOMA', 'AREC  American Resources Corp', 'AMPY  Amplify Energy', 'RTM  Guggenheim S&P 500 Equal Weight Materials', 'EMR  Emerson', 'PSTI  Pluristem', 'ARDS  Aridis Pharmaceuticals', 'KMPR  Kemper', 'JJU  iPath Bloomberg Aluminum ETN', 'BANR  Banner Corp.', 'GS  Goldman Sachs', 'GTYH  GTY Technology (Class A)', 'HSY  Hershey', 'MEME  Roundhill MEME ETF', 'POOL  Pool Corp.', 'CCEP  Coca Cola European Partners', 'IBOC  IBC', '.XEO  European S&P 100 Index', 'IYG  iShares DJ US Financial Services Index Fund', 'TRV  Travelers', 'MTZ  MasTec', 'DOOR  Masonite', 'CVI  CVR Energy', 'IJT  iShares S&P SmallCap 600 Growth Index Fund', 'USDU  WisdomTree Bloomberg US Dollar Bullish Fund', 'APT  Alpha Pro Tech', 'XLC  Communication Services Select Sector SPDR Fund', 'MDY  SPDR S&P MIDCAP 400', 'NFGC  NFGC Stock', 'UNIT  Uniti Group', 'TMHC  Taylor Morrison', 'EDZ  Direxion Emerging Markets Bear 3x', 'ASMB  Assembly Biosciences', 'DAVE  Dave Inc.', 'INDT  INDUS Realty Trust Inc', 'SKYU  ProShares Ultra Nasdaq Cloud Computing ETF', 'PII  Polaris Industries', 'YPS  Arrow Reverse Cap 500 ETF', 'HJEN  Direxion Hydrogen ETF', 'BABA  Alibaba', 'TIPL  TIPL ETF', 'RPRX  Royalty Pharma', 'SPVU  PowerShares S&P 500 Value Portfolio', 'CWEN/A  Clearway Energy Class A', 'HOUS  Anywhere Real Estate Inc', 'IAGG  iShares International Aggregate Bond Fund', 'MATW  Matthews', 'PHAT  Phathom Pharmaceuticals', 'CLFD  Clearfield', 'DECK  Deckers Brands', 'BCE  BCE', 'IMBI  iMedia Brands', 'OSW  OneSpaWorld', 'BZUN  Baozun', 'RYAAY  Ryanair', 'BTTR  Better Choice Company', 'EVGN  Evogene', 'IWIN  IWIN ETF', 'B  Barnes Group', 'CHCO  City Holding', 'AQMS  Aqua Metals', 'MTRX  Matrix Service', 'AGYS  Agilysys', 'TGS  Transportadora de Gas del Sur SA', 'ATO  Atmos Energy', 'MDGL  Madrigal Pharmaceuticals', 'FE  FirstEnergy', 'ENPH  Enphase Energy', 'ABT  Abbott', 'CMBS  iShares CMBS', 'SPWR  SunPower', 'WDIV  SPDR S&P Global Dividend', 'STNG  Scorpio Tankers', 'SFBS  ServisFirst', 'AMCX  AMC Networks', 'JJP  iPath Bloomberg Precious Metals ETN', 'HYLN  Hyliion Holdings', 'TILE  Interface', 'ROCC  Ranger Oil', 'AR  Antero Resources', 'CFRX  ContraFect', 'FRSH  Freshworks', 'GT  Goodyear Tire', 'SEER  Seer', 'EZJ  ProShares MSCI Japan', 'FPX  First Trust US IPO Index Fund', 'BBEU  JPMorgan BetaBuilders Europe', 'IREN  Iris Energy Ltd', 'REPX  Riley Exploration Permian', 'COGT  Cogent Biosciences', 'QRHC  Quest Resource', 'PDBC  PowerShares DB Optimum Yield Diversified Comm Strategy Portfolio', 'AGQ  ProShares Ultra Silver', 'RWX  SPDR International', 'HY  Hyster Yale Materials', 'UHAL  AMERCO', 'PMT  PennyMac Mortgage', 'AVPT  AvePoint', 'XHE  SPDR S&P Health Care Equipment', 'WWW  Wolverine World Wide', 'BK  BNY Mellon', 'SPXZ  Morgan Creek  Exos Spac Originated ETF', 'JBHT  J B Hunt Transport Services', 'ISRA  Market Vectors Israel', 'PYCR  Paycor HCM', 'RLMD  Relmada Therapeutics', 'BTCY  Biotricity', 'SBCF  Seacoast Banking', 'LANC  Lancaster Colony', 'OPCH  Option Care Health', 'MNTV  Momentive Global', 'XYLD  Global X S&P 500 Covered Call ETF', 'RCKT  Rocket Pharmaceuticals', 'ASPY  Asymshares Asymmetric 500 ETF', 'RDN  Radian Group', 'HSBC  HSBC', 'SHYG  iShares 0 5 Year High Yield Corporate Bond', 'VYNE  VYNE Therapeutics', 'STLD  Steel Dynamics', 'NDAQ  Nasdaq', 'DKL  Delek Logistics', 'KEMX  Kraneshares MSCI Emerging Mkts ex China ETF', 'KIND  Nextdoor Holdings', 'CIM  Chimera', 'ETR  Entergy', 'DFH  Dream Finders Homes ', 'CWK  Cushman and Wakefield ', 'DFS  Discover', 'RMNI  Rimini Street', 'ROBO  Robo Stox Global Robotics and Automotion Index', 'REE  REE Automotive', 'CYRX  CryoPort', 'SMWB  Similarweb', 'KFRC  Kforce', 'SPFF  Global X SuperIncome Preferred', 'L  Loews', 'RWO  SPDR DJ Global', 'LDOS  Leidos', 'SLGN  Silgan Holdings', 'MYTE  MYT Netherlands Parent BV', 'USIO  Usio', 'ALNY  Alnylam', 'TYRA  Tyra Biosciences Inc', 'BLDR  Builders First Source', 'PBP  PowerShares S&P 500 Buy Write Portfolio', 'WHR  Whirlpool', 'FTA  First Trust Large Cap Value AlphaDEX Fund', 'LVOX  LiveVox Holding', 'KALU  Kaiser Aluminum', 'VXUS  Vanguard International', 'AWYX  ETFMG 2x Daily Travel Tech ETF', 'CLBK  Columbia Financial', 'CI  Cigna', 'KOS  Kosmos', 'FYBR  Frontier Communications', 'HEEM  iShares', 'XERS  Xeris Pharmaceuticals', 'GLTO  Galecto', 'SFL  Ship Finance', 'IYE  iShares DJ US Energy Sector Index Fund', 'CIA  Citizens', 'RNWK  RealNetworks', 'EIX  Edison International', 'RDVT  Red Violet', 'STOK  Stoke Therapeutics', 'NWL  Newell Brands', 'RENT  Rent the Runway', 'ABTX  Allegiance Bancshares', 'EQNR  Equinor', 'IQM  Franklin Intelligent Machines ETF', 'AGCO  AGCO', 'CVM  Cel Sci', 'TSBK  Timberland Bancorp', 'KT  KT Corporation', 'RYAM  Rayonier Advanced Materials', 'SCCO  Southern Copper', 'ASRT  Assertio Therapeutics', 'APG  APi Group', 'SECT  Main Sector Rotation ETF', 'BCRX  BioCryst', 'ICLN  iShares Global Clean Energy', 'BWMX  Betterware de Mexico SAB de CV', 'XMLV  PowerShares S&P MidCap Low Volatility Portfolio', 'VEGN  US Vegan Climate ETF', 'DH  Definitive Healthcare', 'FIX  Comfort Systems USA', 'OVV  Ovintiv', 'DINO  HollyFrontier', 'FXF  Swiss Franc Trust', 'VLD  Velo3D', 'USO  United States Oil Fund', 'VIOG  Vanguard S&P Small Cap 600 Growth', 'IRBO  iShares Robotics and AI ETF', 'TREE  LendingTree', 'SGOL  ETFS Gold Trust', 'CRBP  Corbus Pharmaceuticals', 'LNC  Lincoln National', 'SCHC  Schwab International', 'PLTR  Palantir Technologies', 'PTH  PowerShares DWA Healthcare Momentum Fund', 'LFTR  Lefteris Acquisition Corp', 'DGL  PowersharesDB Gold', 'MDRX  Allscripts', 'WETF  WisdomTree', 'INVH  Invitation Homes', 'LGMK  NXT-ID', 'CVET  Covetrus', 'ASXC  Asensus Surgical', 'ESSC  East Stone Acquisition Corp', 'CHGG  Chegg', 'WHG  Westwood Holdings Group', 'SCTL  Societal CDMO', 'IOSP  Innospec', 'EMX  Eurasian Minerals', 'LEMB  iShares Emerging Markets', 'OWLT  Sandbridge Aquisition Corp', 'COHR  Coherent', 'TRNO  Terreno Realty', 'TROO  TROOPS Inc', 'UBA  Urstadt Biddle', 'INO  Inovio', 'ITEQ  BlueStar TA BIGITeck Israel Technology', 'TIGR  UP Fintech Holding Ltd', 'VICI  Vici Properties', 'LUNA  Luna Innovations', 'BUD  Anheuser Busch', 'ENOB  Enochian Biosciences', 'UTSL  Direxion Daily Utilities Bull 3x', 'ETRN  Equitrans Midstream', 'ONTX  Onconova', 'MOS  Mosaic', 'H  Hyatt', 'EWMC  Guggenheim S&P MidCap 400 Equal Weight ETF', 'ZEUS  Olympic Steel', 'EHC  Encompass Health', 'TPC  Tutor Perini', 'FLGT  Fulgent Genetics', 'KHC  Kraft Heinz', 'SF  Stifel Financial', 'PPA  PowerShares Aerospace and Defense Portfolio', 'CMCT  CIM Commercial Trust', 'NVO  Novo Nordisk', 'JOE  St Joe Company', 'BCIM  BCIM ETF', 'CVCO  Cavco Industries', 'CVRX  CVRx Inc', 'UCO  ProShares Ultra DJ UBS Crude Oil', 'WHD  Cactus Inc', 'UYM  ProShares Ultra Basic Materials', "KIRK  Kirkland's", 'FREQ  Frequency Therapeutics', 'DAO  Youdao', 'INGN  Inogen', 'MGY  Magnolia Oil and Gas', 'BTBT  Bit Digital', 'TALK  Talkspace', 'KRE  SPDR S&P Regional Banking', 'NTST  NetSTREIT', 'GXC  SPDR S&P China', 'AUDC  AudioCodes', 'SATO  Alerian Galaxy Crypto Economy ETF', 'EVC  Entravision', 'CTVA  Corteva', 'NAT  Nordic American Tankers', 'PGM  iPath Series B Platinum ETN', 'BHR  Braemar Hotels and Resorts', 'ONEY  SPDR Russell 1000 Yield', 'PIZ  PowerShares DWA', 'NRZ  New Residential', 'CDXC  ChromaDex', 'VOYA  Voya Financial', 'NNN  National Retail Properties', 'SKF  ProShares UltraShort Financials', 'GEL  Genesis Energy', 'EQH  AXA Equitable', 'COPX  Global X', 'HCKT  Hackett Group', 'ERIC  Ericsson', 'APLS  Apellis Pharmaceuticals', 'OSUR  OraSure', 'ASPN  Aspen Aerogels', 'MDB  MongoDB', 'SM  SM Energy', 'OTTR  Otter Tail', 'APEN  Apollo Endosurgery', 'ICUI  ICU Medical', 'ADVM  Adverum Biotechnologies', 'CNC  Centene', 'AXNX  Axonics Modulation Technologies', 'SSL  Sasol', 'BOKF  BOK Financial', 'BTG  B2Gold', 'ERY  Direxion Energy Bear 3x', 'ONVO  Organovo', 'SHOO  Steve Madden', 'DCPH  Deciphera Pharmaceuticals', 'HT  Hersha Hospitality', 'PLAB  Photronics', 'WABC  Westamerica', 'GIGB  GS Access Investment Grade Co Bond Etf', 'EES  WisdomTree SmallCap Earnings Fund', 'DDM  Proshares Dow30', 'BIIB  Biogen', 'BND  Vanguard Total Bond', 'RY  Royal Bank of Canada', 'CVLT  CommVault', 'BAB  PowerShares Build America Bond Portfolio', 'PEN  Penumbra', 'VKTX  Viking Therapeutics', 'CRTO  Criteo', 'FDX  Fedex', '.NQX  NASDAQ-100 Reduced Value Index', 'BDX  Becton Dickinson', 'AYTU  AYTU BioPharma', 'WCN  Waste Connections', 'FDRV  Fidelity Electic Vehicle Future Transport ETF', 'VGIT  Vanguard Intermediate Term Government Bond', 'SBH  Sally Beauty', 'LNTH  Lantheus', 'GTE  Gran Tierra', 'POWW  AMMO', 'XLE  Energy Sector SPDR ETF', 'CSCO  Cisco', 'DLTH  Duluth Holdings', 'SMRT  SmartRent', 'FLNT  Fluent', 'RGR  Ruger', 'CPK  Chesapeake Utilities', 'FANG  Diamondback Energy', 'VG  Vonage', 'NVEE  NV5 Global', 'GWX  SPDR International', 'HL  Hecla Mining', 'FYC  First Trust Small Cap Growth AlphaDEX Fund', 'WBS  Webster Financial', 'EFSC  Enterprise Financial', 'UVSP  Univest', 'URA  Global X Uranium', 'PFGC  Performance Food Group', 'TXN  Texas Instruments', 'BTAI  BioXcel Therapeutics', 'LYG  Lloyds Banking', 'DXCM  DexCom', 'RLGT  Radiant Logistics', 'MAG  MAG Silver', 'WEC  WEC Energy', 'GTIM  Good Times Restaurants', 'MSB  Mesabi Trust', 'MEIP  MEI Pharma', 'PSNY  Gores Guggenheim Inc', 'JNJ  Johnson & Johnson', 'CYXT  Cyxtera Technologies', 'PCRX  Pacira', 'CBSH  Commerce Bancshares', 'LAMR  Lamar Advertising', 'ZUO  Zuora', 'GEO  GEO Group', 'BXC  BlueLinx', 'NGG  National Grid', 'VWO  Vanguard MSCI Emerging Markets', 'SPYG  SPDR S&P 500 Growth', 'GBCI  Glacier Bancorp', 'GLW  Corning', 'NMRK  Newmark Group', 'LSXMK  Liberty Media-Series C Stock', 'UHS  Universal Health', 'TNK  Teekay Tankers', 'PAVE  Global X US  Infrastructure Development ETF', 'IQV  IQVIA', 'QDIV  Global X S&P 500 Quality Dividend ETF', 'INDI  indie Semiconductor', 'TPB  Turning Point Brands', 'LX  LexinFintech', 'MMC  Marsh & McLennan', 'PIXY  ShiftPixy', 'RETA  Reata Pharmaceuticals', 'FIGS  FIGS', 'KFVG  Kraneshares Cicc China 5g and Semiconductor Index ETF', 'DRIV  Global X Autonomous and Electric Vehicles ETF', 'FOSL  Fossil Group', 'KELYA  Kelly Services', 'PEO  Petroleum & Resources', 'LGF/A  Lions Gate Entertainment', 'ABNB  Airbnb', 'EVRI  Everi Holdings', 'JAGX  Jaguar Animal Health', 'YYY  YieldShares High Income', 'BRF  Market Vectors Brazil', 'RUN  Sunrun', 'COTY  Coty', 'GSIT  GSI Technology', 'SNV  Synovus', 'HNP  Huaneng Power', 'MC  Moelis', 'BLND  Blend Labs', 'MSOS  Advisorshares Pure Us Cannabis ETF', 'NPTN  NeoPhotonics', 'QRVO  Qorvo', 'TCOM  Trip.com', 'CLF  Cleveland-Cliffs', 'IPI  Intrepid Potash', 'ASTR  Astra Space', 'PNM  PNM Resources', 'VISL  Vislink Technologies', 'SCC  ProShares UltraShort Consumer Services', 'GFI  Gold Fields', 'LTBR  Lightbridge', 'CYBE  CyberOptics', 'GERM  Treatments Testing and Advancements ETF', 'DRD  DRDGold', 'BOOT  Boot Barn', 'TIXT  TELUS International', 'EAF  GrafTech', 'BFIN  BankFinancial', 'BRSP  BrightSpire Capital', 'LCTU  Blackrock Us Carbon Transition Readiness ETF', 'SOGU  Short De-SPAC ETF', 'COLB  Columbia Banking', 'FNDF  Schwab Fundamental International Large Company', 'ALE  Allete', 'TRUE  TrueCar', 'BURL  Burlington Stores', 'RKLY  Rockley Photonics Holdings', 'BKYI  Biokey International', 'INFY  Infosys', 'DWM  WisdomTree International Equity Fund', 'SCSC  ScanSource', 'CDC  Victory CEMP US EQ Income Enhanced Volatility Weighted Index', 'FSTR  L B Foster', 'HYFM  Hydrofarm Holdings Group', 'SLQT  SelectQuote', 'TUYA  Tuya Inc', 'AFYA  Afya Ltd', 'WEAT  Teucrium Wheat', 'ESTA  Establishment Labs', 'PPG  PPG Industries', 'ITOT  iShares Core Total', 'XL  XL Fleet', 'SPG  Simon Property', 'LOOP  Loop Industries', 'TARA  Protara Therapeutics', 'FLYW  Flywire', 'ED  Consolidated Edison', 'WKSP  Worksport Ltd', 'PBH  Prestige Brands', 'MNRO  Monro Muffler', 'TAP  Molson Coors', 'GPL  Great Panther Silver', 'TFLO  iShares Treasury Floating Rate Bond', 'DVY  iShares Div Index', 'SATS  EchoStar', 'STT  State Street', 'ILCB  iShares Morningstar US Equity ETF', 'SMN  ProShares UltraShort Basic Materials', 'FXA  Australian Dollar Trust', 'PRTS  US Auto Parts Network', 'BBJP  JPMorgan BetaBuilders Japan ETF', 'XRX  Xerox', 'SAMG  Silvercrest Asset', 'Y  Alleghany', 'PAWZ  ProShares Pet Care ETF', 'QIWI  QIWI', 'ANGO  AngioDynamics', 'LEV  Lion Electric', 'UTHR  United Therapeutics', 'IART  Integra LifeSciences', 'LMNR  Limoneira', 'SCHA  Schwab US Small Cap', 'HUYA  HUYA', 'AMT  American Tower', 'NERV  Minerva', 'ROM  ProShares Ultra Technology', 'WISA  Summit Wireless Technologies', 'PLCE  Childrens Place', 'RCD  Guggenheim S&P 500 Equal Weight Consumer Discretionary', 'WDNA  WisdomTree BioRevolution ETF', 'BENE  Benessere Capital Acquisition Corp', 'EZPW  EZCORP', 'XHS  SPDR S&P Health Care Services', 'IGV  iShares Goldman Sachs Software Index Fund', 'BRBR  BellRing Brands', 'APO  Apollo Global', 'FDUS  Fidus Investment', 'FAZ   Financial Bear 3X', 'TSLA  Tesla', 'SRE  Sempra Energy', 'OBSV  ObsEva', 'LOUP  Innovator Loup Frontier Tech ETF', 'GDE  WisdomTree Gold Plus ETF', 'XAIR  Beyond Air', 'SAA  ProShares Ultra SmallCap600', 'HTBK  Heritage Commerce', 'DZSI  DASAN Zhone', 'SWN  Southwestern Energy', 'TSM  Taiwan Semi', 'EVOK  Evoke Pharma', 'HCCI  Heritage Crystal Clean', 'ARGX  Argenx', 'PEAR  Pear Therapeutics Inc', 'RYF  Guggenheim S&P 500 Equal Weight Financials', 'DGX  Quest Diagnostics', 'TRS  TriMas', 'CVGW  Calavo Growers', 'SCWX  SecureWorks', 'WEED  WEED ETF', 'ABUS  Arbutus Biopharma', 'NETI  Eneti', 'JRVR  James River Group', 'IOO  iShares S&P 100 Global Index Fund', 'FTCS  First Trust Capital Strength', 'RADI  Radius Global Infrastructure', 'SYF  Synchrony', 'OASQW  OASQW Stock', 'VDC  Vanguard Consumer', 'ASGN  On Assignment', 'BRY  Berry Petroleum', 'MCFT  MCBC Holdings', "PZZA  Papa John's", 'DRTT  Dirtt Environmental Solutions', 'CHRW  CH Robinson Worldwide', 'TVTY  Tivity Health', 'HIMX  Himax', 'ASRV  AmeriServ Financial', 'DLHC  DLH Holdings Corp.', 'IYF  iShares Dow Jones US Finance Sector Index Fund', 'ONCR  Oncorus', 'AIV  Aimco', 'AMRX  Amneal Pharmaceuticals', 'KAMN  Kaman', 'CHNG  Change Healthcare', 'LGF/B  Lions Gate Entertainment', 'SJNK  SPDR Barclays Short Term', 'CVCY  Central Valley Community', 'KURE  KraneShares MSCI All China Health Care Index ETF', 'KVSA  Khosla Ventures Acquisition Corp', 'FOR  Forestar Group', 'KTOS  Kratos Defense', 'ECOM  ChannelAdvisor', 'FISI  Financial Institutions', 'ESAB  ESAB Corp', 'MUB  iShares S&P National AMT Free Municipal Bond Fund', 'PRTK  Paratek Pharmaceuticals', 'HWC  Hancock Whitney', 'ACGL  Arch Capital', 'KOD  Kodiak Sciences', 'PRGO  Perrigo', 'AKAM  Akamai', 'TSN  Tyson Foods', 'BBCA  JPMorgan BetaBuilders Canada', 'CNX  CNX Resources', 'ODP  Office Depot', 'SJM  J.M. Smucker', 'LZ  LegalZoom', 'MCS  Marcus', 'EQX  Equinox Gold', 'CINF  Cincinnati Financial', 'HON  Honeywell', 'DUK  Duke Energy', 'NWN  Northwest Natural Gas', 'LEGN  Legend Biotech', 'STSA  Satsuma Pharmaceuticals', 'SRI  Stoneridge', 'TGTX  TG Therapeutics', 'CANO  Cano Health', 'AZEK  AZEK', 'DIV  Global X SuperDividend US', 'USMC  Principal US Mega Cap Multi Factor Index', 'SPB  Spectrum Brands', 'IDN  Intellicheck', 'NATR  Natures Sunshine Products', 'KLTR  Kaltura', 'HI  Hillenbrand', 'CGEM  Cullinan Oncology', 'FPH  Five Point Holdings', 'FDMO  Fidelity Momentum Factor ETF', 'BETZ  Roundhill Sports Betting', 'TMO  Thermo Fisher', 'SBGI  Sinclair Broadcast', 'MANH  Manhattan Associates', 'HLT  Hilton', 'PBR  Petrobras', 'VMD  VieMed Healthcare', 'CEVA  CEVA', 'QUOT  Quotient Technology', 'FIDI  Fidelity International High Dividend ETF', 'APTS  Preferred Apartment', 'AUY  Yamana Gold', 'AEP  American Electric', 'APDN  Applied DNA Sciences', 'WSM  Williams Sonoma', 'CERT  Certara', 'CHMI  Cherry Hill Mortgage', 'GERN  Geron', 'RPTX  Repare Therapeutics', 'ROKT  SPDR S&P Kensho Final Frontiers ETF', 'FSM  Fortuna Silver Mines', 'FAD  First Trust Multi Cap Growth AlphaDEX Fund', 'OWL  Blue Owl Capital', 'HAS  Hasbro', 'NH  NantHealth', 'VEV  Vicinity Motor Corp', 'FNDX  Schwab Fundamental US Large Company', 'BNS  Scotiabank', 'CPIX  Cumberland', 'RXI  iShares Global Consumer Discretionary', 'RDFN  Redfin', 'CCAP  Crescent Capital', 'ACIU  AC Immune', 'NDLS  Noodles & Company', 'IMNM  Immunome', 'DIDIY  DiDi Global Inc - ADR', 'PRPB  CC Neuberger Principal Holdings II', 'CME  CME Group', 'HUMA  Hertz Global Holdings', 'CLLS  Cellectis', 'SNN  Smith & Nephew', 'DON  WisdomTree Trust', 'SHYF  Shyft Group', 'SLCA  Silica Holdings', 'GMTX  Gemini Therapeutics', 'FXL  AlphaDEX Tech', 'FDN  Trust DJ Index', 'RYI  Ryerson', 'EWV  ProShares Japan', 'KALA  Kala Pharmaceuticals', 'IEI  iShares Barclays 3 to 7 Year Treasury Bond Fund', 'IWB  iShares Russell 1000 Index Fund', 'SACH  Sachem Capital', 'USPH  U.S. Physical Therapy', 'ZROZ  Pimco 25yr Treasury ETF', 'SVVC  Firsthand Technology Value Fund', 'HIG  Hartford Financial', 'ENTG  Entegris', 'IDRV  iShares Self driving ETF', 'IZEA  IZEA', 'PIPR  Piper Sandler', 'SR  Spire', 'ENOV  Enovis Corp', 'GWH  ESS Tech', 'CXSE  WisdomTree China ex State Owned Enterprises Fund', 'RCEL  Avita Medical', 'ICLR  ICON plc', 'NYCB  New York Community Bancorp', 'FCFS  First Cash', 'KRNT  Kornit Digital', 'FELE  Franklin Electric', 'HELE  Helen of Troy', 'III  Information Services', 'HYRE  HyreCar', 'NIO  NIO', 'HRL  Hormel Foods', 'IIPR  Innovative Industrial Properties', 'BIP  Brookfield Infrastructure', 'MDWD  MediWound', 'LOV  Spark Networks', 'ACTG  Acacia Research', 'LPI  Laredo Petroleum', 'ZEST  Ecoark Holdings', 'SCJ  iShares MSCI JaPAN', 'QABA  First Trust Nasdaq ABA Community Bank Index Fund', 'AUTL  Autolus Therapeutics', 'ARVR  ARVR ETF', 'DSX  Diana Shipping', 'SIMO  Silicon Motion', 'AJG  Arthur J. Gallagher', 'UNP  Union Pacific', 'CRU  Crucible Acquisition Corp', 'XPND  First Trust Expanded Technology ETF', 'RVP  Retractable Technologies', 'DAVA  Endava PLC', 'QQQ  Invesco QQQ ETF', 'WDFC  WD 40 Co', 'VNQI  Vanguard Global US', 'HBB  Hamilton Beach Brands', 'EMBK  Embark Technology', 'WDC  Western Digital', 'TIP  iShares Barclays Tips Bond Fund', 'GATX  GATX', 'PTCT  PTC Therapeutics', 'FN  Fabrinet', 'RES  RPC', 'PHX  Panhandle Oil & Gas', 'ZION  Zions Bancorp', 'AMJ  Alerian MLP', 'TXG  10x Genomics', 'AMC  AMC Entertainment', 'UXIN  Uxin Ltd', 'AVCT  American Virtual Cloud Technologies', 'TEO  Telecom Argentina', 'CRCT  Cricut', 'BLCO  Bausch + Lomb Corp', 'ASYS  Amtech Systems', 'WRLD  World Acceptance', 'VTWG  Vangard Russell 2000 Growth', 'NGMS  NeoGames SA', 'SASI  Sigma Labs Inc', 'BLDP  Ballard Power', 'RFP  Resolute Forest', 'IDA  IDACORP', 'BKEP  Blueknight Energy Partners', 'MODV  ModivCare', 'NEXA  Nexa Resources', 'SID  Companhia Siderugica Hacional', 'CWCO  Consolidated Water', 'SPR  Spirit AeroSystems', 'TREC  Trecora Resources', 'LNDC  Landec', 'VRTX  Vertex', 'BVH  Bluegreen Vacations', 'OPY  Oppenheimer Holdings', 'WMG  Warner Music Group', 'EFTR  eFFECTOR Therapeutics', 'SVM  Silvercorp Metals', 'FXI  iShares China Large Cap ETF', 'FIXD  First Trust TCW Opportunistic Fixed Income ETF', 'IS  Iron Source', 'QLD  ProShares Ultra QQQ', 'NRBO  NeuroBo Pharmaceuticals', 'PKW  PowerShares Buyback', 'FEP  First Trust Europe AlphaDEX', 'CYD  China Yuchai', 'TRN  Trinity Industries', 'ASLE  AerSale', 'BW  Babcock & Wilcox', 'PSEP  Innovator S&P 500 Power Buffer ETF', 'GIB  CGI Group', 'ECH  iShares MSCI Chile', 'NEX  NexTier Oilfield Solutions', 'INCO  EG Shares India Consumer', 'VPN  Global X DataCtr REITs Digital Infrastructure', 'HZNP  Horizon Pharma', 'WTRE  WTRE ETF', 'WIRE  Encore Wire', 'HEAR  Turtle Beach', 'CLVR  Clever Leaves', 'VSTO  Vista Outdoor', 'FNTC  FNTC ETF', 'STOR  STORE Capital', 'DICE  Dice Therapeutics', 'BSGM  Biosig Technologies', 'SWK  Stanley Black and Decker', 'ENSV  ENSERVCO', 'DRV  Direxion Bear 3x', 'TRU  TransUnion', 'PWP  Perella Weinberg Partners', 'TBBK  Bancorp', 'DFAR  DFAR ETF', 'GLP  Global Partners', 'DBEF  db X trackers MSCI', 'MJIN  MJIN ETF', "CONN  Conn's", 'PRTY  Party City', 'ECL  Ecolab', 'ACC  American Campus', 'TRYP  Sonicshares Airlines Hotels Cruise Lines ETF', 'ABEO  Abeona Therapeutics', 'AX  Axos Financial', 'MLCO  Melco Resorts', 'USHY  iShares Broad USD High Yield Corporate Bond ETF', 'HYD  Market Vectors High Yield Municipal Index', 'CALT  Calliditas Therapeutics', 'CFG  Citizens Financial', 'EDV  Vanguard Extended Duration Treasury', 'SMHI  SEACOR Marine Holdings', 'ATSG  Air Transport Services', 'IEZ  iShares US Oil Equipment and Services', 'DHS  WisdomTree High Dividend Fund', 'ADMA  Adma Biologics', 'JLL  Jones Lang LaSalle', 'TTMI  TTM Technologies', 'AZZ  AZZ Inc.', 'PRDO  Perdoceo Education', 'MCBC  Macatawa Bank', 'TIMB  TIM SA', 'SPNS  Sapiens', 'FBT  NYSE Biotech Trust', 'CABA  Cabaletta Bio', 'LH  LabCorp', 'SOND  Sonder Holdings', 'JMST  JPMorgan Ultra Short Municipal Income ETF', 'SRNE  Sorrento Therapeutics', 'KVHI  KVH Industries', 'FEMS  First Trust Emerging Markets Small Cap AlphaDEX Fund', 'PSI  PowerShares Dynamic Semiconductors Portfolio', 'LIDR  AEye Inc', 'SUM  Summit Materials', 'ORN  Orion Group', 'VFF  Village Farms International', 'AEL  American Equity', 'RSI  Rush Street Interactive', 'LFG  Archaea Energy', 'SPIP  SPDR Portfolio TIPS ETF', 'AGR  Avangrid', 'CDW  CDW', 'IVOV  Vanguard S&P Mid Cap 400 Value', 'PRFT  Perficient', 'TCHI  TCHI ETF', 'ODTC  Odonate Therapeutics', 'GFL  GFL Environmental', 'ESMT  EngageSmart', 'CEMI  Chembio Diagnostics', 'UBSI  United Bankshares', 'BOIL  ProShares Ultra', 'VIAV  Viavi', 'FBHS  Fortune Brands', 'CIFR  Cipher Mining', 'XBI  SPDR S&P Biotech', 'VIV  Telefonica Brasil', 'SYRS  Syros Pharmaceuticals', 'CEW  Wisdom Tree Emerging Currency', 'PLUS  ePlus', 'GCO  Genesco', 'BYRN  Byrna Technologies', 'ZNTL  Zentalis Pharmaceuticals', 'PPBT  Purple Biotech', 'TH  Target Hospitality Corp', 'SP  SP Plus Corp.', 'XMTR  Xometry Inc', 'ITP  IT Tech Packaging', 'GLT  Glatfelter', 'SPDN  Direxion Daily S&P 500 Bear 1X Shares', 'DIS  Disney', 'BEAM  Beam Therapeutics', 'LASR  nLIGHT', 'SAP  SAP', 'JOBY  Joby Aviation', 'AUD  Audacy', 'BPMC  Blueprint Medicines', 'MRNA  Moderna', 'IMMP  Immutep Ltd ', 'BATT  Amplify Lithium & Battery Tech ETF', 'ARCO  Arcos Dorados', 'CNQ  Canadian Natural', 'MNM  MNM ETF', 'BEPC  Brookfield Renewable', 'BLX  Banco Latinoamericano', 'COM  Direxion Auspice Broad Commodity Strategy ETF', 'CDLX  Cardlytics', 'STZ  Constellation Brands', 'LRND  LRND ETF', 'SWKS  Skyworks', 'CWI  SPDR MSCI ACWI', 'SPLB  SPDR Portfolio Long Term Corporate Bond ETF', 'ASH  Ashland', 'CELU  Celularity', 'BRCC  SilverBox Engaged Merger Corp', 'CHRA  Charah Solutions', 'XSOE  WisdomTree Emerging Mkts exState Owned Enterprises Fund', 'OESX  Orion Energy', 'OBOR  Kraneshares Msci One Belt One Road Index ETF', 'PSCM  PowerShares S&P SmallCap Materials Portfolio', 'DOX  Amdocs', "VSCO  Victoria's Secret", 'ANIP  ANI Pharmaceuticals', 'DBS  Powershares Silver', 'HOLX  Hologic', 'ORA  Ormat Technologies', 'PCYO  Pure Cycle', 'WSO  Watsco', 'PGRU  Bridgetown 2 Holdings', 'GDXJ  GDXJ', 'BBAI  BigBearai Holdings', 'REVG  REV Group', 'XENE  Xenon Pharmaceuticals', 'TARO  Taro Pharmaceutical', 'SOL  ReneSola', 'ISHP  First Trust Nasdaq Retail ETF', 'PYPD  PolyPid', 'HEWJ  iShares MSCI Japan', 'ZEV  Lightning eMotors', 'CSIQ  Canadian Solar', 'LGHL  Lion Group Holding Ltd', 'GBUG  Pacer  iPath Gold ETN', 'EVBG  Everbridge', 'DNOW  NOW Inc.', 'SITC  SITE Centers', 'ONB  Old National Bancorp', 'SMLV  SPDR Russell 2000 Low Volatility ETF', 'CRGY  Crescent Energy', 'AZRE  Azure Power', 'KYMR  Kymera Therapeutics', 'LBRT  Liberty Oilfield Services', 'BBH  Market Vectors Biotech', 'SPT  Sprout Social', 'PLUG  Plug Power', 'SLI  Standard Lithium', 'WGMI  Valkyrie Bitcoin Miners ETF', 'TRIRF  Triterras', 'CVLG  Covenant Logistics', 'AON  Aon PLC', 'SCVL  Shoe Carnival', 'LSXMA  Liberty Media Corp-Series A Stock', 'BRMK  Broadmark Realty', 'DBX  Dropbox', 'EVLO  Evelo Biosciences ', 'INFN  Infinera', 'CULP  Culp', 'RCM  RCM Technologies', 'SCHY  Schwab International Dividend Equity ETF', 'UFCS  United Fire Group', 'SSYS  Stratasys', 'CAPR  Capricor Therapeutics', 'NSC  Norfolk Southern', 'PNFP  Pinnacle Financial', 'AAL  American Airlines', 'TILL  TILL ETF', 'WOOF  Petco', 'HFFG  HF Foods Group', 'HTLF  Heartland Financial', 'RVLV  Revolve Group', 'XHR  Xenia Hotels & Resorts', 'FCEL  FuelCell', 'LGO  Largo Resources', 'IGIB  iShares Intermediate Term Corporate Bond ETF', 'PANW  Palo Alto Networks', 'SQM  SQM', 'UXI  ProShares Ultra Industrials', 'TRC  Tejon Ranch', 'AFL  AFLAC', 'PINC  Premier', 'MSP  Datto Holdings', 'XBTF  VanEck Bitcoin Futures ETF', 'NFTZ  Defiance NFT Focused ETF', 'IAC  IAC', 'AHCO  AdaptHealth', 'AIR  AAR Corp.', 'ZHDG  Zega Buy and Hedge ETF', 'FXO  Financial AlphaDEX', 'FLBR  Franklin FTSE Brazil ETF', 'ROIC  Retail Oppurtunity Investments', 'ORCL  Oracle', 'IGSB  iShares Short Term Corporate Bond ETF', 'CRTD  Creatd', 'PDP  PowerShares DWA Technical Leaders Portfolio', 'VICE  AdvisorShares Vice ETF', 'FPI  Farmland Partners', 'SCHO  Schwab Short Term US Treasury', 'HLGN  Heliogen', 'PAM  Pampa Energia S.A.', 'SGTX  Sigilon Therapeutics', 'XLP  Consumer Staples Sector SPDR ETF', 'NKTR  Nektar', 'SVFA  SVF Investment Corp', 'EWRE  Guggenheim S&P 500 Equal Weight Real Estate ETF', 'ETN  Eaton', 'RNG  Ringcentral', 'CLSD  Clearside Biomedical', 'SCHM  Schwab Mid Cap', 'SBOW  SilverBow Resorces', 'EYE  National Vision Holdings', 'LFC  China Life Insurance', 'VICR  Vicor Corporation', 'DTC  Solo Brands Inc', 'GLPG  Galapagos NV', 'ORGO  Organogenesis Holdings', 'FBK  FB Financial', 'BCEL  Atreca', "LOW  Lowe's", 'TIG  Trean Insurance Group', 'MAX  MediaAlpha', 'ESGE  iShares MSCI EM ESG Select', 'QUAD  Quad Graphics', 'RMTI  Rockwell Medical', 'UDOW  ProShares UltraPro Dow 30', 'BGS  B&G Foods', 'VRCA  Verrica Parmaceuticals', 'DRVN  Driven Brands Holdings', 'IMCC  IM Cannabis', 'IPA  ImmunoPrecise Antibodies', 'FRHC  Freedom Holding Corp', 'KEP  Korea Electric Power', 'TM  Toyota', 'WBX  Wallbox', 'YEXT  Yext', 'NWG  NatWest Group', 'OFG  OFG Bancorp', 'CRS  Carpenter Technology', 'INT  World Fuel Services', 'EVH  Evolent Health', 'WAT  Waters', 'RDY  Dr Reddys Laboratories', 'LITE  Lumentum', 'ROP  Roper Technologies', 'RIGZ  Viridi Cleaner Energy Crypto Mining and Semi ETF', 'WNDY  Global X Wind Energy ETF', 'PAYC  Paycom', 'HII  Huntington Ingalls', 'BG  Bunge', 'MOV  Movado', 'SABR  Sabre', 'ZD  Ziff Davis', 'TGNA  TEGNA', 'LBRDA  Liberty Broadband', 'BOC  BOSTON OMAHA Corp', 'ANPC  AnPac Bio Medical Science Company', 'FTXL  First Trust Nasdaq Semiconductor', 'GHL  Greenhill & Co.', 'TCS  Container Store', 'DAC  Danaos', 'STIP  iShares 0 to 5 Year TIPS Bond', 'KMX  CarMax', 'GREK  GlobalFTSE Greece', 'ARI  Apollo Commercial', 'MNTK  Montauk Renewables Inc', 'FTXN  First Trust Nasdaq Oil and Gas', 'EXTN  Exterran Corp', 'LFT  Lument Finance Trust', 'OXSQ  Oxford Square Capital', 'PSR  PowerShares Active US Real Estate Fund', 'TRVG  trivago', 'FEM  First TrustAlphaDEX', 'LFUS  Littelfuse', 'MITC  MeaTech 3D', 'VALE  Vale', 'LEAP  Ribbit LEAP Ltd', 'CTMX  CytomX Therapeutics', 'ALG  Alamo Group', 'TELL  Tellurian', 'EFA  EFA', 'SLG  SL Green Realty', 'NTES  NetEase', 'PB  Prosperity Bancshares', 'SBAC  SBA Communications', 'RPAY  Repay Holdings', 'VATE  Innovate', 'RPD  Rapid7', 'BRAG  Bragg Gaming Group', 'FTCV  FinTech Acquisition Corp V', 'GRC  Gorman Rupp Co', 'DSGN  Design Therapeutics', 'MG  Mistras', 'NLOK  NortonLifeLock', 'MAA  MAA', 'BORR  Borr Drilling Limited', 'PGJ  PowerShares Golden Dragon China Portfolio', 'RICK  RCI Hospitality', 'ABC  AmerisourceBergen', 'MCRB  Seres Therapeutics', 'GIC  Global Industrial', 'BSV  Vanguard Short Bond', 'MGC  Vanguard Mega Cap', 'CMPS  COMPASS Pathways PLC', 'OTLY  Oatly Group AB', 'SPVM  Invesco S&P 500 Value with Momentum ETF', 'RANI  Rani Therapeutics Holdings Inc (Class A Stock)', 'IMKTA  Ingles Markets Incorporated', 'BRKL  Brookline Bancorp', 'NSR  Nomad Royalty Company', 'RVNC  Revance', 'HDRO  Defiance Next Gen H2 ETF', 'ONCT  Oncternal Therapeutics', 'ASUR  Asure Software', 'RVSB  Riverview Bancorp', 'SCZ  iShares MSCI EAFE Small Cap Index Fund', 'KCCA  Kraneshares Calif Carbon Allowance ETF', 'KSA  iShares MSCI Saudi Arabia Capped ETF', 'VXX  VIX ShortTerm Futures ETN', 'HOOD  Robinhood Markets', 'DMAT  DMAT ETF', 'CAPL  CrossAmerica', 'TNP  Tsakos Energy', 'IQDG  WisdomTree International Quality Dividend Growth Fund', 'VNO  Vornado', 'IQ  iQiyi', 'UAL  United Continental', 'JTKWY  Just Eat Takeaway.com NV', 'CHPT  ChargePoint Holdings', 'ALV  Autoliv', 'MREO  Mereo BioPharma Group', 'SAND  Sandstorm Gold', 'K  Kellogg', 'BCPC  Balchem', 'NVTS  Navitas Semiconductor', "WEN  Wendy's", 'AIRC  Apartment Income REIT', 'VWE  Vintage Wine Estates', 'DEO  Diageo', 'RXDX  Prometheus Biosciences', 'AMPL  Amplitude', 'MMS  MAXIMUS', 'OCFC  OceanFirst Financial', 'EMXC  iShares MSCI Emerging Markets Ex China', 'GLPI  Gaming and Leisure Properties', 'NGD  New Gold', 'EQR  Equity Residential', 'CIBR  NASDAQ Cybersecurity', 'KAR  KAR Auction Services', 'AEHR  Aehr Test Systems', 'HCAT  Health Catalyst', 'RNR  RenaissanceRe', 'FUN  Cedar Fair', 'GH  Guardant Health', 'NIB  iPath Cocoa', 'KMB  Kimberly Clark', 'MFC  Manulife Financial', 'HEP  Holly Energy', 'RWJ  RevenueShares Small', 'GMGI  Golden Matrix Group Inc', 'DXJ  Widsomtree Japan', 'CHEF  Chefs Warehouse', 'TPTX  Turning Point Therapeutics', 'SPXT  ProShares S&P 500 ExTechnology ETF', 'HTBI  HomeTrust', 'MUFG  Mitsubishi UFJ Financial Group', 'NEWR  New Relic', 'BKLN  Powershares Loan ETF', 'HAAC  Health Assurance Acquisition Corp', 'ELP  Companhia Parana', 'CMCO  Columbus McKinnon', 'SCHV  Schwab US  Large Cap', 'LBTYA  Liberty Global', 'PANL  Pangaea Logistics', "BLMN  Bloomin' Brands", 'ELF  E.l.f. Beauty', 'FMS  Fresenius Medical', 'ENFR  Alerian Energy Infrastructure', 'HNST  Honest Company', 'HYMB  Nuveen S&P High Yield', 'PRG  PROG Holdings', 'NICE  NICE Systems', "DDS  Dillard's", 'MTUM  iShares MSCI USA Momentum Factor', 'SGC  Superior Uniform', 'SMOG  VanEck Vectors Low Carbon Energy ETF', 'ONEM  1Life Healthcare', 'WFC  Wells Fargo', 'GLOP  GasLog Partners', 'KERN  Akerna Corp', 'IIVI  II VI', 'SDIV  Global X', 'PRSO  MoSys', 'VIGI  Vanguard International Dividend Appreciation ETN', 'TSEM  Tower Semiconductor', 'FLS  Flowserve', 'PLTK  Playtika Holding Corp', 'API  Agora', 'MTW  Manitowoc', 'EGLE  Eagle Bulk Shipping', 'HST  Host Hotels', 'AVUS  Avantis US Equity ETF', 'MDVL  MedAvail Holdings', 'UUP  PowerShares DB US Dollar Index Bullish Fund', 'ORI  Old Republic', 'IYJ  iShares Industrial Index', 'EEFT  Euronet', 'ML  Moneylion', 'BF/B  Brown Forman', '.XDZ  PHLX New Zealand Dollar Index', 'TMDX  TransMedics Group', 'BBWI  Bath & Body Works', 'RSVR  Reservoir Media Inc', 'ME  23andMe', 'TCMD  Tactile Systems', 'RACE  Ferrari', 'AKTS  Akoustis', 'VGSH  Vanguard Short Term Government Bond', 'BXP  Boston Properties', 'ATMP  Barclays ETN Plus Select MLP', 'VTI  Vanguard Total Stock Market ETF', 'LPRO  Open Lending Corp', 'OPAD  Offerpad Solutions', 'RBOT  Vicarious Surgical', 'NEXT  NextDecade Corp', 'RUBY  Rubius Therapeutics', 'TAST  Carrols Restaurant', 'CTXR  Citius Pharmaceuticals', 'GRFS  Grifols', 'WEJO  Wejo Group', '.SPESG  S&P 500 ESG INDEX', 'CHIM  Global X China Materials', 'PRAX  Praxis Precision Medicines', 'MHK  Mohawk', 'NEP  NextEra Energy Partners', 'SLX  Market Vectors Steel Index Fund', 'AIRG  Airgain', 'NAK  Northern Dynasty', 'VRSN  VeriSign', 'IXJ  iShares Global Healthcare', 'CNO  CNO Financial', 'GLCN  VanEck China Growth ETF', 'FCA  China AlphaDEX ETF', 'ELDN  Eledon Pharmaceuticals', 'SHAK  Shake Shack', 'NANR  SPDR S&P North American Natural Resources ETF', 'BVS  Bioventus', 'PFC  Premier Financial', 'PRK  Park National', 'ABBV  AbbVie', 'URNM  Northshore Global Uranium Mining ETF', 'DSPC  De-SPAC ETF', 'IPOS  Renaissance International IPO', 'STAR  iStar', 'SENS  Senseonics', 'SURF  Surface Oncology ', 'FISV  Fiserv', 'GWRS  Global Water Resources', 'RELY  Remitly Global', 'HDSN  Hudson Technologies', 'MRAM  Everspin Technologies', 'FHTX  Foghorn Therapeutics', 'ALKT  Alkami Technology', 'TU  Telus', 'LPG  Dorian LPG', 'LXRX  Lexicon Pharmaceuticals', 'ICL  Israel Chemicals', 'FRSX  Foresight Autonomous Holdings', 'BDTX  Black Diamond Therapeutics', 'BFIT  Global X Health and Wellness Thematic', 'DB  Deutsche Bank', 'URGN  UroGen Pharma', 'PXD  Pioneer Natural', 'AMPS  Altus Power', 'THC  Tenet', 'XNTK  SPDR NYSE Technology', 'AER  Aercap', 'ALEX  Alexander & Baldwin', 'NOVT  Novanta', 'BDC  Belden', 'PIRS  Pieris Pharmaceuticals', 'WSBC  WesBanco', 'CC  Chemours', 'QDF  FlexShares Quality Dividend Index Fund', 'URI  United Rentals', 'DSEY  Diversey Holdings', 'BXMT  Blackstone Mortgage', 'LILM  Lilium NV', 'TUFN  Tufin Software', 'VIXY  ProShares VIX Short Term Futures', 'APP  Applovin', 'BSX  Boston Scientific', 'MFIN  Medallion Financial', 'VIR  Vir Biotechnology', 'ALKS  Alkermes', 'TSAT  Telesat', 'AJX  Great Ajax', 'AMWL  American Well Corp', 'NAAS  RISE Education Cayman Ltd (ADR)', 'CBAY  CymaBay', 'SCHN  Schnitzer Steel', 'DFSV  DFSV ETF', 'GENY  Principal Millennials Index', 'VEC  Vectrus', 'JBLU  JetBlue', 'SPWH  Sportsmans Warehouse', 'SRLN  SPDR Blackstone GSO Senior Loan ETF', 'IVE  iShares S&P 500 Value Index Fund', 'YANG  Direxion Daily China Bear 3X Shares', 'LE  Lands End', 'SPGM  SPDR Portfolio MSCI Global Stock Market ETF', 'MFG  Mizuho Financial Group', 'NAVI  Navient', 'CDTX  Cidara Therapeutics', 'RHI  Robert Half', 'AEIS  Advanced Energy', 'CLDX  Celldex', 'MASS  908 Devices', 'VMEO  Vimeo', 'T  AT&T', 'SMFR  Sema4 Holdings', 'RVPH  Reviva Pharmaceuticals Holdings', 'VYGG  Vy Global Growth', 'CALF  Pacer US Small Cap Cash Cows 100', 'ARBK  Argo Blockchain', 'AOUT  American Outdoor Brands', 'NOBL  ProShares 500 Aristocrats', 'PULS  Pgim Ultra Short Bond ETF', 'UBER  Uber', 'MGV  Vanguard Mega Cap Value', 'GRWG  Growgeneration', 'JSPR  Jasper Therapeutics', 'FDL  Morningstar Dividend', 'CAJ  Canon', 'TLSA  Tiziana Life Sciences', 'EWG  iShares Germany', 'CPSI  Computer Programs', 'AGM  Farmer Mac', 'ISPO  Inspirato', 'MSFT  Microsoft', 'BTX  Brooklyn ImmunoTherapeutics', 'RPAR  Rpar Risk Parity ETF', 'HIMS  Hims and  Hers Health', 'BITO  Proshares Bitcoin ETF', 'FLNC  Fluence Energy', 'BKH  Black Hills', 'ONDS  Ondas Holdings', 'QYLG  Global X Nasdaq 100 Covered Call and Growth ETF', 'OLP  One Liberty Properties', 'BLDE  Blade Air Mobility', 'TMP  Tompkins Financial', 'LEVI  Levi Strauss', 'MFH  Mercurity Fintech Holdings', 'IBLC  IBLC ETF', 'RJN  AB Svensk Ekportkredit', 'FAB  First Trust Multi Cap Value AlphaDEX Fund', 'MUX  McEwen Mining', 'CMTL  Comtech', 'IWX  iShares Russell Top 200 Value', 'CBOE  CBOE Holdings', 'DAPP  VanEck Vectors Digital Transformation', 'PYZ  PowerShares DWA Basic Materials Momentum Portfolio', 'CAMT  Camtek', 'BANX  StoneCastle', 'MED  Medifast', 'YPF  YPF', 'HEI  HEICO', 'OCGN  Ocugen', 'RCON  Recon Technology', 'LAC  Lithium Americas', 'IBBQ  Invesco Nasdaq Biotechnology ETF', 'SGG  iPath Sugar ETN', 'IJK  iShares S&P MidCap 400 Growth Index Fund', 'XLRE  Real Estate SPDR', 'XDAT  Franklin Exponential Data ETF', 'UIS  Unisys', 'IGMS  IGM Biosciences', 'BOX  Box', 'OSG  Overseas Shipholding', 'EXC  Exelon', 'LOPX  Direxion Low Priced Stock ETF', 'FAS  FAS', 'TS  Tenaris', 'SCU  Sculptor Capital Management', 'LXP  Lexington Realty', 'EVA  Enviva', 'CWEN  Clearway Energy', 'WMT  Wal-Mart', 'IEFN  iShares Evolved US Financials ETF', 'CDMO  Avid Bioservices', 'RWLK  ReWalk Robotics', 'ADBE  Adobe', 'LEN  Lennar', 'MIDU  Direxion Daily Mid Cap Bull', 'MOGO  Mogo', 'WTFC  Wintrust Financial', 'THFF  First Financial', 'W  Wayfair', 'PRGS  Progress Software', 'LYTS  LSI Industries', 'MDLZ  Mondelez', 'IPOF  Social Capital Hedosophia VI', 'HOG  Harley Davidson', 'HCIC  Hennessy Capital Investment Corp V', 'CTAS  Cintas', 'IHE  iShares US Pharmaceuticals', 'QUAL  iShares MSCI USA Quality', 'VTRS  Viatris', 'DVYE  iShares Emerging Markets Dividend', 'MTG  MGIC Investment', 'TQQQ  ProShares UltraPro QQQ', 'TCRT  ZIOPHARM Oncology', 'PRPO  Precipio', 'CFR  Cullen Frost Bankers', 'DOUG  Douglas Elliman', 'MATX  Matson', 'GVI  iShares Intermediate', 'XLB  Materials Sector SPDR ETF', 'LDI  loanDepot', 'RPT  Ramco Gershenson Properties Trust', 'FNV  Franco-Nevada', '.SPX  S&P 500 Index', 'PGX  PowerShares Preferred', 'ALGS  Aligos Therapeutics', 'ALPP  Alpine 4 Holding', 'RYN  Rayonier', 'MNOV  Medicinova', 'EUM  Proshares Emerging Markets', 'BWAC  Better World Acquisition Corp', 'ASHS  X Trackers China', 'HUSA  Houston American Energy', 'MTA  Metalla Royalty & Streaming', 'UPLD  Upland Software', 'CATH  Global X S&P 500 Catholic Values', 'NRXP  NRX Pharmaceuticals', 'MGTA  Magenta Therapeutics', 'OSBC  Old Second Bancorp', 'LNW  Scientific Games Corp', 'FPE  First Trust Preferred Securities and Income', 'CNCE  Concert Pharmaceuticals', 'IPO  Renaissance IPO', 'ARQT  Arcutis Biotherapeutics', 'FSLY  Fastly Inc', 'JCI  Johnson Controls', 'HZN  Horizon Global', 'QQQJ  Invesco NASDAQ Next Gen 100 ETF', 'PSY  Defiance Next Gen Altered Experience ETF', 'BMRN  BioMarin', 'MRCC  Monroe Capital', 'NKLA  Nikola', 'OMER  Omeros', 'BKF  iShares MSCI', 'OACP  OACP ETF', 'INTC  Intel', 'AUTO  AutoWeb', 'TIME  TIME ETF', 'HRT  HireRight Holdings Corp', 'AUMN  Golden Minerals', 'UAMY  US Antimony', 'MCG  Membership Collective Group', 'AIZ  Assurant', 'RNLX  RenalytixAI PLC', 'WOR  Worthington', 'BSRR  Sierra Bancorp', 'SFT  Shift Technologies', 'COLM  Columbia Sportswear', 'FXG  Consumer Staples AlphaDEX', 'LCTX  Lineage Cell Therapeutics', 'MEG  Montrose Environmental Group', 'KRC  Kilroy Realty', 'NAVB  Navidea', 'BKU  BankUnited', 'ROLL  RBC Bearings', 'VBR  Vanguard Small Cap Value ETF', 'EH  EHang Holdings Ltd', 'CBRE  CBRE Group', 'IMPP  Imperial Petroleum', 'SST  System1', 'AGRI  AgriFORCE Growing Systems Ltd', 'IYC  iShares US Consumer Services', 'LKQ  LKQ Corp.', 'FNK  First Trust Mid Cap Value AlphaDEX Fund', 'ARCC  Ares Capital', 'GOAU  US Global Gold and Precious Metal Miners ETF', 'BNGE  First Trust S network Streaming and Gaming Etf', 'XVOL  Acruence Active Hedge US Equity ETF', 'FCNCA  First Citizens', 'ITUB  Itau Unibanco', 'LAUR  Laureate Education', 'GNCA  Genocea', 'SRLP  Sprague Resources', 'V  Visa', 'CRWD  CrowdStrike Holdings', 'MTEM  Molecular Templates', 'IHAK  iShares Cybersecurity and Tech ETF', 'HMC  Honda', 'PSJ  Power Shares Dynamic Software Portfolio', 'MJXL  ETFMG 2x Daily Alternative Harvest ETF', 'MNDO  MIND C.T.I.', 'ALLY  Ally Financial', 'FLO  Flowers Foods', 'RSG  Republic Services', 'SWIR  Sierra Wireless', 'SNPO  Snap One Holdings', 'SCS  Steelcase', 'AVDV  Avantis International Small Cap Value ETF', 'AMRS  Amyris', 'STRO  Sutro Biopharma', 'EMKR  EMCORE', 'GIII  G-III Apparel', 'LAB  Standard Biotools Inc', 'PTPI  Petros Pharmaceuticals', 'RELL  Richardson Electronics', 'VTV  Vanguard Value', 'PCH  Potlatch', 'SILV  SilverCrest Metals', 'NMR  Nomura', 'TV  Grupo Televisa', '.XDN  PHLX Yen Index', 'KTB  Kontoor Brands', 'AY  Atlantica Yield', 'LNN  Lindsay', 'TBT  ProShares UltraShort Lehman 20 plus Year Treasury', 'CGNT  Cognyte Software', 'CHH  Choice Hotels', 'CSH  CSH ETF', 'HCA  HCA Holdings', 'BTN  Ballantyne Strong', 'KEYS  Keysight Technologies', 'IYH  iShares DJ US Healthcare Sector Index Fund', 'FIVN  Five9', 'BMY  Bristol Myers Squibb', 'VTOL  Bristow Group', 'DFUS  Dimensional US Equity ET', 'FULC  Fulcrum Therapeutics', 'CMCL  Caledonia Mining Corp', 'SGDJ  Sprott Junior Gold Miners ETF', 'EQOS  Diginex', 'CPS  Cooper Standard Holdings', 'TITN  Titan Machinery', 'RGNX  REGENXBIO', 'ESI  Element Solutions', 'ES  Eversource', 'DAM  VanEck Digital Assets Mining ETF', 'CAC  Camden National', 'TXT  Textron', 'PWB  Powershares Dynamic Large Cap Growth Portfolio', 'VXZ  VIX MidTerm Futures ETN', 'DLB  Dolby', 'BP  BP', 'SRG  Seritage Growth Properties', 'ASTE  Astec', 'ABCL  AbCellera Biologics', 'GMF  SPDR S&P Emerging Asia Pacific', 'XHB  SPDR S&P Homebuilders', 'OPRA  Opera Ltd', 'KDNY  Chinook Therapeutics', 'TRST  Trustco Bank', 'SPMB  SPDR Portfolio Mortgage Backed Bond ETF', 'FARO  FARO Technologies', 'DAIO  Data I/O', 'CURE  Direxion Healthcare Bull 3x', 'EVTL  Vertical Aerospace', 'SGFY  Signify Health', 'NEAR  iShares Short Maturity Bond ETF', 'SBUX  Starbucks', 'MKSI  MKS Instruments', 'DSI  iShares MSCI KLD 400 Social Index Fund', 'PHAS  PhaseBio Pharmaceuticals', 'INQQ  INQQ ETF', 'BHC  Bausch Health', 'CCSI  Consensus Cloud Solutions', 'IWO  iShares Russell 2000 Growth Index Fund', 'EDR  Endeavor Group Holdings', 'TUEM  Tuesday Morning Corp', 'EC  Ecopetrol', 'APTX  Aptinyx', 'UROY  Uranium Royalty Corp', '.NYFANG  NYSE FANG+ Index', 'IMV  IMV', 'AIRR  First Trust RBA American Industrial Renaissance', 'MMYT  MakeMyTrip', 'FSK  FS KKR Capital', 'OSTK  Overstock com', 'OYST  Oyster Point Pharma', 'DAKT  Daktronics', 'FTV  Fortive', 'ADMP  Adamis Pharmaceuticals', 'CMRX  Chimerix', 'TTGT  Tech Target', 'KRNY  Kearny Financial', 'VOOV  Vanguard S&P 500 Value', 'BCDA  BioCardia', 'ACHR  Archer Aviation', 'CIG  CEMIG', 'FDRR  Fidelity Dividend ETF for Rising Rates', 'HAP  Market Vectors Natural Resources', 'MILE  MetroMile', 'SGML  SGML Stock', 'COMP  Compass Inc', 'DFEN  Direxion Aerospace and Defense Bull 3x Shares', 'CP  Canadian Pacific', 'TGH  Textainer', 'FAF  First American', 'FV  Dorsey Wright Focus', 'TBLA  Taboola.com', 'XPEL  XPEL', 'SPXE  ProShares S&P 500 Ex Energy ETF', 'AIQ  Global X Future Analytics Tech ETF', 'PRLB  Proto Labs', 'FBNC  First Bancorp', 'KBWB  PowerShares KBW Bank Portfolio', 'FERG  Ferguson Plc', 'EBON  Ebang International', 'UCBI  United Community Banks', 'FCBC  First Community', 'CORT  Corcept Therapeutics', 'FSLR  First Solar', 'ABOS  Acumen Pharmaceuticals', 'APYX  Apyx Medical', 'USB  US Bancorp', 'CARR  Carrier Global', 'NYC  New York City REIT', 'ITGR  Integer', 'MX  MagnaChip', 'WFRD  Weatherford International', 'ELV  Anthem Inc', 'OUSM  OShares US Small Cap Quality Dividend ETF', 'STPZ  PIMCO 1 to 5 Year US TIPS Index ETF', 'IYR  iShares DJ US Real Estate Index Fund', 'PCY  PowerShares Emerging Markets Sovereign Debt Portfolio', 'PRMW  Primo Water Corp', 'LIT  Global X Lithium', 'METV  Roundhill Ball Metaverse ETF', 'PEB  Pebblebrook Hotel', 'DGT  SPDR', 'IPG  Interpublic Group', 'LQDH  iShares Interest Rate Hedged Corporate Bond', 'VERX  Vertex', 'AVAH  Aveanna Healthcare Holdings', 'TDIV  First Trust NASDAQ', 'ZVO  Zovio', 'CGC  Canopy Growth', 'ZLAB  Zai Lab Ltd', 'ZKIN  ZK International Group Ltd', 'ENZL  iShares MSCI New Zealand', 'SGU  Star Gas Partners, L.P.', 'GSP  Barclays Bank Plc', 'WVE  WAVE Life Sciences', 'LSPD  Lightspeed POS', 'META  Meta Platforms Inc', 'TNA  Direxion Daily Smal Cap Bull 3X Shares', 'ONTF  ON24', 'RYLD  Global X Russell 2000 Covered Call ETF', 'WM  Waste Management', 'EPR  EPR Properties', 'NSIT  Insight Enterprises', 'XPRO  Expro Group Holdings NV', 'WNEB  Western New England Bancorp', 'AFRM  Affirm Holdings', 'ICPT  Intercept', 'VRM  Vroom', '.SIXRE  S&P Real Estate', 'VSS  Vanguard FTSE', 'PM  Philip Morris', 'VOR  Vor Biopharma', 'OMCL  Omnicell', 'EOSE  B Riley Principal Merger Corp II', 'MTB  M&T Bank', 'DCT  Duck Creek Technologies', 'HPK  HighPeak Energy Inc', 'OTIS  Otis Worldwide', 'EPD  Enterprise Products', 'BIO  Bio Rad', 'FTXG  First Trust Nasdaq Food and Beverage', 'EAGG  iShares ESG Aware US Aggregate Bond ETF', 'WLK  Westlake Chemical', 'AI  C3 AI', 'EBLU  Tortoise Global Water ESG ETF', 'WES  Western Gas Partners', 'IBKR  Interactive Brokers', 'PACW  PacWest Bancorp', 'WTRG  Essential Utilities', 'GEVO  Gevo', 'MGTX  MeiraGTx Holdings', 'RMO  Romeo Power', 'AQGX  AI Quality Growth ETF', 'AQST  Aquestive Therapeutics', 'FIBK  First Interstate', 'ACRS  Aclaris Therapeutics', 'BGXX  Bright Green Corp', 'GNPX  Genprex', 'ORC  Orchid Island Capital', 'ACET  Adicet Bio', 'ABST  Absolute Software Corp', 'LEGR  First Tr Innovative Transaction and Process ETF', 'FBMS  First Bancshares', 'DBE  Powershares Energy', 'CGBD  TCG BDC Inc', 'VIRX  Viracta Therapeutics', 'ZEN  Zendesk', 'SKYX  SQL Technologies Corp', 'COF  Capital One', 'KNOP  KNOT Offshore Partners', 'CPER  United States Copper Index Fund', 'PHG  Philips', 'ZIP  ZipRecruiter', 'PHO  PowerShares Water Resources Portfolio', 'UFI  Unifi', 'OPRT  Oportun Financial', 'VOOG  Vanguard S&P 500 Growth', 'THD  iShares MSCI Thailand', 'BCS  Barclays', 'CTRM  Castor Maritime', 'AAPL  Apple', 'OSK  Oshkosh', 'PKI  PerkinElmer', 'WNC  Wabash', 'CCI  Crown Castle', 'IETC  iShares Evolved US Technology ETF', 'RESI  RESI ETF', 'YELL  Yellow Corp', "AAN  Aaron's", 'KIM  Kimco Realty', 'OPTN  OptiNose', 'ATNM  Actinium Pharmaceuticals', 'SMAR  Smartsheet', 'EDUT  Global X Education ETF', 'ETSY  Etsy', 'ENDP  Endo', 'DK  Delek US', 'PSTH  Pershing Square Tontine Holdings', 'HOLI  Hollysys', 'REI  Ring Energy', 'CDR  Cedar Realty Trust', 'IWR  iShares Russell Midcap Index Fund', 'SHLX  Shell Midstream', 'FEX  AlphaDEX Large Cap', 'RGLS  Regulus', 'SIG  Signet Jewelers', 'ERAS  Erasca Inc', 'RCAT  Red Cat Holdings', 'WBD  Warner Bros Discovery Inc', 'ORGN  Origin Materials', 'ENTX  Entera Bio', 'EFC  Ellington Financial', 'RHS  Guggenheim S&P 500 Equal Weight Consumer Staples', 'GIL  Gildan Activewear', 'EKSO  Ekso Bionics', 'EFV  iShares MSCI Value Index', 'ITRN  Ituran', 'AOS  A. O. Smith', 'QID  ProShares UltraShort QQQ', 'DUG  ProShares Oil & Gas', 'IUSB  iShares Core Total USD Bond Market', 'PSCE  PowerShares S&P SmallCap Energy Portfolio', 'DOZR  Direxion US Infrastructure Bull 2x ETF', 'ORAN  Oange', 'QUMU  Qumu', 'AHH  Armada Hoffler', 'DPRO  Draganfly', 'VUG  Vanguard Growth ETF', 'JOAN  JOANN', 'MO  Altria Group', 'DTOX  Amplify Cleaner Living ETF', 'DOMA  Doma Holdings', 'POLY  Plantronics', 'INTT  inTest', 'VS  Versus Systems ', 'SUP  Superior Industries', 'PEBO  Peoples Bancorp', 'FALN  iShares Fallen Angels USD Bond', 'DGICA  Donegal Group', 'BBIO  BridgeBio Pharma', 'XLK  Technology Sector SPDR ETF', 'CBZ  CBIZ', 'ATUS  Altice USA', 'GSL  Global Ship Lease', 'LYEL  Lyell Immunopharma', 'MVIS  Microvision', 'SKLZ  Skillz', 'BTCM  BIT Mining Ltd', 'AMBA  Ambarella', 'HUM  Humana', 'TDC  Teradata', 'AMR  Alpha Metallurgical Resources', 'TBK  Triumph Bancorp', 'HDV  iShares High Dividend', 'NUS  Nu Skin', 'DBB  PowerShares Metal', 'WOW  WideOpenWest', 'YCL  ProShares Ultra Yen', 'CAKE  Cheesecake Factory', 'PBS  PowerShares Dynamic Media Portfolio', 'LTRPA  Liberty TripAdvisor', 'EXPR  Express', 'EUO  Proshares Euro', 'ASEA  Global X Southeast Asia', 'TLT  iShares 20 Year Treasury Bond ETF', 'CDE  Coeur Mining', 'PAVM  PAVmed', 'IVOL  Quadratic Interest Rate Volatility and Inflation Hedge ETF', 'PARA  Paramount Global', 'RWVG  Direxion Russell 1000 Value Over Growth ETF', 'HASI  Hannon Armstrong', 'CURI  CuriosityStream', 'DBRG  DigitalBridge Group', 'OEF  iShares S&P 100 Index Fund', 'DIA  SPDR DJIA ETF', 'SRGA  Surgalign Holdings', 'EWX  SPDR Emerging Markets', 'RVIC  Retail Value Inc', 'MAGA  Point Bridge GOP Stock Tracker ETF', 'BGFV  Big 5 Sporting', 'NHI  National Health Investors', 'BAP  Credicorp', 'ORCC  Owl Rock Capital', 'SEA  SEA ETF', 'NDSN  Nordson', 'SLNO  Soleno Therapeutics', 'ELMD  Electromed', 'VCYT  Veracyte', 'DBD  Diebold', 'MPW  Medical Properties Trust', 'CEIX  CONSOL Energy', 'OLED  Universal Display', 'ONON  On Holding AG', 'PCOR  Procore Technologies', 'EGO  Eldorado Gold', 'ILMN  Illumina', 'SU  Suncor', 'ARNC  Arconic', 'APRN  Blue Apron', 'PBD  PowerShares Global Clean Energy Portfolio', 'GNR  SPDR Natural Recources', 'DXLG  Destination XL Group', 'BMEA  Biomea Fusion', 'IWL  iShares Russell Top 200', 'KRP  Kimbell Royalty Partners', 'GHYB  GS Access High Yield Corporate Bond Etf', 'MRTX  Mirati Therapeutics', 'AFIB  Acutus Medical', 'KNBE  KnowBe4', 'UVXY  ProShares Ultra VIX Short Term Futures', 'KNTE  Kinnate Biopharma', 'LSCC  Lattice Semiconductor', 'HYBB  iShares Bb Rated Corporate Bond ETF', 'HIW  Highwoods Properties', 'IVOG  Vanguard S&P Mid Cap 400 Growth', 'MLPA  Global X MLP', 'TCDA  Tricida', 'IWF  iShares Russell 1000 Growth Index Fund', 'MXCT  MaxCyte', 'AEO  American Eagle', 'CTEC  Global X CleanTech ETF', 'DNAY  Codex DNA Inc', 'ZNH  China Southern Airlines', 'RAIL  Freightcar America', 'NWBI  Northwest Bancshares', 'EWS  iShares Singapore', 'ALGN  Align Technology', 'RRX  Regal Rexnord', 'PRIM  Primoris', 'EKG  EKG ETF', 'GGG  Graco', 'EQRX  CM Life Sciences III', 'HYZN  Hyzon Motors', 'OOTO  Direxion Daily Travel & Vacation Bull 2x', 'CISO  Cerberus Cyber Sentinel', 'BPOP  Popular', 'CRNX  Crinetics Pharmaceuticals', 'PAYA  Paya Holdings', 'SGBX  SG Blocks', 'GHM  Graham', 'BRLT  Brilliant Earth Group', 'SGLY  Sino-Global Shipping America', 'BYSI  BeyondSpring', 'IOT  Samsara', 'NYT  New York Times', 'VGK  Vanguard MSCI Europe', 'VAC  Marriott Vacations', 'FTEC  Fidelity MSCI Information Technology Index', 'AXS  Axis Capital', 'MARK  Remark Media', 'GTLB  Gitlab', 'ETNB  89bio', 'INVA  Innoviva', 'LOAN  Manhattan Bridge Capital', 'SEAS  SeaWorld', 'FLT  FleetCor', 'CWT  California Water Service', 'FNGS  Microsectors Fang ETNs  due Jan 8 2038', 'YUMC  Yum China', 'BILL  Bill.com Holdings', 'AGFS  AgroFresh Solutions', 'A  Agilent', 'PETQ  PetIQ', 'UUUU  Energy Fuels', 'GWW  Grainger', 'JPM  JPMorgan', 'FILL  iShares MSCI Global Energy Producers', 'PDD  Pinduoduo', 'XXII  22nd Century', 'MP  MP Materials', 'DIVB  iShares US Div and Buyback ETF', 'PDN  PowerShares FTSE RAFI Developed Markets Ex US Sm Mid Portfolio', 'XOM  Exxon Mobil', 'CRMD  CorMedix', 'KLR  Kaleyra', 'GUSH  Direxion Oil & Gas 3x Bull ETF', 'CARA  Cara Therapeutics', 'VQS  VIQ Solutions', 'SY  So Young International', 'TAGS  Teucrium Agricultural Fund', 'PNW  Pinnacle West', 'HUGS  USHG Acquisition Corp (Class A Stock)', 'DBJP  db X trackers MSCI Japan', 'GPK  Graphic Packaging', 'ACLS  Axcelis', 'IDT  IDT Corp', 'FRNW  Fidelity Clean Energy ETF', 'CHRS  Coherus BioSciences', 'NVCR  NovoCure', 'BNL  Broadstone Net Lease', 'CTKB  Cytek Biosciences Inc', 'OLN  Olin Corp.', 'DHC  Diversified Healthcare', 'FIVG  Defiance Next Gen Connectivity ETF', 'AVAN  Avanti Acquisition Corp', 'PRSR  Prospector Capital Corp', 'SMG  Scotts Miracle-Gro', 'SEIC  SEI Investments', 'FDNI  First Trust Dow Jones International Internet ETF', 'EBC  Eastern Bankshares', 'ADV  Advantage Solutions', 'IYW  iShares Dow Jones US Technology Sector Index Fund', 'WRB  WR Berkley', 'S  SentinelOne', 'IPOD  Social Capital Hedosophia Holdings IV', 'TECS  Direxion Daily Bear', 'ACEL  Accel Entertainment', 'BCLI  BrainStorm', "CHS  Chico's", 'USM  U.S. Cellular', 'TRTX  TPG RE Finance Trust', 'ALK  Alaska Air', 'LDEM  iShares ESG MSCI EM Leaders', 'MNTS  Momentus', 'IBUY  Amplify Online Retail', 'HBNC  Horizon Bancorp', 'ACWV  iShares MSCI ACWMV', 'DEN  Denbury', 'FULT  Fulton Financial', 'BRDG  Bridge Investment Group', 'JJC  iPath Copper ETN', 'ACCD  Accolade', 'SDP  ProShares UltraShort Utilities', 'SLYG  SPDR S&P 600 Small Cap Growth', 'FTK  Flotek', 'BTWN  Bridgetown Holdings', 'MNKD  MannKind', 'DTEA  Davids Tea', 'DEF  Guggenheim Defensive Equity', 'RL  Ralph Lauren', 'ALT  Altimmune', 'LAND  Gladstone Land', 'VNRX  VolitionRX', 'ICHR  Ichor Holdings', 'GBX  Greenbrier', 'C  Citi', 'DBTX  Decibel Therapeutics', 'NYMX  Nymox', 'KTEC  Kraneshares Hang Seng Tech ETF', 'BCYC  Bicycle Therapeutics', 'MPWR  Monolithic Power', 'FR  First Industrial Realty', 'SNPS  Synopsys', 'BRG  Bluerock Residential', 'ETON  Eton Pharmaceuticals', 'CONX  CONX Corp', 'IVLU  iShares MSCI International Developed Value Factor', 'HBIO  Harvard Bioscience', 'DAN  Dana Holding', 'SAN  Banco Santander', 'AAON  AAON', 'TAK  Takeda Pharmaceutical', 'RF  Regions Financial', 'PWR  Quanta Services', 'BBC  BioShares Biotechnology Clinical Trials Fund', 'APA  Apache', 'EPI  WisdomTree India Earnings', 'NOC  Northrop Grumman', 'APPN  Appian Corp', 'ASR  Grupo Aeroportuario', 'DCBO  Docebo ', 'NTRS  Northern Trust', 'OMGA  Omega Therapeutics', 'FTC  First Trust AlphaDEX', 'OTMO  Otonomo Technologies', 'PLSE  Pulse Biosciences', 'DTIL  Precision BioSciences', 'GPP  Green Plains Partners', 'RBA  Ritchie Brothers Auctioneers', 'DQ  Daqo New Energy', 'ZTS  Zoetis', 'INCY  Incyte', 'KGRN  KraneShares MSCI China Clean Technology Index ETF', 'WOLF  Wolfspeed', 'JNUG  Direxion Junior Gold', 'MGM  MGM Resorts', 'MSA  MSA Safety', 'RLAY  Relay Therapeutics', 'PBA  Pembina Pipeline', 'EQT  EQT Corporation', 'PFLT  PennantPark Floating Rate', 'CMBM  Cambium Networks', 'GSEW  GS Equal Weight US Large Cap Equity ETF', 'MDC  M D C', 'IGE  iShares North American Natural Resources', 'LOGC  LogicBio Therapeutics', 'TA  TravelCenters', 'ARGT  Global X', 'STN  Stantec', 'SOYB  Teucrium Soybean', 'SYNA  Synaptics', 'PACB  Pacific Biosciences', 'NVNO  enVVeno Medical', 'SILJ  PureFunds Junior Silver Small Cap', 'FTS  Fortis', 'AAOI  Applied Optoelectronics', 'CRWS  Crown Crafts', 'SUNW  Sunworks', 'MRC  MRC GLOBAL', 'FTCI  FTC Solar', 'CAF  Morgan Stanley China A Share Fund', 'PSCH  PowerShares S&P SmallCap Health Care', 'PRF  PowerShares Exchange', 'OII  Oceaneering', 'XOP  SPDR S&P Oil and Gas Exploration and Production', 'EMN  Eastman Chemical', 'WBAT  WBAT ETF', 'AXGN  AxoGen', 'IMTX  Immatics NV', 'IDEV  iShares Core MSCI International Developed Markets ETF', 'SBS  Companhia Sanea', 'EVER  EverQuote', 'LQDA  Liquidia Technologies', 'PDS  Precision Drilling', 'CUTR  Cutera', 'EUSA  iShares MSCI USA Equal Weighted ETF', 'KLNE  Direxion Daily Global Clean Energy Bull 2x Shares', 'RLX  RLX Technology', 'CIR  CIRCOR', 'FRT  Federal Realty', 'AAXJ  iShares', 'DHI  D R Horton', 'AEYE  AudioEye', 'ARKX  ARK Space Exploration and Innovation', 'VONG  Vanguard Russell 1000 Growth', 'TRHC  Tabula Rasa HealthCare', 'PLRX  Pliant Therapeutics', 'BBSI  Barrett Business', 'CSR  Centerspace', 'WATT  Energous', 'KNDI  Kandi Technologies', 'NSP  Insperity', 'VTSI  Virtra', 'NZUS  NZUS ETF', 'ALEC  Alector', 'BANC  Banc of California', 'PTVE  Pactiv Evergreen', 'QCOM  Qualcomm', 'ONG  ONG ETF', 'SPDW  SPDR Portfolio Developed World ex US ETF', 'ARDX  Ardelyx', 'VRDN  Viridian Therapeutics', 'STIM  Neuronetics', 'UYG  ProShares Ultra Financials', 'RETL  Direxion Daily Retail Bull 3X Shares', 'VVV  Valvoline', 'XHYH  XHYH ETF', 'TTAC  TrimTabs Float Shrink ETF', 'TLS  Telos Corp', 'SLAB  Silicon Laboratories', 'SMPL  Simply Good Foods', 'UGL  ProShares Ultra Gold', 'TRIB  Trinity Biotech', 'ZEPP  Zepp Health Corp', 'REMX  Market Vectors Rare Earth Strategic Metals', 'SINV  ETFMG Prime 2x Daily Inverse Junior Silver Miners ETF', 'SSRM  SSR Mining', 'WW  Weight Watchers', 'SHV  iShares Treasury', 'PWSC  PowerSchool Holdings', 'TME  Tencent', 'SEMR  SEMrush Holdings', 'SVXY  ProShares Short VIX Short Term Futures', 'PUBM  PubMatic', 'RTH  Market Vectors Retail', 'SYNH  Syneos Health', 'TPGY  TPG Pace Beneficial Finance', 'SPHB  PowerShares S&P 500', 'SQZ  SQZ Biotechnologies', 'SWBI  Smith and Wesson', 'WTTR  Select Energy Services', 'UTL  Unitil', 'ROG  Rogers Corporation', 'VITL  Vital Farms', 'RWT  Redwood Trust', 'TWNK  Hostess Brands', 'XSD  Semiconductor SPDR', 'TASK  TaskUs', 'SHW  Sherwin Williams', 'ZSL  ProShares UltraShort Silver', 'RPG  Guggenheim S&P 500', 'PXLW  Pixelworks', 'USX  US Xpress Enterprises', 'VNLA  Janus Henderson Short Duration Income ETF', 'TPIC  TPI Composites', 'WSC  WillScot Corp', 'SND  Smart Sand', 'SJT  San Juan Basin Royalty Trust', 'TSP  TuSimple Holdings', 'SHC  Sotera Health', 'ZUMZ  Zumiez', 'RGTI  Rigetti Computing', 'TRUP  Trupanion', 'RC  Ready Capital', 'WTS  Watts Water', 'TMST  TimkenSteel', 'SIL  Global X Silver Miners', 'RXD  ProShares UltraShort Health Care', 'RJF  Raymond James', 'SFE  Safeguard Scientifics', 'WGO  Winnebago', 'UAA  Under Armour Class A', 'TTOO  T2 Biosystems', 'WYNN  Wynn Resorts', 'TENB  Tenable Holdings', 'PTC  PTC', 'WASH  Washington Trust', 'VTIP  Vanguard Short Term Inflation Protected Securities Index Fund', 'SLVM  Sylvamo Corp', 'TMF  Direxion Daily 20 plus Year Treasury Bull 3X Shares', 'RVMD  Revolution Medicines', 'PMVP  PMV Pharmaceuticals', 'SPXC  SPX Corporation', 'VSEC  VSE Corp.', 'VTEX  VTEX ', 'REV  Revlon', 'TTI  TETRA', 'QURE  uniQure', 'PSB  PS Business Parks', 'SPI  SPI Energy', 'PYR  PyroGenesis Canada', 'SPHQ  S&P 500 High Quality Portfolio', 'VREX  Varex Imaging', 'SQSP  Squarespace', 'SNOW  Snowflake', 'SUPN  Supernus', 'VOXX  VOXX', 'VIST  Vista Oil & Gas SAB de CV', 'SPRO  Spero Therapeutics', 'RCMT  RCM Technologies', 'STX  Seagate', 'VAPO  Vapotherm', 'USD  ProShares Ultra Semiconductors', 'SCI  Service Corp.', 'SUMO  Sumo Logic', 'YORW  York Water', 'SRS  ProShares UltraShort Real Estate', 'TRI  Thomson Reuters', 'TARS  Tarsus Pharmaceuticals', 'UGI  UGI Corporation', 'UDR  UDR', 'PSN  Parsons', 'TAIL  Cambria Tail Risk ETF', 'REK  ProShares Short Real Estate', 'UNVR  Univar', 'TYL  Tyler Technologies', 'PSCI  PowerShares S&P SmallCap Industrials Portfolio', 'RM  Regional Management', 'PUNK  PUNK ETF', 'TARK  TARK ETF', 'RRC  Range Resources', 'STCN  Steel Connect', 'SMDD  ProShares UltraPro Short MidCap 400', 'SMCI  Super Micro', 'UFPI  Universal Forest', 'SBM  ProShares Short Basic Materials', 'ROK  Rockwell Automation', 'SNLN  Highland iBoxx Senior Loan', 'VRE  Mack Cali Realty', 'SIJ  ProShares UltraShort Industrials', 'QNST  QuinStreet', 'TW  Tradeweb Markets', 'XELA  Exela Technologies', 'ROBT  First Trust Nasdaq AI and Robotics ETF', 'WDS  WDS Stock', 'QLGN  Qualigen Therapeutics', 'THCX  The Cannabis ETF', 'SEAC  SeaChange', 'WEBR  Weber Inc', 'VCLO  Simplify Volt Cloud and Cybersecurity Disruption ETF', 'VXF  Vanguard Extended', 'VZIO  VIZIO Holding Corp', 'PUMP  ProPetro Holding', 'VCNX  Vaccinex', 'TWTR  Twitter', 'ULTA  Ulta Beauty', 'SIVR  ETFS Silver Trust', 'PRVA  Privia Health Group', 'ROCK  Gilbraltar Industries', 'VIRT  Virtu Financial', 'RGS  Regis', 'SOVO  Sovos Brands', 'SHBI  Shore Bancshares', 'SAGE  Sage Therapeutics', 'RDIV  RevenueShares Ultra Dividend Fund', 'QCLN  First Trust NASDAQ Clean Edge Green Energy Index Fund', 'ZYNE  Zynerba', 'SNSR  Global X Internet of Things Thematic', 'SASR  Sandy Spring Bancorp', 'SRVR  Pacer Benchmark Data & Infrastructure Real Estate ETF', 'TG  Tredegar', 'SLNH  Mechanical Technology', 'WPS  iShares International Developed Property', 'UNFI  United Natural Foods', 'SMMT  Summit Therapeutics', 'ULCC  Frontier Group Holdings', 'SMDV  ProShares Russell 2000 Dividend Growers ETF', 'TAC  TransAlta', 'QQXT  First Trust NASDAQ 100 Ex Technology Sector Index', 'PZN  Pzena Investment', 'RMBL  RumbleOn', 'YCBD  cbdMD', 'WLL  Whiting Petroleum', 'SPYV  SPDR S&P 500 Value', 'U  Unity Software', 'STR  Sitio Royalties Corp', 'YOLO  Advisorshares Pure Cannabis', 'PRVB  Provention Bio', 'RMAX  RE MAX Holdings', 'STRL  Sterling Construction', 'STAB  Statera BioPharma', 'WANT  Direxion Daily Consumer Discretionary Bull 3x', 'TRIP  TripAdvisor', 'TWOU  2U', 'VB  Vanguard Small Cap ETF', 'TX  Ternium', 'RWK  RevenueShares Mid', 'WPP  WPP PLC', 'VFH  Vanguard Financials ETF', 'UMBF  UMB Financial', 'RVLP  RVL Pharmaceuticals', 'SA  Seabridge Gold', 'TUR  iShares Turkey', 'WNS  WNS Holdings', 'PNC  PNC Financial', 'PXH  PowerShares FTSE RAFI Emerging Markets', 'VTC  Vanguard Total Corporate Bond', 'VV  Vanguard Large Cap', 'WAL  Western Alliance', 'SHEL  Shell PLC', 'WSBF  Waterstone Financial', 'WLKP  Westlake Chemical Partners', 'VAW  Vanguard Materials', 'SILK  Silk Road Medical', 'SCHD  Schwab US', 'SCHB  Schwab US Broad Market', 'RBCAA  Republic Bancorp', 'VTNR  Vertex Energy', 'SMH  Market Vectors Semiconductor', 'TWST  Twist Bioscience', 'SHO  Sunstone Hotel', 'QUS  SPDR MSCI USA Quality Mix ETF', 'RRR  Red Rock Resorts', 'SITE  SiteOne Landscape', 'SANA  Sana Biotechnology', 'RXL  ProShares Ultra Health Care', 'PROF  Profound Medical', 'QDEL  Quidel', 'POR  Portland General Electric', 'WBT  Welbilt', 'UBOT  Direxion Daily Robotics AI and Automation Bull 2x', 'SRAX  Social Realty', 'TCN  Tricon Residential', 'SCHW  Charles Schwab', 'SEF  ProShares Short Financials', 'SRTS  Sensus Healthcare', 'PRAA  PRA Group', 'PRCT  Procept Biorobotics Corp', 'UBT  ProShares Ultra 20 Plus Year Treasury', 'RARE  Ultragenyx', 'SAM  Boston Beer', 'TWO  Two Harbors', 'SBUG  iPath Silver ETN', 'QQQE  Direxion Nasdaq 100 Equal Weighted ETF', 'RBAC  RedBall Acquisition Corp', 'PRU  Prudential', 'SB  Safe Bulkers', 'PST  ProShares UltraShort Lehman 7 10 Year Treasury', 'QSI  Quantum Si', 'SEM  Select Medical', 'RGEN  Repligen', 'SNEX  StoneX Group', 'SYK  Stryker', 'SPCE  Virgin Galactic', 'USIG  Investment Grade Corporate Bond ETF', 'TECH  Bio Techne', 'TDY  Teledyne', 'UMDD  ProShares UltraPro MidCap 400', 'VVPR  VivoPower', 'UMH  UMH Properties', 'PODD  Insulet', 'UEIC  Universal Electronics', 'QTNT  Quotient', 'SGDM  Sprott Gold Miners', 'SEAT  Vivid Seats', 'TMQ  Trilogy Metals', 'VVI  Viad', 'UTSI  UTStarcom', 'VINO  Gaucho Group Holdings', 'TFX  Teleflex', 'SAVA  Cassava Sciences', 'PRPH  ProPhase Labs', 'RYTM  Rhythm Pharmaceuticals', 'RFV  Guggenheim S&P Midcap 400 Pure Value', 'VYMI  Vanguard International High Dividend Yield ETN', 'STXS  Stereotaxis', 'PTON  Peloton Interactive', 'SUSA  iShares MSCI USA ESG Select', 'SREV  ServiceSource', 'SATL  Satellogic V Inc', 'SMED  Sharps Compliance', 'PTGX  Protagonist Therapeutics', 'XEL  Xcel Energy', 'STEM  Stem Inc', 'TRX  Tanzanian Royalty', 'SPEU  SPDR Europe ETF', 'OEG  Orbital Energy', 'IDXX  IDEXX', 'GRAB  Altimeter Growth', 'LRNZ  TrueShares AI and Deep Learning ETF', 'BCOV  Brightcove', 'SOUN  SoundHound AI Inc', 'ADP  ADP', 'ENB  Enbridge', 'TPX  Tempur Sealy', 'ELD  Wisdom Tree Debt Fund', 'EET  ProShares Ultra MSCI', 'BBDC  Barings BDC', 'ARC  ARC Document', 'CHIR  Global X MSCI China Real Estate ETF', 'NBR  Nabors Industries', 'CPSH  CPS Technologies', 'GCI  Gannett', 'ENVX  Enovix', 'TIPD  TIPD ETF', 'ASAI  Sendas Distribuidora SA', 'SOFI  SoFi Technologies', 'FC  Franklin Covey', 'EMLP  North American Energy', 'FDS  FactSet', 'BE  Bloom Energy', 'DISH  Dish Network', 'DAL  Delta Air Lines', 'FLNG  Flex LNG Limited', 'GTBP  GT Biopharma', 'XHYF  XHYF ETF', 'SFYX  Sofi Next 500 ETF', 'ANGL  Market Vectors Fallen Angel High Yield Bond', 'PBR/A  Petroleo Brasileiro', 'CNK  Cinemark', 'PBI  Pitney Bowes', 'ATKR  Atkore', 'MDEV  First Trust Indxx Medical Device ETF', 'CTOS  Custom Truck One Source', 'HOWL  Werewolf Therapeutics', 'KRMD  Repro Med Systems', 'RIG  Transocean', 'FF  FutureFuel', 'HUGE  FDS Pharma', 'PFE  Pfizer', 'SPH  Suburban Propane', 'CBIO  Catalyst Biosciences', 'SSG  ProShares UltraShort Semiconductors', 'GMED  Globus Medical', "MCO  Moody's", 'BUR  Burford Capital ', 'AZN  AstraZeneca', 'EVX  Market Vectors Environmental Services Fund', 'NMTR  9 Meters Biopharma', 'WULF  IKONICS Corporation', 'VRNA  Verona Pharma', 'IGOV  iShares International', 'RNA  Avidity Biosciences', 'CAN  Canaan Inc', 'EQ  Equillium', 'DBEU  Deutsche X trackers MSCI Europe Hedged Equity', 'INFU  InfuSystem', 'BLFS  BioLife Solutions', 'SJR  Shaw Communications', 'CVE  Cenovus Energy', 'ATEX  Anterix', 'AQB  AquaBounty Technologies', 'PAAS  Pan American Silver', 'OUT  OUTFRONT Media', 'NBTB  NBT Bancorp', 'HLLY  Holley Inc', 'NLR  Market Vectors Uranium Plus Nuclear Energy', 'IFV  First Trust Dorsey Wright International Focus 5', 'EXEL  Exelixis', 'IMMR  Immersion', 'PYPL  PayPal', 'ATAX  America First Multifamily Investors', 'OMF  OneMain Holdings', 'SOHU  Sohu.com', 'CNMD  CONMED', 'GRPH  Graphite Bio', 'MMSI  Merit Medical', 'ANTE  AirNet Technology', 'GOL  GOL Linhas', 'TPYP  Tortoise North American Pipeline Fund', 'THRY  Thryv Holdings', 'NEWT  Newtek Business Services', 'MUDS  Mudrick Capital Acquisition Corp II', 'RXT  Rackspace Technology', 'GLDB  Strategy Shares Gold Hedged Bond', 'REPL  Replimune Group', 'HNRG  Hallador Energy', 'NVEI  Nuvei', 'SPEM  SPDR Portfolio Emerging Markets ETF', 'SFYF  SoFi Social 50 ETF', 'VTGN  VistaGen Therapeutics', 'FSTA  Fidelity MSCI Consumer Staples Index', 'PKOH  ParkOhio Holdings', 'AINV  Apollo Investment', 'HDGE  Ranger Equity Bear', 'PEGA  Pegasystems', 'SES  SES AI Corp', 'PAHC  Phibro Animal Health', 'LOGI  Logitech', 'IR  Ingersoll Rand Plc', 'VNQ  Vanguard REIT ETF', 'ENV  Envestnet', 'AVID  Avid Technology', 'HROW  Harrow Health', 'UBS  UBS', 'CNYA  iShares MSCI China A ETF', 'VLTA  Volta', 'LRMR  Larimar Therapeutics', 'OHI  Omega Healthcare', 'PBF  PBF Energy', 'ADCT  ADC Therapeutics', 'BLK  BlackRock', 'CHIQ  Global X China Consumer', 'HYMC  Hycroft Mining Holding Corp', 'IGAC  IG Acquisition Corp', 'ENVB  Enveric Biosciences', 'SNDR  Schneider National', 'KNX  Knight Transportation', 'CXDO  Crexendo', 'LLAP  Tailwind Two Acquisition Corp', 'INSM  Insmed', 'ASAN  Asana', 'ALLK  Allakos', 'RLJ  RLJ Lodging Trust', 'RLI  RLI,', 'FXR  Durables AlphaDEX', 'CLPS  CLPS', 'CLR  Continental Resources', 'GLBS  Globus Maritime', 'MRUS  Merus', 'VOE  Vanguard Mid Cap', 'CTS  CTS Corporation', '.XAU  Philadelphia Gold and Silver Index', 'HIBL  Direxion Daily S&P 500 High Beta Bull 3x', 'MIMO  Airspan Networks', 'TTEC  TeleTech', 'PXF  PowerShares FTSE RAFI Developed Markets ex US Portfolio', 'ESPR  Esperion', 'RFL  Rafael Holdings', 'FLM  First Trust ISE Global Engineering and Construction Index Fd', 'SCHK  Schwab 1000 Index ETF', 'SSO  ProShares Ultra S&P 500', 'EBIX  Ebix', 'BAD  B.A.D. Betting Alcohol Pharma ETF', 'OUST  Ouster Inc', 'SDVY  First Trust SMID Cap Rising Dividend Achievers ETF', 'SAR  Saratoga Investment', 'BSAC  Santander Chile', 'REKR  Rekor Systems', 'RDHL  Redhill Boipharma', 'EGLX  Enthusiast Gaming Holdings', 'FGEN  FibroGen', 'SYLD  Cambria Shareholder Yield', 'LUXA  Lux Health Tech Acquisition Corp', 'PCTI  PCTEL', 'LQDT  Liquidity Services', 'XHYI  XHYI ETF', 'EPAM  EPAM Systems', 'AMAL  Amalgamated Financial', 'BNGO  Bionano Genomics', 'SWX  Southwest Gas', 'GRPN  Groupon', 'YSG  Yatsen Holding Limited', 'SSTI  ShotSpotter', 'LVHD  Legg Mason Low Volatility High Dividend', 'ALLT  Allot Communications', 'OLO  Olo', 'SHLS  Shoals Technologies Group', 'RIGL  Rigel Pharmaceuticals', 'MTRN  Materion', 'MA  Mastercard', 'SLM  Sallie Mae', 'IBCP  Independent Bank', 'BIOR  Biora Therapeutics Inc', 'MVO  MV Oil Trust', 'GNE  Genie Energy', 'NR  Newpark Resources', 'CM  Canadian Imperial', 'MHLD  Maiden Holdings', 'YMM  Full Truck Alliance', 'ARKQ  ARK ETF Trust', 'ECON  EGShares Emerging Markets', 'JPIN  JPMorgan Diversified Return International Equity ETF', 'GENE  Genetic Technologies', 'HRTG  Heritage Insurance', 'ILPT  Industrial Logistics Properties', 'DWX  SPDR', 'CAT  Caterpillar', 'ECLN  First Trust EIP Carbon Impact ETF', 'ABM  ABM', 'CLPR  Clipper Realty', 'AWK  American Water', 'ACI  Albertsons', 'CLB  Core Laboratories', 'GDVD  GDVD ETF', 'EXLS  ExlService', 'SIGA  Siga Technologies', 'EWC  iShares Canada', 'APTO  Aptose Biosciences', 'YQ  17 Education and Technology Group', 'GDS  GDS Holdings (ADS)', 'GRTX  Galera Therapeutics', 'QFIN  360 DigiTech', 'CVEO  Civeo', 'AAT  American Assets Trust', 'IP  International Paper', 'MGIC  Magic Software', 'OPRX  OptimizeRx', 'OBLG  Oblong', 'ACWI  iShares MSCI ACWI Index Fund', 'PSL  PowerShares DWA Consumer Staples Momentum Fund', 'YUM  Yum! Brands', 'FLL  Full House Resorts', 'ACER  Acer Therapeutics', 'NBEV  New Age Beverages', 'PTBD  Pacer Trendpilot US Bond ETF', 'EPIX  Essa Pharma', 'AN  AutoNation', 'CCCC  C4 Therapeutics', 'NET  Cloudflare', 'BAL  iPath Cotton ETN', 'PRM  Perimeter Solutions SA', 'MBIO  Mustang Bio', 'SUPV  Supervielle', 'EW  Edwards Lifesciences', 'EVFM  Evofem Biosciences', 'HTLD  Heartland Express', 'INVO  INVO BioScience', 'GLUE  Monte Rosa Therapeutics Inc', 'MTH  Meritage Homes', 'VBLT  Vascular Biogenics', 'TDS  Telephone and Data', 'INBK  First Internet Bancorp', 'LESL  Leslies', 'PCVX  Vaxcyte', "LUB  Luby's", 'UNM  Unum Group', 'EMLC  Emerging Markets Sovereign Debt ETF', 'ACHC  Acadia', 'UMC  United Microelectronics', 'BIDU  Baidu', 'APD  Air Products', 'MBB  iSharesMBS Mixed Rate', 'PCB  PCB Bancorp', 'BCSF  Bain Capital', 'TLRY  Tilray', 'GAMR  PureFunds Video Game Tech ETF', 'TOKE  Cambria Cannabis ETF', 'AMLP  Alerian', 'IDEX  Ideanomics', 'TWI  Titan International', 'KIDS  OrthoPediatrics', 'TCBI  Texas Capital', 'ICLK  iClick Interactive Asia', 'PLBY  PLBY Group', 'EWD  iShares Sweden', 'GPRO  GoPro', 'LMFA  LM Funding America', 'SSTK  Shutterstock', 'FORM  FormFactor', 'YNDX  Yandex', 'AMTX  Aemetis', 'HARP  Harpoon Therapeutics', 'MKC  McCormick', 'PK  Park Hotels & Resorts', 'PJP  PowerShares Dynamic Pharma', 'BGSF  BG Staffing', 'TROW  T Rowe Price', 'PBW  Powershares Wilderhill Clean Energy Portfolio', 'EWP  iShares Spain', 'DXPE  DXP Enterprises', 'SHEN  Shendoah Telecom', 'LL  Lumber Liquidators', 'PPL  PPL Corp.', 'MLR  Miller Industries', 'TPH  TRI Pointe', 'ETWO  E2open Parent Holdings', 'INOD  Innodata', 'CCCS  CCC Intelligent Solutions Holdings', 'PINE  Alpine Income Property Trust', 'SWTX  SpringWorks Therapeutics', 'DSP  Viant Technology', 'FNKO  Funko', 'SXC  SunCoke Energy', 'TRMK  Trustmark', 'EMGF  iShares Edge MSCI Multifactor Emerging Markets ETF', 'BBCP  Concrete Pumping Holdings', 'PUI  PowerShares DWA Utilities Momentum Portfolio', 'CLS  Celestica', 'ODFL  Old Dominion', 'SRTY  ProShares UltraPro Short Russell 2000', 'NCNO  nCino', 'AGTC  Applied Genetic', 'REG  Regency Centers', 'FDD  STOXX European', 'IMPX  AEA-Bridges Impact Corp', 'ATY  AcuityAds Holdings', 'UVV  Universal Corporation', 'FST  FAST Acquisition Corp', 'ADN  Advent Technologies', 'AORT  Artivion', 'DRIO  DarioHealth', 'XPP  ProShares Ultra FTSE China 25', 'OKE  ONEOK', 'OMI  Owens & Minor', 'EMWP  Eros Media World PLC', 'SCHF  Schwab International', 'RPM  RPM', 'ALTO  Alto Ingredients', 'GD  General Dynamics', 'PRI  Primerica', 'CODX  Co-Diagnostics', 'FEZ  SPDR Euro STOXX', 'SMP  Standard Motor', 'MVPS  Amplify Thematic All Stars ETF', 'INST  Instructure Holdings', 'ARCT  Arcturus Therapeutics', 'PRN  PowerShares DWA Industrials', 'CHX  ChampionX', 'IFGL  IShares International Real Estate', 'YVR  Liquid Media Group', 'LPSN  LivePerson', 'SNAP  Snap Inc.', 'NHTC  Natural Health Trends', 'SAH  Sonic Automotive', 'ALVR  AlloVir', 'SDEM  Global X SuperDividend Emerging Markets ETF', 'IDAT  iShares Cloud 5G and Tech ETF', 'VRNS  Varonis', 'EPP  iShares MSCI Pacific Japan', 'BIOX  Bioceres Crop Solutions', 'QTEC  First Trust NASDAQ', 'CTXS  Citrix', 'CSSE  Chicken Soup for the Soul Entertainment', 'AERI  Aerie Pharmaceuticals', 'ACR  ACRES Commercial Realty', 'IBIO  iBio', 'CDNS  Cadence Design', 'ADM  Archer Daniels', 'KRON  Kronos Bio', 'DAWN  Day One Biopharmaceuticals', 'HCP  Hashicorp', 'EELV  PowerShares Emerging Markets', 'HWM  Howmet Aerospace', 'OMC  Omnicom', 'LGV  Longview Acquisition Corp II', 'SKX  Skechers', 'ACES  ALPS Clean Energy ETF', "PLAY  Dave & Buster's", 'AU  AngloGold', 'SRCE  1st Source', 'CNTB  Connect Biopharma Holdings', 'SNDX  Syndax', 'BECN  Beacon Roofing', 'NAN  Nuveen New York Dividend Advantage Municipal Fund', 'GFS  Globalfoundries', 'IGIC  International General Insurance', 'SGEN  Seattle Genetics', 'ALTR  Altair Engineering', 'GGB  Gerdau', 'PFSW  Pfsweb', 'VNET  21Vianet', 'PFI  PowerShares DWA Financial Momentum Portfolio', 'VCR  Vanguard Consumer', 'MJ  ETFMG Alternative Harvest ETF', 'MORF  Morphic Holding', 'DSL  DoubleLine', 'TEF  Telefonica', 'PTRA  Proterra', 'TZA  Direxion Daily Small Cap Bear 3X Shares', 'CHWY  Chewy', 'WTT  Wireless Telecom Group Inc', 'NVAX  Novavax', 'GNL  Global Net Lease', 'IGM  iShares North American Tech', 'TMX  Terminix Global Holdings', 'LOMA  Loma Negra Compania Argentina S.A.', 'NAIL  Direxion Daily Homebuilders and Supplies Bull 3X Shares', 'EPC  Edgewell Personal Care', 'CRAI  CRA International', 'FHN  First Horizon', 'NNI  Nelnet', 'IGF  iShares Infastructure', 'ROL  Rollins', 'DCO  Ducommun', 'ANGI  Angies List', 'INDB  Independent Bank', 'CHTR  Charter', 'EGY  VAALCO Energy', 'WAFD  Washington Federal', 'XYL  Xylem', 'LILAK  Liberty Global LiLAC', 'VEA  Vanguard MSCI EAFE', 'MVST  Microvast Holdings', 'AAIC  Arlington Asset', 'BEEM  Beam Global', 'BMRA  Biomerica Inc', 'COUP  Coupa Software', 'COOK  Traeger', 'HZO  MarineMax', 'WEAV  Weave Communications Inc', 'RDW  Redwire', 'TYD  Direxion Daily 7 to 10 Year Treasury Bull 3X Shares', 'IGN  iShares Goldman Sachs Networking Index Fund', 'HLI  Houlihan Lokey', 'QCRH  QCR Holdings', 'POWI  Power Integrations', 'CD  Chindata Group', 'HMLP  Hoegh LNG', 'BGNE  BeiGene', 'JBSS  John B Sanfilippo and Son', 'ESCA  Escalade', 'SPOK  Spok', 'FREE  Whole Earth Brands', 'PWFL  PowerFleet', 'WDAY  Workday', 'CZNC  Citizens & Northern', 'MGNX  MacroGenics', 'AGIO  Agios Pharmaceuticals', 'SNX  SYNNEX', 'DFAI  Dimensional International Core Equity ETF', 'CBU  Community Bank System', 'EEMV  iShares MSCI', 'CIO  City Office REIT', 'AVY  Avery Dennison', 'PCYG  Park City Group', 'HUBB  Hubbell', 'GPOR  Gulfport Energy', 'QMCO  Quantum Corp', 'SOS  SOS Limited', 'BWX  SPDR Barclays Capital Intl T Bond', 'UGP  Ultrapar', 'SGH  SMART Global', 'MD  Mednax', 'RGP  Resources Connection', 'BKCH  Global X Blockchain ETF', 'CPT  Camden Property', 'LCI  Lannett', 'PJT  PJT Partners', 'PSNL  Personalis', 'AMZA  ETFis Series Trust I', 'PNTG  Pennant Group', 'KRYS  Krystal Biotech', 'STGW  Stagwell', 'NNBR  NN', 'FXY  Japanese Yen Trust', 'ROVR  Rover Group', 'BSEA  Breakwave Sea Deacarbonization ETF', 'REAX  Real Brokerage', 'AXLA  Axcella Health', 'JNCE  Jounce Therapeutics', 'EXPD  Expeditors', 'MEOH  Methanex', 'CMPR  Cimpress', 'DBV  Powershares G10', 'CTT  CatchMark Timber', 'UPH  UpHealth', 'BHP  BHP Billiton', 'BBAR  BBVA Banco Frances', 'INSG  Inseego', 'ZBH  Zimmer Biomet', 'FFNW  First Financial Northwest', 'ATEC  Alphatec', 'KBE  SPDR S&P Bank', 'HACK  ISE Cyber Security ETF', 'BV  BrightView Holdings', 'IHY  Market Vectors International', 'DIOD  Diodes', 'PFMT  Performant Financial', 'DEUS  Deutsche Xtrackers Russell 1000 Enhanced Beta ETF', 'NCMI  National CineMedia', 'VIS  Vanguard Industrial', 'FXH  AlphaDEX Health Care', 'DRH  Diamond Rock', 'HAFC  Hanmi Financial', 'RGLD  Royal Gold', 'CIB  Bancolombia', 'IVC  Invacare', 'LMT  Lockheed Martin', 'STKS  ONE Group Hospitality', 'XRT  SPDR S&P Retail', 'YCS  ProShares UltraShort Yen', 'BGCP  BGC Partners', 'PGTI  PGT, Inc.', 'CGEN  Compugen', 'LKFN  Lakeland Financial', 'ENLV  Enlivex Therapeutics', 'VXRT  Vaxart', 'FKWL  Franklin Wireless', 'BOND  PIMCO Total Return Exchange Traded Fund', 'RILY  B Riley Financial', 'RUTH  Ruths Hospitality Group', 'TBF  ProShares Short 20 plus Year Treasury', 'ANIK  Anika Therapeutics', 'APRE  Aprea Therapeutics', 'FM  iShares Frontier 100', 'CPRI  Capri Holdings', 'EDOW  Dow 30 Equal Weight ETF', 'BBP  BioShares Biotechnology Products Fund', 'BQ  Boqii Holding Limited', 'IYZ  Telecommunications Sector iShares ETF', 'PEZ  PowerShares Consumer Cyclicals Momentum Fund', 'VMBS  Vanguard Mortgage Backed Securities Bond', 'QRTEA  Qurate Retail QVC Group', 'NCR  NCR Corp.', 'GTHX  G1 Therapeutics', 'VMC  Vulcan Materials', 'LMB  Limbach Holdings', 'PHB  PowerShares Fundamental High Yield Corporate Bond', 'TAN  Guggenheim Global Solar Index', 'HYS  PIMCO 0 to 5 CorporateBond', 'SCHJ  Schwab 1 to 5 Year Corporate Bond ETF', 'MCHI  iShares MSCI China', 'CFVI  CF Acquisition Corp', 'GOED  1847 Goedeker', 'MCHX  Marchex', 'CYN  Cyngn Inc', 'COMT  iShares Commodities Select Strategy', 'SWI  SolarWinds', 'KZR  Kezar Life Sciences', 'BNFT  Benefitfocus', 'IGLD  FT Cboe Vest Gold Strategy Target Income', 'MULN  Net Element', 'ARR  ARMOUR Residential', 'CAAS  China Automotive', 'IAK  iShares US Insurance', 'OSH  Oak Street Health', 'CTIC  CTI BioPharma', 'DKNG  DraftKings', 'TAL  Tal Education Group', 'RKLB  Rocket Lab USA', 'SOXX  iShares PHLX SOX Semiconductor Sector Index Fund', 'CWEB  Direxion Daily Csi China Internet Index Bull 2x Shares', 'IEX  IDEX Corp.', 'EAT  Brinker', 'JBI  Janus International', 'KGC  Kinross Gold', 'NSA  National Storage', 'AMOT  Allied Motion', 'KODK  Kodak', 'FRBK  Republic First Bancorp', 'BCAB  BioAtla', 'MSM  MSC Industrial', 'RDUS  Radius Health', 'NTAP  NetApp', 'FFTY  Academy Funds Trust Innovator IBD 50 Fund', 'TMAT  Main Thematic Innovation', 'ERX  Direxion Energy Bull 3x', 'BFTR  Blackrock Future Innovators ETF', 'CACC  Credit Acceptance', 'BLBD  Blue Bird', 'ELY  Callaway Golf', 'CMC  Commercial Metals', 'MRSN  Mersana Therapeutics', 'PFM  PowerShares Dividend Achievers', 'GSM  Ferroglobe', 'DYAI  Dyadic International', 'SBIO  ALPS Medical Breakthroughs ETF', 'BTCR  Volt Crypto Industry Revolution ETF', 'EE  Excelerate Energy Inc', 'RWM  ProShares Short Russell2000', 'FDMT  4D Molecular Therapeutics', 'TSLX  TPG Specialty Lending', 'SI  Silvergate Capital Corp', 'SCHE  Schwab Emerging', '.DJX  DJIA', 'BIS  ProShares UltraShort Biotech', 'PEAK  Healthpeak Properties', 'RYAN  Ryan Specialty Group Holdings', 'PRQR  ProQR Therapeutics', 'NMFC  New Mountain Finance', 'VC  Visteon', 'EFAS  Globa X MSCI SuperDividend EAFE', 'DE  Deere', 'LHX  L3Harris Technologies', 'CENN  Cenntro Electric Group', 'TD  Toronto Dominion Bank', 'RH  Restoration Hardware', 'TEN  Tenneco', 'GRNR  GRNR ETF', 'SWVL  Swvl Holdings Corp', 'BRO  Brown & Brown', 'WU  Western Union', 'NHC  National HealthCare', "M  Macy's", 'BILI  Bilibili', 'CNM  Core and Main', '.UTY   PHLX Utility Sector Index', 'XLY  Consumer Discretionary Sector SPDR ETF', 'MDYV  SPDR S&P 400 Mid Cap Value', 'EWQ  iShares France', 'MTCH  Match Group', 'NTCT  NetScout', 'SPXN  ProShares S&P 500 Ex Financials ETF', 'VERV  Verve Therapeutics', 'SKYT  SkyWater Technology', 'OVID  Ovid Therapeutics', 'ICAD  Icad', 'IHRT  iHeartMedia', 'ATGE  Adtalem Global Education', 'DLS  WisdomTree International SmallCap Dividend Fund', 'LILA  Liberty Global LiLAC', 'VERI  Veritone', 'PARR  Par Pacific', 'WCBR  WisdomTree Cybersecurity Fund', 'SSNC  SS&C', 'CLH  Clean Harbors', 'CPNG  Coupang', 'FTI  FMC Technologies', 'APLE  Apple Hospitality REIT', 'GMS  GMS Inc.', 'NAPA  Duckhorn Portfolio', 'VRP  PowerShares Variable Rate Preferred', 'ENIAY  Enel Americas SA (ADR)', 'TTM  Tata Motors', 'PRO  PROS', '.SIXU  S&P Utilities', 'PRCH  Porch Group', 'VLU  SPDR S&P 1500 Value Tilt', 'THCA  Tuscan Holdings Corp', 'APPS  Digital Turbine', 'RS  Reliance Steel', 'IOGPQ  Ion Geophysical Corp', 'TOTL  SPDR DoubleLine Total Return Tactical ETF', 'COLD  Americold Realty Trust', 'QUIK  QuickLogic', 'DNB  Dun and Bradstreet', 'BSIG  BrightSphere Investment Group', 'MSGS  Madison Square Garden Sports', 'UVE  Universal Insurance', 'HEI/A  HEICO', "MCD  McDonald's", 'NXTC  NextCure', 'STKL  SunOpta', 'VPU  Vanguard Utilities', 'ITI  Iteris', 'USMV  iShares MSCI Minimum Volatility', '.BKX  BKX Index', 'VIA  Via Renewables', 'GL  Globe Life', 'MYE  Myers Industries', 'HR  Healthcare Realty Trust', 'APAM  Artisan Partners', 'CHIS  Global X MSCI China Consumer Staples ETF', 'CLOU  Cloud Computing ETF', 'IVT  Inventrust Properties ', 'YRD  Yirendai', 'ASTL  Legato Merger Corp', 'MANT  ManTech', 'NOVA  Sunnova Energy', 'EMFM  Global X Next Emerging and Frontier', 'CTG  Computer Task Group', 'TTC  Toro Company', 'SUSL  iShares ESG MSCI USA Leaders ETF', 'TYO  Direxion Daily 7 to 10 Year Treasury Bear 3X Shares', 'RODM  Lattice Developed Markets ex US Strategy', 'SVFAU  SVF Investment Corp', 'IRWD  Ironwood', 'EYES  Second Sight', 'KNSL  Kinsale Capital', 'OOMA  Ooma', 'GLNG  Golar LNG', 'IVR  Invesco Mortgage', 'EWM   iShares Malaysia', 'UEC  Uranium Energy', 'CLBT  Cellebrite Digital Intelligence', 'LAD  Lithia Motors', 'WSR  Whitestone REIT', 'NMRD  Nemaura Medical', 'ATLC  Atlanticus Holdings', 'AUB  Atlantic Union Bankshares', 'IHDG  WisdomTree International Hedged Quality Dividend', 'AQN  Algonquin Power', 'LPTX  Leap Therapeutics', 'HLMN  Hillman Solutions Corp', 'SPTN  SpartanNash', 'JPPYY  Jupai Holdings Ltd (ADR)', 'VBIV  VBI Vaccines', 'FLOT  iShares Floating Rate', '.NDX  Nasdaq 100 Index', 'ARRY  Array Technologies', 'PING  Ping Identity', 'MRCY  Mercury Systems', 'IWV  iShares Russell 3000 Index Fund', 'CLVS  Clovis', 'TJX  TJX', 'POST  Post Holdings', 'PUTW  WisdomTree CBOE S&P 500 PutWrite Strategy Fund', 'CASH  Meta Financial', 'PNNT  PennantPark', 'WMK  Weis Markets', 'MCB  Metropolitan Bank Holding Corp', 'RGCO  RGC Resources', 'VYGR  Voyager Therapeutics', 'NETL  Netlease Corporate Real Estate ETF', 'PPBI  Pacific Premier', 'XLG  Guggenheim Russell Top Mega Cap ETF', 'CCL  Carnival', 'SMTC  Semtech', 'FTSM  First Trust Enhanced Short Maturity', 'FICO  Fair Isaac', 'UWM  ProShares Ultra Russell2000', 'LCII  LCI Industries', 'YMTX  Yumanity Therapeutics', 'PSCF  PowerShares S&P SmallCap Financials Portfolio', 'TKAT  Takung Art', 'RVAC  Riverview Acquisition Corp', 'YXI  ProShares Short FTSE China 50', 'WRAP  Wrap Technologies', 'FIW  First Trust ISE index', 'MIRM  Mirum Pharmaceuticals', 'XAR  SPDR S&P Aerospace and Defense', 'TNDM  Tandem Diabetes', 'PTEN  Patterson-UTI', 'XPH  Pharmaceuticals SDPR', 'LFVN  Lifevantage', 'MCHP  Microchip', 'DFIN  Donnelley Financial Solutions', 'ESTE  Earthstone Energy', 'ZTO  ZTO Express', 'TXMD  TherapeuticsMD', 'TOL  Toll Brothers', 'ABR  Arbor Realty', 'SKE  Skeena Resources', 'STER  Sterling Check Corp', 'MIST  Milestone Pharmaceuticals', 'MTSI  MA COM Tech Solutions', 'MAIN  Main Street', 'KULR  KULR Technology Group', 'RDWR  RADWARE', 'MOON  Direxion Moonshot Innovators ETF', 'FLRN  SPDR Barclays Investment Grade Floating Rate', 'IIIV  i3 Verticals', 'DXC  DXC Technology', 'IMTM  iShares Edge MSCI Intl Momentum Factor ETF', 'BR  Broadridge', 'LTC  LTC Properties', 'SPTI  SPDR Portfolio Intermediate Term Treasury ETF', 'TPR  Tapestry', 'SIX  Six Flags', 'IDV  iShares Dow Jones International Select Dividend Index', 'DMAC  DiaMedica Therapeutics', 'TTD  Trade Desk', 'HEWG  iShares MSCI Germany', 'ESPO  VanEck Video Gaming and eSports ETF', 'SLF  Sun Life Financial', 'OSS  One Stop Systems', 'GBIO  Generation Bio', 'GMBL  Esports Entertainment Group', 'FBRX  Forte Biosciences', 'URE  ProShares Ultra Real Estate', 'NTR  Nutrien', 'QGEN  Qiagen', 'WOOD  iShares Timber', 'ICSH  iShares Ultra Short Term Bond ETF', 'FMC  FMC Corporation', 'VEGI  iShares MSCI Global Agriculture Producers', 'FVRR  Fiverr International ', "KSS  Kohl's", 'JWN  Nordstrom', 'BITS  Global X Blockchain ETF', 'GOVT  iShares Core US Treasury Bond', 'MAS  Masco', 'NVST  Envista Holdings', 'TRDF  TRDF ETF', 'EVR  Evercore Partners', 'AGG  iShares Barclays', 'WTBA  West Bancorp', 'INVZ  Innoviz Technologies', 'PECO  Phillips Edison and Co', 'SIFY  Sify Technologies', 'CBRL  Cracker Barrel', 'JMBS  Janus Henderson Mortgage-backed Sec ETF', 'NEO  NeoGenomics', 'VLDR  Velodyne Lidar', 'EDIV  SPDR Emeging Markets', 'IRT  Independence Realty Trust', 'QIPT  Quipt Home Medical Corp', 'JEF  Jefferies Financial Group', 'WPC  WP Carey', 'GLYC  GlycoMimetics', 'AM  Antero Midstream', 'SHOP  Shopify', 'AXSM  Axsome Therapeutics', 'BAH  Booz Allen', 'BDRY  Breakwave Dry Bulk Shipping ETF', 'ACA  Arcosa', 'CHIH  MSCI China Health Care ETF', 'WY  Weyerhaeuser', 'SPHY  SPDR Portfolio High Yield Bond ETF', 'PILL  Direxion Daily Pharmaceutical & Medical Bull 3x', 'HCSG  Healthcare Services', 'MODN  Model N', 'SPNT  SiriusPoint', 'EDIT  Editas Medicine', 'IGHG  Proshares Investment Grade Interest Rate Hedged', 'CWST  Casella Waste', 'HA  Hawaiian Holdings', 'AOSL  Alpha & Omega', 'IUSG  iShares Core US Growth', 'JKHY  Jack Henry', 'SFNC  Simmons First National', 'MSGE  Madison Square Garden Entertainment', 'GK  Advisorshares Gerber Kawasaki ETF', 'SRET  Global X SuperDividend REIT', 'OILK  ProShares K 1 Free Crude Oil Strategy ETF', 'UTZ  Utz Brands', 'WPRT  Westport', 'OPI  Office Properties', 'PSMT  PriceSmart', 'FINV  FinVolution Group', 'WFH  Work From Home ETF', 'CIVB  Civista', 'ACM  Aecom', 'SWIM  Latham Group', 'EXPI  eXp World Holdings', 'EAST  Eastside Distilling', 'SMLP  Summit Midstream', 'SPUU  Direxion Daily S&P 500 Bull 2X Shares', 'VTR  Ventas', 'INN  Summit Hotel Properties', 'ALR  Five Star Senior Living', 'SLRC  Solar Capital', 'FCRD  First Eagle Alternative Capital', 'BBIN  JPMorgan BetaBuilders International Equity', 'AGFY  Agrify Corp', 'WIX  Wix.com', 'SRPT  Sarepta', 'FTXR  First Trust Nasdaq Transportation', 'BPT  BP Prudhoe Bay', 'EWT   iShares Taiwan', 'IBP  Installed Building Products', 'BLI  Berkeley Lights', 'NXST  Nexstar', 'OFC  Corporate Office Properties', 'RCII  Rent A Center', 'LNSR  LENSAR', 'ZIM  ZIM Integrated Shipping Services', 'AMWD  American Woodmark', 'EDC  Direxion Emerging Markets Bull 3x', 'REGL  ProShares S&P MidCap 400 Dividend Aristocrats ETF', 'AVDL  Avadel Pharmaceuticals', 'ATNI  ATN International', 'ADC  Agree Realty', 'SONO  Sonos', 'PINS  Pinterest', 'SUI  Sun Communities', 'ICMB  Investcorp Credit Management', 'CNXT  Market Vectors ChinaAMC', 'CGW  Guggenheim S&P Global Water Index', 'JJG  iPath Grains ETN', 'NMM  Navios Maritime Partners', 'HE  Hawaiian Electric', 'MBUU  Malibu Boats', 'AIMC  Altra Industrial Motion', 'BLUE  Bluebird Bio', 'IMRA  IMARA', 'STM  STMicroelectronics', 'HFWA  Heritage Financial', 'FDIS  Fidelity MSCI Consumer Discretionary Index', 'LMND  Lemonade', 'SCHR  Schwab Intermediate Term US Treasury', 'CEG  Constellation Energy', 'RELX  RELX', 'MGK  Vanguard Mega Cap Growth', 'GFOF  GFOF ETF', 'DNLI  Denali Therapeutics', 'FHB  First Hawaiian', 'BLZE  Backblaze', 'PGR  Progressive', 'XMVM  Invesco S&P MidCap Value with Momentum ETF', 'RIOT  Riot Blockchain', 'CARZ  First Trust NASDAQ Global Auto Index Fund', 'PFFD  Global X US Preferred ETF', 'MITK  Mitek Systems', 'TWND  Tailwind Acquisition Corp', 'JMIA  Jumia Technologies', 'SPMO  PowerShares S&P 500 Momentum Portfolio', 'TDUP  ThredUp', 'CHIC  MSCI China Communication Services ETF', 'SPXS  Direxion Daily S&P Bear', 'IPGP  IPG Photonics', 'ALGT  Allegiant Travel', 'ARCB  ArcBest', 'ARKF  Ark Fintech Innovation', 'SFM  Sprouts Farmers Markets', 'RNST  Renasant', 'VNOM  Viper Energy', 'GPS  Gap', 'VEU  Vanguard FTSE All World Ex US', 'FREL  Fidelity MSCI Real Estate Index ETF', 'EGIO  Limelight Networks, Inc.', 'INTU  Intuit', 'CUBE  CubeSmart', 'IEFA  iShares Core MSCI', 'ICF  iShares Cohen and Steers Realty Majors Index Fund', 'ADTX  ADiTx Therapeutics', 'SPYX  SPDRFossil Fuel Free ETF', 'LPL  LG Display', 'ITM  Market Vectors Muni', 'VLUE  iShares MSCI USA Value Factor', 'SVRA  Savara', 'BSMX  Santander Mexico', 'ZYXI  Zynex', 'GROM  Grom Social Enterprises', 'AGCB  Altimeter Growth Corp 2 ', 'CTLP  Cantaloupe', 'ESGD  iShares MSCI EAFE ESG Select', 'ABEV  Ambev', 'BITI  BITI Stock', 'SBEV  Splash Beverage Group', 'CIXX  CI Financial', 'TACT  Transact Technologies', 'ALRM  Alarm.com', 'WST  West Pharmaceutical', 'ATHX  Athersys', 'NSPR  InspireMD', 'IRBT  iRobot', 'QYLD  Recon Capital Series Trust', 'IECS  iShares Evolved US Consumer Staples ETF', "TLYS  Tilly's", 'IMGN  ImmunoGen', 'FSR  Fisker', 'CPUH  Compute Health Acquisition Corp', 'WCC  WESCO', 'CANE  Teucrium Sugar Fund', 'TKC  Turkcell', 'MKFG  Markforged Holding Corp', 'OFIX  Orthofix', 'IVW  iShares S&P 500 Growth Index Fund', 'BFRI  Biofrontera Inc', 'FRG  Franchise Group', 'LU  Lufax Holdings', 'COST  Costco', 'AFCG  AFC Gamma', 'SELF  Self Storage Group', 'AL  Air Lease', 'LEG  Leggett & Platt', 'XHYT  XHYT ETF', 'NPO  EnPro', 'ISSC  Innovative Solutions', 'JHML  John Hancock Multifactor Large Cap ETF', 'HEES  H&E Equipment', 'MERC  Mercer', 'REW  ProShares UltraShort Technology', 'VALT  ETFMG Sit Ultra Short ETF', 'GOOG  Alphabet', 'CVAC  CureVac', 'AFIIQ  Armstrong Flooring Inc', 'FXE  Euro ETF', 'AES  AES Corp.', 'CCNE  CNB Financial', 'PPTA  Perpetua Resources', 'KMED  Kraneshares Emerging Mkts Healthcare ETF', 'HSII  Heidrick & Struggles', 'MRK  Merck', 'TWM  ProShares UltraShort Russell2000', 'BTRS  BTRS Holdings', 'SJB  ProShares Short High Yield', 'SITM  SiTime', 'IEME  iShares Evolved US Media and Entertainment ETF', 'JJA  iPath Bloomberg Agriculture ETN', 'CSL  Carlisle', 'REAL  RealReal', 'TECL  Direxion Daily Bull', 'TSQ  Townsquare Media', 'FUSN  Fusion Pharmaceuticals', 'XPO  XPO Logistics', 'CRL  Charles River', 'ADPT  Adaptive Biotechnologies', 'DCOM  Dime Community', 'LSAK  Net 1 UEPS Technologies Inc', 'OUSA  O Shares FTSE US Quality Dividend ETF', 'GNTX  Gentex', 'XME  SPDR S&P Metals and Mining', 'HYLB  Xtrackers USD High Yield Corporate Bond ETF', 'SPSB  SPDR Portfolio Short Term Corporate Bond ETF', 'VVNT  Vivint Smart Home', 'STE  STERIS', 'MTLS  Materialise', 'PDCE  PDC Energy', 'ORIC  Oric Pharmaceuticals', 'GTX  Garrett Motion', 'CW  Curtis Wright', 'RMBS  Rambus', '.SOX  PHLX Semiconductor Sector Index', 'PSTL  Postal Realty Trust', 'EVTC  Evertec', 'OBE  OBE Stock', 'CZOO  Cazoo Group', 'WING  Wingstop', 'SYM  SVF Investment Corp 3', 'NBSE  NeuBase Therapeutics', 'KOF  Coca Cola FEMSA', 'IDU  Ishares DJ Utilities Sector', 'STRA  Strayer', 'GROY  Gold Royalty Corp', 'CMS  CMS Energy', "CRI  Carter's", 'ERYP  Erytech Pharma SA', 'KBAL  Kimball International', 'SYPR  Sypris Solutions', 'SBNY  Signature Bank', 'SPAK  Defiance Next Gen Spac Derived ETF', 'ACN  Accenture', 'PASG  Passage Bio', 'VGLT  Vanguard Long Term Government Bond', 'ARCH  Arch Coal', 'ESRT  Empire State Realty', 'LYB  LyondellBasell', 'USRT  iShares Core US REIT ETF', 'GRAY  Graybug Vision', 'ADNT  Adient', 'SLGG  Super League Gaming', 'DV  DoubleVerify Holdings', 'SJI  South Jersey Industries', 'BUG  Global X CyberSecurity ETF', 'LTHM  Livent', 'AVGO  Broadcom', 'GMRE  Global Medical REIT', 'HITI  High Tide', 'CGRN  Capstone Green Energy', 'QTUM  Defiance Quantum ETF', 'IUSV  iShares Core US Value', 'KAI  Kadant', 'SPLK  Splunk', 'GENI  Genius Sports', 'VFVA  Vanguard US Value Factor ETF', 'GES  Guess', 'SPXV  ProShares S&P 500 Ex Health Care ETF', 'NVVE  Nuvve Holding Corp', 'IAA  IAA Inc', 'AGS  PlayAGS', 'BJRI  BJs', 'SCYX  Scynexis', 'USAP  Universal Stainless & Alloy', 'MET  MetLife', 'THR  Thermon Group', 'ARMP  Armata Pharmaceuticals', 'MSGR  MSGR ETF', 'NMTC  NeuroOne Medical Technologies', 'KC  Kingsoft Cloud Holdings', 'KDP  Keurig Dr Pepper', 'SECO  Secoo Holding', '.OEX  S&P 100 Index', 'BMED  Blackrock Future Health ETF', 'HAUZ  Xtrackers International Real Estate ETF', 'MMLP  Martin Midstream Partners', 'NOK  Nokia', 'PETS  PetMed Express', 'EXTR  Extreme Networks', 'SMBK  SmartFinancial', 'GREE  Greenridge Generation Holdings', 'BY  Byline Bancorp', 'SON  Sonoco Products', 'RWR  SPDR DJ REIT', 'PSO  Pearson', 'GXG  Global X FTSE', 'PACK  Ranpak Holdings', 'PERI  Perion Network', 'HSKA  Heska', 'BKSY  BlackSky Technology', 'MANU  Manchester United', 'CPG  Crescent Point Energy', 'NFBK  Northfield Bancorp', 'BLV  Vanguard Long Term Bond', 'BTEK  Blackrock Future Tech ETF', 'GDOC  Goldman Sachs Future Health Care ETF', 'JJSF  J & J Snack Foods', 'AIIQ  AI Powered International Equity ETF', 'ASB  Associated Banc-Corp', 'LIZI  LIZHI Inc', 'IRM  Iron Mountain', 'ANAB  AnaptysBio', 'MACK  Merrimack', 'ALBO  Albireo', 'MGEE  MGE Energy', 'STRY  FirstMark Horizon Acquisition Corp', 'VET  Vermilion Energy', 'KSTR  Kraneshares Sse Star Market 50 Index ETF', 'SRRK  Scholar Rock Holdings', 'UI  Ubiquiti', 'AMX  AMX', 'PAA  Plains All American', 'MITT  AG Mortgage', 'SAVE  Spirit Airlines', 'WERN  Werner', 'TDOC  Teladoc', 'LADR  Ladder Capital', 'RE  Everest Re', 'KBR  KBR', 'DTE  DTE Energy', 'VMW  VMware', 'CRD/A  Crawford and Company', '.SIXV  S&P Health Care', 'FYX  First Trust Small Cap Core AlphaDEX Fund', 'GPRK  GeoPark', 'SCHL  Scholastic', 'HCM  Hutchinson China MediTech', 'NVRO  Nevro', 'XLF  Financial Sector SPDR ETF', 'WELL  Welltower', 'HQY  HealthEquity', 'NFE  New Fortress Energy', 'XTN  SPDR S&P Transportation', 'DRE  Duke Realty', 'EFX  Equifax', 'BKI  Black Knight', 'LCUT  Lifetime Brands', 'XLV  Health Care Sector SPDR ETF', 'SKYY  First Trust ISE Cloud Computing Index Fund', 'CRBU  Caribou Biosciences', 'FGD  First Trust DJ Global Select Dividend Index Fund', 'SRC  Spirit Realty', 'YETI  YETI Holdings', 'TGT  Target', 'INMB  INmune Bio', 'LIND  Lindblad Expeditions', 'PXQ  Power Shares Dynamic Networking Portfolio', 'VRTV  Veritiv', 'NXGN  NextGen Healthcare', 'VOD  Vodafone', 'MRVI  Maravai LifeSciences', 'SGRY  Surgery Partners', 'QQQM  Invesco NASDAQ 100 ETF', 'QEMM  SPDR MSCI Emerging Markets Quality Mix', 'ITCI  Intra Cellular Therapies', 'NINE  Nine Energy Service', 'ROAD  Construction Partners', 'UL  Unilever', 'MGNI  Magnite Inc', 'SCOR  comScore', 'EGP  EastGroup Properties', 'AAU  Almaden Minerals', 'KD  Kyndryl Holdings', 'ASX  Advanced Semiconductor', 'EYEN  Eyenovia', 'GATO  Gatos Silver', 'NRT  North European Oil Royalty Trust', 'INGR  Ingredion', 'PCEF  PowerShares CEFome Composite Portfolio', 'CMI  Cummins', 'GUNR  Global Upstream Natural Recources', 'FLTN  FLTN ETF', 'AUPH  Aurinia Pharmaceuticals', 'NUVB  Nuvation Bio', 'LRN  K12', 'MUNI  PIMCO Intermediate Muni', 'CNET  ChinaNet Online', 'AMGN  Amgen', 'REFR  Research Frontiers', 'CFLT  Confluent', 'NRC  National Research Corporation', 'DTM  DT Midstream', 'NBIX  Neurocrine', 'NRG  NRG Energy', 'ALC  Alcon', 'SII  Sprott', 'FRC  First Republic', 'GOVZ  iShares 25 Year Treasury STRIPS Bond ETF', 'AQWA  Global X Clean Water ETF', 'VCTR  Victory Capital Holdings', 'GBTG  Global Business Travel Group', 'SHCR  Sharecare', 'MUR  Murphy Oil', 'ISTB  iShares Core 1-5 Year USD Bond ETF', 'IIIN  Insteel', 'DLX  Deluxe', 'VGT  Vanguard Information Technology ETF', 'EWA  iShares Australia', 'CASY  Caseys General Stores', 'CNRG  SPDR Kensho Clean Power ETF', 'ISEE  IVERIC bio', 'CAH  Cardinal Health', 'TRP  Transcanada', 'CPA  Copa Holdings', 'LTRX  Lantronix', 'MAC  Macerich', 'TEVA  Teva', 'SGMO  Sangamo', 'IDNA  iShares Genomics Immunology and Healthcare ETF', 'FLQL  Franklin LibertyQ US Equity ETF', 'EVAX  Evaxion Biotech', 'ARCE  Arco Platform Ltd', 'BALY  Ballys', 'ASML  ASML', 'SIOX  Sio Gene Therapies', 'URTH  iShares MSCI World', 'SPIB  SPDR Portfolio Intermediate Term Corp Bond ETF', 'LLY  Eli Lilly', 'CLVT  Clarivate', 'VWOB  Vanguard Emerging Markets Government Bond', 'PLYM  Plymouth Industrial REIT', 'ORMP  Oramed', 'PSX  Phillips 66', 'ICVT  iShares Convertible Bond ETF', 'GSBD  Goldman Sachs BDC', 'QS  Quantumscape', 'OMEX  Odyssey Marine', 'SSD  Simpson Manufacturing', 'GILD  Gilead', 'KN  Knowles', 'XP  XP Inc', 'GMDA  Gamida Cell Ltd', 'ADTN  ADTRAN', 'RXRX  Recursion Pharmaceuticals', 'UBR  ProShares Ultra MSCI Brazil Capped', 'JGLD  Amplify Pure Junior Gold Miners ETF', 'IXP  iShares Telecom', 'RIO  Rio Tinto', 'EMFQ  Amplify Emerging Mkts Fintech ETF', 'JYNT  Joint Corp.', 'ARQQ  Arqit Quantum', 'MQ  Marqeta', 'NRIX  Nurix Therapeutics', 'SGHC  Super Group Limited', 'ZETA  Zeta Global Holdings', 'KLXE  KLX Energy Services', 'AMZN  Amazon', 'UNF  UniFirst', 'PTR  PetroChina', 'ARW  Arrow Electronics', 'GNK  Genco Shipping', 'AGLE  Aeglea Biotherapeutics', 'TFII  TFI International', 'EXAI  Exscientia PLC', 'GTN  Gray Television', 'CDK  CDK Global', 'SPAB  SPDR Portfolio Aggregate Bond ETF', 'NKE  Nike', 'O  Realty Income', 'ZWS  Zurn Water Solutions', 'GDEN  Golden Entertainment', 'MMAT  Meta Materials', 'CLDS  Direxion Daily Cloud Computing Bear 2x', 'XRAY  Dentsply Sirona', 'CFB  CrossFirst Bankshares', "CRMT  America's Car-Mart", 'WTRH  Waitr Holdings', 'IBRX  ImmunityBio', 'OUNZ  Van Eck Merk Gold Trust', 'EURL  Direxion Daily FTSE Europe Bull 3x Shares', 'RYT  Guggenheim S&P 500 Equal Weight Technology', 'JEPI  Jpmorgan Equity Premium Income ETF', 'IPKW  PowerShares International BuyBack Achievers Portfolio', 'BNED  Barnes & Noble Education', 'GAIA  Gaia', 'TTCF  Tattooed Chef', 'GRMN  Garmin', 'FRO  Frontline', 'STC  Stewart', 'CUT  Guggenheim Timber', 'VEEV  Veeva', 'HMY  Harmony Gold', 'SONY  Sony Group', 'EQWL  PowerShares Russell Top 200 Equal Weight Portfolio', 'WIMI  WiMi Hologram Cloud ', 'GSY  Guggenheim Enhanced Short Duration ETF', 'COO  Cooper Companies', 'CPSS  Consumer Portfolio', 'SMSI  Smith Micro', 'WCLD  WisdomTree Cloud Computing ETF', 'UFO  Procure Space ETF', 'DTEC  ALPS Disruptive Technologies ETF', 'CNTY  Century Casinos', 'HOMZ  Hoya Capital Housing ETF', 'DRN  Direxion Bull 3x', 'IFN  India Fund', 'ACT  Enact Holdings', 'SLDP  Solid Power', 'KLAC  KLA-Tencor', 'PTE  PolarityTE', 'VLO  Valero', 'PAYS  Paysign', 'CSTR  CapStar Financial', 'CMDY  iShares Bloomberg Roll Select Commodity Strategy ETF', 'ROOT  Root Inc', 'LC  LendingClub', 'HXL  Hexcel', 'SKIN  Beauth Health Company', 'CCO  Clear Channel Outdoor', 'CALA  Calithera Biosciences', 'ENG  ENGlobal', 'TRMR  Tremor International', 'ADUS  Addus HomeCare', 'NGS  Natural Gas Services', 'SCPH  scPharmaceuticals', 'AME  Ametek', 'SWCH  Switch', 'NEON  Neonode', 'BRZU  Direxion Brazil Bull', 'AGRX  Agile Therapeutics', 'EDOC  Global X Telemedicine and Digital Health ETF', 'MZZ  ProShares UltraShort MidCap400', 'PFF  iShares S&amp Preferred Stock', 'JHG  Janus Henderson Group', 'MOAT  Market Vectors Morningstar Wide Moat ETF', 'ALFA  AlphaClone Alternative Alpha', 'SEEL  Seelos Therapeutics', 'CSTL  Castle Biosciences', 'WBA  Walgreen Boots', 'UAVS  AgEagle Aerial Systems', 'LHDX  Lucira Health', 'CDNA  CareDx', 'RKDA  Arcadia Biosciences', 'EZM  WisdomTree MidCap Earnings Fund', 'GGAL  Grupo Financiero', 'OPFI  OppFi', 'DVA  DaVita', 'EVLV  Evolv Technologies Holdings', 'BHG  Bright Health Group', 'SPSC  SPS Commerce', 'BFOR  Barrons', 'HYLV  IQ S&P High Yield Low Volatility Bond ETF', 'WISH  ContextLogic', 'CLBS  Caladrius', 'PLMR  Palomar Holdings', 'EZU   iShares Emu Index', 'AGZ  iShares Agency Bond', 'MRNS  Marinus Pharmaceuticals', 'TEL  TE Connectivity', 'MNMD  MindMed', 'VCSH  Vanguard Short Term', 'HUBS  HubSpot', 'GMET  VanEck Green Metals ETF', 'PIO  PowerShares Global Water Portfolio', 'VNDA  Vanda', 'RIVN  Rivian', 'MNSO  MINISO Group Holding Limited', 'RTLR  Rattler Midstream', 'QUBT  Quantum Computing', 'SD  SandRidge Energy', 'SCHH  Schwab REIT', 'VLRS  Volaris', 'RING  iShares MSCI Global Miners', 'ITW  Illinois Tool Works', 'ALPN  Alpine Immune Sciences', 'BNO  United States Brent Oil Fund', 'VRNT  Verint', 'RIDE  Lordstown Motors', 'GAN  GAN Ltd', 'SAFT  Safety Insurance', 'UIHC  United Insurance', 'TPOR  Direxion Daily Transportation Bull 3x ETF', 'GSBC  Great Southern Bancorp', 'ASC  Ardmore Shipping', 'ONTO  Onto Innovation', 'LOB  Live Oak Bancshares', 'OEC  Orion Engineered', 'ADAP  Adaptimmune', 'ZH  Zhihu', 'AEMD  Aethlon Medical', 'HYPR  HealthCor Catalio Acquisition Corp', 'GFF  Griffon', 'IAG  IAMGOLD', 'DRQ  Dril Quip', 'NRGV  Energy Vault Holdings', '.XDC  PHLX Canadian Dollar Index', 'SLVP  iShares MSCI Global Silver Miners', 'KWR  Quaker Chemical', 'HTEC  Robo Global Healthcare Technology and Innovation ETF', 'ASPS  Altisource', 'ATRA  Atara Biotherapeutics', 'PPC  Pilgrims Pride', 'UTI  Universal Technical', 'CUE  Cue Biopharma', 'RMD  ResMed', 'VIEW  View Inc', 'PRPL  Purple Innovation', 'BDSX  Biodesix', 'EGHT  8x8', 'SNT  Senstar Technologies', 'RJI  AB Svensk Ekportkredit', 'XSVM  Invesco S&P SmallCap Value with Momentum ETF', 'UNL  United States 12 Month Natural Gas Fund LP', 'FSP  Franklin Street', 'PFSI  PennyMac Financial', 'ABCB  Ameris Bancorp', 'KRBP  Kiromic BioPharma', 'LRGF  iShares FactorSelect MSCI USA ETF', 'ESGR  Enstar Group', 'CAPD  Cap iPath Shiller CAPE ETN', 'AGI  Alamos Gold', 'ACVA  ACV Auctions', 'ACH  CHINALCO', 'BSY  Bentley Systems', 'KTCC  KeyTronicEMS', 'HTH  Hilltop Holdings', 'WD  Walker & Dunlop', 'DSTL  Distillate US Stability Value ETF', 'UPWK  Upwork', 'OSPN  OneSpan', 'GBLD  Invesco MSCI Green Building ETF', 'MORT  Market Vectors Mortgage REIT Income', 'CHAD  Direxion China Bear', 'TWKS  Thoughtworks', 'TFSL  TFS Financial', 'SSP  EW Scripps', 'HGER  HGER ETF', 'BCOR  Blucora', 'HBAN  Huntington Bancshares', 'SPIR  Spire Global', 'DVN  Devon Energy', 'NARI  Inari Medical', 'RZV  Guggenheim S&P Smallcap 600 Pure Value ETF', 'FINX  Global X FinTech Thematic', 'DX  Dynex Capital', 'JO  iPath Coffee ETN', 'MCY  Mercury General', 'EPRT  Essential Properties', 'VVOS  Vivos Therapeutics', 'LZB  La Z Boy', 'HALL  Hallmark Financial', 'EGAN  eGain', 'IMOS  ChipMOS', 'CMCM  Cheetah Mobile', 'NEU  NewMarket', 'IRIX  IRIDEX', 'CNOB  ConnectOne Bancorp', 'SOI  Solaris Oilfield', 'SIZE  iShares MSCI USA Size Factor', 'CDRE  Cadre Holdings Inc', 'AVUV  Avantis US Small Cap Value ETF', 'SWAN  Amplify Blackswan Growth & Treasury Core ETF', 'TREX  Trex Company', 'VBTX  Veritex', 'EXP  Eagle Materials', 'AZTA  Brooks Automation', 'SCL  Stepan Company', 'VTWO  Vanguard Russell', 'GURU  Global X Guru', 'FATE  Fate Therapeutics', 'HBI  HanesBrands', 'SMFG  Sumitomo Mitsui', 'HGEN  Humanigen', 'MYRG  MYR Group', 'SNBR  Sleep Number', 'AGX  Argan', 'AP  Ampco-Pittsburgh', 'NUZE  NuZee', 'RGI  Guggenheim S&P 500 Equal Weight Industrials', 'OIL  iPath Pure Beta Crude Oil', 'OCUL  Ocular Therapeutix', 'PBYI  Puma Biotechnology', 'CCRN  Cross Country Healthcare', 'URBN  Urban Outfitters', 'CFFN  Capitol Federal Financial', 'NNDM  Nano Dimension', 'JBGS  JBG SMITH Properties', 'PGC  Peapack Gladstone Financial', 'ELA  Envela Corp', 'BYTE  IO Digital Infrastructure ETF', 'RHP  Ryman Hospitality', 'CECE  CECO Environmental', 'BERY  Berry Plastics', 'ARVN  Arvinas', 'GRBK  Green Brick', 'AMG  Affiliated Managers', 'VFC  V F Corp', 'AVO  Mission Produce', 'CNA  CNA Financial', 'EWU  iShares UK', 'ORLA  Orla Mining', 'CB  Chubb', 'NXE  NexGen Energy', 'HRZN  Horizon Technology', 'LGND  Ligand', 'OCSL  Oaktree Specialty Lending', 'MCW  Mister Car Wash', 'DJD  Guggenheim DJ Industrials  Dividend ETF', 'DIN  DineEquity', 'GME  GameStop', 'GOEV  Canoo', 'CQP  Cheniere Partners', 'RRGB  Red Robin', 'WFG  West Fraser Timber', 'LTRN  Lantern Pharma', 'BBBY  Bed Bath & Beyond', 'PLD  Prologis', 'EWK  iShares Belgium', 'GCP  GCP Applied Technologies', 'DCP  DCP Midstream', 'CPE  Callon Petroleum', 'TGI  Triumph Group', '.XSP  Mini SPX ', 'MCK  McKesson', 'ASO  Academy Sports and Outdoors', 'MU  Micron', 'RBBN  Ribbon Communications', 'IWY  iShares Russell Top 200 Growth', 'NTB  Butterfield', 'NRDY  Nerdy', 'SGHT  Sight Sciences Inc', 'DLO  DLocal Ltd', 'MAPS  WM Technology', 'ET  Energy Transfer', 'CNNE  Cannae Holdings', 'VO  Vanguard Mid Cap ETF', 'XNCR  Xencor', 'DEA  Easterly Government Properties', 'KRG  Kite Realty Group', 'AMSF  AMERISAFE', 'SHY  iShares Barclays 1 to 3 Year Treasury Bond Fund', 'HNGR  Hanger Inc', 'GNLN  Greenlane Holdings', 'DSKE  Daseke', 'SLDB  Solid Biosciences', 'DAX  Recon Capital DAX Germany', 'ITB  iShares DJ US Home Construction Index Fund', 'IAT  iShares DJ US Regional Banks Index Fund', 'CNDT  Conduent', 'UP  Wheels Up Experience', 'VIG  Vanguard Dividend Appreciation ETF', 'KROP  Global X AgTech and Food Innovations ETF', 'NCMGF  Newcrest Mining Limited', 'CRVL  CorVel', 'LWLG  Lightwave Logic', '.RUT  Russell Index', 'PIE  PowerShares DWA Emerging Markets Momentum Portfolio', 'UCTT  Ultra Clean', 'GMAB  Genmab AS', 'HES  Hess Corp.', 'XNET  Xunlei', 'TKR  Timken', 'OIH  Market Vectors Oil Services', 'TT  Trane Technologies', 'DCFC  Tritium DCFC', 'SQQQ  ProShares UltraPro Short QQQ', 'BRKR  Bruker', 'CHIE  Global X China Energy', 'LPCN  Lipocine', 'SANW  S&W Seed', 'XYLG  Global X S&P 500 Covered Call and Growth ETF', 'VGR  Vector Group', 'CRESY  Cresud SA', 'KOPN  Kopin', 'PBT  Permian Basin Royalty Trust', 'CFA  Victory CEMP US 500 Volatility Weighted Index', 'ICVX  Icosavax', 'PKX  Posco', 'CLSK  CleanSpark', 'IEP  Icahn Enterprises', 'RDVY  First Trust NASDAQ Rising Dividend Achievers', 'XBIT  XBiotech', 'PDFS  PDF Solutions', 'SNCY  Sun Country Airlines Holdings', 'VCAR  Simplify Volt Robocar Disruption and Tech ETF', 'AVRO  AVROBIO', 'RJZ  AB Svensk Ekportkredit', 'STAG  Stag Industrial', "PTLO  Portillo's", 'ALTY  Global X SuperDividend Alternatives', 'VAL  Valaris Limited', 'ATCX  Atlas Technical Consultants', 'XWEB  SPDR S&P Internet ETF', 'DORM  Dorman Products', 'AMRN  Amarin', 'FBRT  Capstead Mortgage Corp', 'ENR  Energizer', 'RWL  Large Cap Revenue ETF', 'EWEB  Global X Emerging Markets Internet and E Commerce ETF', 'HELX  Franklin Genomic Advancements ETF', 'ATIP  ATI Physical Therapy', 'HYLD  Peritus High', 'SMIN  iShares MSCI India Small Cap ETF', 'HURN  Huron', 'KOMP  SPDR S&P Kensho New Economies Composite ETF', 'AVD  American Vanguard', 'NFLX  Netflix', 'PHIO  Phio Pharmaceuticals Corp', 'UNG  U.S. Natural Gas Fund', 'ERTH  Invesco MSCI Sustainable Future ETF', 'BHF  Brighthouse Financial', 'ACIW  ACI Worldwide', 'CYCN  Cyclerion Therapeutics', 'SESN  Sesen Bio', 'QQEW  NASDAQ 100 Equal Weighted Index Fund', 'TUSK  Mammoth Energy', 'FCPI  Fidelity Stocks for Inflation ETF', 'GAZ  iPath Series B Nat Gas ETN', 'NEM  Newmont Mining', 'OCN  Ocwen Financial', 'UGA  UGA', 'SOCL  Global X Social Media Index', 'ACOR  Acorda Therapeutics', 'VERU  Veru', 'ALGM  Allegro MicroSystems', 'EOLS  Evolus', 'EEV  Proshares UltraShort MSCI', 'AIT  Applied Industrial Technologies', 'SIEN  Sientra', 'DLNG  Dynagas LNG', 'MKTW  MarketWise', 'XTL  SPDR S&P Telecom', 'CTSO  CytoSorbents', 'CCLP  CSI Compressco', 'NGVC  Natural Grocers', 'RADA  RADA Electronic', 'UBX  Unity Biotechnology', 'DCGO  DocGo', 'SXI  Standex', 'CN  Deutsche X trackers MSCI All China Equity ETF', 'KR  Kroger', 'ZDGE  Zedge', 'XMMO  Invesco S&P MidCap Momentum ETF', 'UA  Under Armour', 'SPY  SPDR S&P 500', 'NOV  National Oilwell', 'CGUS  CGUS ETF', 'SAIL  SailPoint Technologies', 'MDT  Medtronic', 'XITK   Innovative Technology ETF', 'ALRS  Alerus Financial', 'CRM  Salesforce', 'CYTO  Altamira Therapeutics', 'TMV  Direxion Daily 20 plus Year Treasury Bear 3X Shares', 'AADI  Aadi Bioscience', 'LXU  LSB Industries', 'TMFC  Motley Fool 100 Index ETF', 'ARIS  Aris Water Solutions Inc', 'TFC  Truist Financial', 'BSQR  Bsquare Corp', 'PUK  Prudential Plc', 'KMLM  KFA Mount Lucas Index Strategy ETF', 'PD  PagerDuty', 'CHAU  Direxion CSI 300 China A Share Bull 2X', 'OM  Outset Medical', 'YMAB  Y mAbs Therapeutics', 'PBPB  Potbelly', 'BKD  Brookdale Senior Living', 'DOCU  DocuSign', 'AROW  Arrow Financial', 'HAYN  Haynes', 'ZI  ZoomInfo Technologies', 'FVAL  Fidelity Value Factor ETF', 'MPLX  MPLX', 'USAI  Pacer American Energy Independence ETF', 'CORN  Teucrium Corn Fund', 'HYZD  WisdomTree BofA Merrill Lynch High Yield Bond Zero Duration Fund', 'LETB  LETB ETF', 'CTSH  Cognizant', 'TXRH  Texas Roadhouse', 'LOTZ  CarLotz', 'WEBS  Direxion Daily Dow Jones Internet Bear 3x Shares', 'HAIN  Hain Celestial', 'SPLV  PowerShares S&P 500 Low Volatility Portfolio', 'EIGR  Eiger BioPharmaceuticals', 'NTGR  NETGEAR', 'QLYS  Qualys', 'INFL  Horizon Kinetics Inflation Beneficiaries ETF', 'ENIC  Enersis Chile', 'KRMA  Global X Conscious Companies', 'VNTR  Venator Materials', 'BOAT  Sonicshares Global Shipping ETF', 'FUTU  Futu Holdings', 'CMF  iShares California AMT Free Muni Bond', 'KORU  Direxion Daily South Korea Bull 3X Shares', 'PAX  Patria Investments', 'RYH  S&P 500 Equal Weight Healthcare', 'PSQ  ProShares Short QQQ', 'FXP  FTSE  China 25', 'BSBR  Santander Brasil', 'EMQQ  EMQQ The Emerging Markets Internet and Ecommerce ETF', 'AMTI  Applied Molecular Transport', 'CWAN  Clearwater Analytics', 'EBAY  EBay', 'BNTX  BioNTech SE', 'TLH  iShares Barclays 10 to 20', 'ERO  Ero Copper Corporation', 'MEDP  Medpace', 'RFIL  RF Industries', 'AXTI  AXT', 'CX  Cemex', 'WMC  Western Asset Mortgage', 'XSW  SPDR S&P Software and Services', 'CHIK  Global X MSCI China IT ETF', 'IRDM  Iridium', 'NVT  nVent Electric', 'REGN  Regeneron', 'FTNT  Fortinet', 'CASA  Casa Systems', 'KNSA  Kiniksa Pharmaceuticals', 'POSH  Poshmark', 'FTDR  FrontDoor', 'GDX  Gold Miners ETF ', 'SRT  StarTek', 'CS  Credit Suisse', 'KINS  Kingstone Companies', 'SHPW  Shapeways Holdings', 'DIBS  1stdibs com Inc', 'CCJ  Cameco', 'KLCD  KFA Large Cap Quality Dividend Index ETF', 'EFZ  ProShares Short MSCI', 'PVH  PVH Corp.', 'PRFZ  PowerShares FTSE RAFI US 1500 Small Mid Portfolio', 'REM  IShares Mortgage Real Estate Capped', 'PCT  PureCycle Technologies ', 'FHLC  Fidelity MSCI Health Care Index', 'IMXI  International Money Express', 'AKYA  Akoya BioSciences', 'MLSS  Milestone Scientific', 'EIG  Employers Holdings', 'SH  ProShares Short S&P 500', 'GOTU  GSX Techedu', 'PXJ  PowerShares Oil and Gas', 'EOG  EOG Resources', 'SNP  China Petroleum', 'MDXG  MiMedx', 'PEP  PepsiCo', 'VGZ  Vista Gold', 'TGA  Transglobe', 'VCSA  TPG Pace Solutions Corp', 'KMDA  Kamada', 'CSWC  Capital Southwest Corp', 'CHT  Chunghwa Telecom', 'EUFN  iShares MSCI Europe', 'AIVL  Wisdomtree US AI Enhanced Value ETF', 'AXON  Axon Enterprise', 'CLNN  Clene', 'IHS  IHS Holding Ltd', 'PFG  Principal Financial', 'CRGE  CRGE Stock', 'TAOP  Taoping', 'SIVB  SVB Capital II', 'SBSW  Sibanye Stillwater', 'SARK  Tuttle Capital Short Innovation ETF', 'ASIX  AdvanSix', 'XSLV  PowerShares S&P SmallCap Low Volatility Portfolio', 'CRPT  First Trust Crypto Industry ETF', 'PENN  Penn National', 'OR  Osisko Gold Royalties', 'NEWP  New Pacific Metals', 'BATRA  Liberty Media Series A', 'XPER  Xperi Corp', 'EGBN  Eagle Bancorp', 'BXRX  Baudax Bio', 'LGLV  SPDR Russell 1000 Low Volatility ETF', 'NLS  Nautilus', 'LII  Lennox', 'HGV  Hilton Grand Vacations', 'SBTX  Silverback Therapeutics', 'PXI  PowerShares DWA Energy Momentum Fund', 'MPLN  Churchill Capital Corp', 'MDU  MDU Resources', 'CATO  Cato Corp.', 'DBI  Designer Brands', 'SDIG  Stronghold Digital Mining ', 'BLCN  Reality Shares Nasdaq NextGen Economy', 'XM  Qualtrics International', 'OCUP  Ocuphire Pharma', 'NWS  News Corp', 'STAA  STAAR Surgical', 'DADA  Dada Nexus', 'BTF  Valkyrie Bitcoin Strategy ETF', 'KMT  Kennametal', 'UPS  UPS', 'SELB  Selecta Biosciences', 'GNSS  LRAD Corp', 'CURO  CURO Group', 'OMAB  Grupo Aeroportuario del Centro Norte SAB de CV', 'ETD  Ethan Allen Interiors', 'AMRC  Ameresco', 'ENZ  Enzo Biochem', 'ABG  Asbury Automotive', 'CUK  Carnival Plc', 'EQC  Equity Commonwealth', 'SYY  Sysco', 'PMCB  PharmaCyte Biotech', 'IVV  IShares S&P 500 Index Shares', 'BRT  BRT Realty Trust', 'DOGZ  Dogness Internationa', 'ANET  Arista Networks', 'XHYC  XHYC ETF', 'PEJ  PowerShares Dynamic Leisure and Entertainment Portfolio', 'WMS  Advanced Drainage', 'ALDX  Aldeyra', 'CIGI  Colliers International', 'ROST  Ross Stores', 'AAWW  Atlas Air', 'NATI  National Instruments', 'SSSS  Sutter Rock Capital', 'BRDS  Switchback II', 'FNGG  Direxion Large Cap FANG Bull 2x ETF', 'ROKU  Roku', 'FNDC  Schwab Fundamental International Small Company Index', 'PHR  Phreesia', 'BKE  Buckle', 'FNB  F.N.B.', 'LQD  iShares Investment Grade Corporate Bond ETF', 'GP  GreenPower Motor Company', 'IYK  iShares US Consumer Goods', 'FOXF  Fox Factory', 'AVYA  Avaya Holdings', 'TDW  Tidewater', 'SKT  Tanger Factory', 'USL  United States 12 Month Oil Fund LP', 'CWH  Camping World', 'WK  Workiva', 'ALZN  Alzamend Neuro', 'JNK  SPDR Barclays Capital High Yield Bond', 'CYH  Community Health', 'NIU  Niu Technologies', 'ZG  Zillow', 'SOLO  Electrameccanica Vehicles Corp', 'RCL  Royal Caribbean', 'LYV  Live Nation', 'XT  iShares Exponential Technologies ETF', 'EFAV  iShares MSCI EAFE', 'LMACA  Liberty Media Acquisition Corp', 'TEAM  Atlassian', 'FARM  Farmer Bros.', 'GROW  U S Global Investors', 'SAIC  Science Applications', 'ETAC  E Merge Technology Acquisition Corp', 'COIN  Coinbase', 'POTX  Global X Cannabis', 'ASTS  AST SpaceMobile', 'GSK  GlaxoSmithKline', 'FLIC  First Of Long Island', 'AMYT  Amryt Pharma', 'SBSI  Southside Bancshares', 'ECVT  Ecovyst', 'DOCN  DigitalOcean Holdings', 'KMPH  KemPharm', 'WLY  John Wiley &amp; Sons Inc', 'NM  Navios Maritime', 'NBHC  National Bank', 'TK  Teekay Corp.', 'NYMT  New York Mortgage', 'BMBL  Bumble', 'HTOO  Fusion Fuel Green PLC', 'MBI  MBIA', 'GB  Global Blue Group', 'INSP  Inspire Medical Systems', 'IVES  Wedbush ETFMG Global Cloud Technology ETF', 'COMS  ComSovereign Holding Corp', 'MGRC  McGrath RentCorp', 'ON  ON Semiconductor', 'RAVE  Rave Restaurant', 'PEY  PowerShares High Yield Equity Dividend', 'RWGV  Direxion Russell 1000 Growth Over Value ETF', 'FCX  Freeport McMoRan', 'OGI  Organigram Holdings', 'GCLN  GS Bloomberg Clean Energy ETF', 'IMH  Impac Mortgage', 'APLT  Applied Therapeutics', 'CELC  Celcuity', 'EWCZ  European Wax Center Inc (Class A Stock)', 'ONCS  OncoSec', '.OSX  PHLX Oil Service Sector Index', 'FUL  H B Fuller', 'FIXX  Homology Medicines', 'VERS  VERS ETF', 'IPAY  PureFunds ISE Mobile Payments ETF', 'FDP  Fresh Del Monte', 'SRRA  Sierra Oncology', 'XFOR  X4 Pharmaceuticals', 'PTLC  Pacer Trendpilot 750 ETF', 'CVBF  CVB Financial', 'BFI  BurgerFi International', 'PL  Planet Labs PBC', 'PVL  Permianville Royalty Trust', 'NXPI  NXP Semiconductors', 'WMB  Williams', 'DUST  Gold Bear 3x', 'PFBC  Preferred Bank', 'DBA  PowerShares Agriculture', 'ENSG  Ensign Group', 'HUT  Hut 8 Mining', 'GDYN  Grid Dynamics Holdings', 'GXTG  Global X Thematic Growth ETF', 'XES  SPDR S&P Oil and Gas Equipment and Services', 'OXM  Oxford Industries', 'INFI  Infinity Pharmaceuticals', 'GABC  German American Bancorp', 'BELFB  Bel Fuse', 'NBRV  Nabriva Therapeutics', 'BITQ  Bitwise Crypto Innovators Etf', 'WRE  Washington REIT', 'AHT  Ashford Hospitality', 'AVLR  Avalara', 'SKM  SK Telecom', 'AAP  Advance Auto Parts', 'KYN  Kayne Anderson', 'MINT  PIMCO Enhanced Maturity', 'ELYS  Elys Game Tech', 'NGL  NGL Energy Partners', 'FXLV  F45 Training Holdings', 'ATVI  Activision Blizzard', 'SLYV  SPDR S&P 600 Small Cap Value', 'ERIE  Erie Indemnity', "REED  Reed's", 'RSX  Market Vectors Russia', 'LNT  Alliant Energy', 'EME  EMCOR', 'BIGC  BigCommerce Holdings ', 'FTRI  First Trust II Global Natural Resources Income', 'GKOS  Glaukos', 'FUBO  fuboTV', 'GNRC  Generac', 'WH  Wyndham Hotels & Resorts', 'INMD  InMode', 'USDP  USD Partners', 'FNCH  Finch Therapeutics', 'OTIC  Otonomy', 'REZI  Resideo Technologies', 'KE  Kimball Electronics', 'UNH  UnitedHealth', 'IWD  iShares Russell 1000 Value Index Fund', 'BRPM  B Riley Principal 150 Merger Corp', 'DEEP  Roundhill Acquirers Deep Value ETF', 'DNMR  Danimer Scientific', 'SPTS  SPDR Portfolio Short Term Treasury ETF', 'SG  Sweetgreen', 'CRK  Comstock Resources', 'COWZ  Pacer US Cash Cows ETF', 'FNDB  Schwab Fundamental US Broad Market Index', 'DS  Drive Shack', 'INSE  Inspired Entertainment', 'ABB  ABB', 'BAM  Brookfield', 'WLMS  Williams Industrial Services Group', 'CIVI  Bonanza Creek Energy', 'KALV  KalVista', 'USNA  USANA', 'BKNG  Booking Holdings', 'HIPO  Hippo Holdings', 'DHX  DHI Group', 'EINC  Energy Income ETF', 'HSC  Harsco', 'OZK  Bank OZK', 'TR  Tootsie Roll', 'EXK  Endeavour Silver', 'ABIO  ARCA Biopharma', 'DFAX  World ex-US Core Equity 2 ETF', 'FYT  First Trust Small Cap Value AlphaDEX Fund', 'SVIX  SVIX ETF', 'UPST  Upstart Holdings', 'FPXI  First Trust International IPO', 'MYOV  Myovant Sciences', "ORLY  O'Reilly Automotive", 'AWR  American States Water', '.RUI  Russell 1000 Index', 'INVE  Identiv', 'DGS  WidomTree Emerging Dividend', 'CALM  Cal Maine Foods', 'MNST  Monster Beverage', 'SCPL  SciPlay Corp', 'NGVT  Ingevity', 'NEOG  Neogen', 'PSP  PowerShares Global Listed Private Equity Portfolio', 'ULE  ProShares Ultra Euro', 'BOSS  Linkage Technologies', 'VUZI  Vuzix', 'ACHV  Achieve Life Sciences', 'CR  Crane', 'X  US Steel', 'MRKR  Marker Therapeutics', 'WHF  WhiteHorse Finance', 'KMI  Kinder Morgan', 'BUYZ  Franklin Disruptive Commerce ETF', 'ENTA  Enanta', 'ING  ING Group', 'IRTC  iRhythm Technologies', 'SAFE  Safety Income & Growth', 'EEX  Emerald Expositions Events', 'XLU  Utilities Sector SPDR ETF', 'CCS  Century Communities', 'WAB  Wabtec', 'AMLX  Amylyx Pharmaceuticals', 'BOXD  Boxed', 'DLR  Digital Realty', 'BANF  BancFirst', 'MGA  Magna', 'VOO  Vanguard S&P 500', 'LIQT  LiqTech', 'HLF  Herbalife', 'PCTY  Paylocity', 'SUN  Sunoco', 'IFRX  InflaRx NV', 'CCXI  ChemoCentryx', 'ODDS  ODDS ETF', 'MMI  Marcus & Millichap', 'CLSN  Celsion', 'EPS  WisdomTree Earnings 500 Fund', 'MESO  Mesoblast', 'JBL  Jabil Circuit', 'FTGC  First Trust Global Tactical Commodity Strategy Fund', 'EWY   iShares South Korea', 'BCM  iPath Pure Beta Broad Commodity', 'SWEB  SWEB ETF', 'SNCR  Synchronoss', 'ARLP  Alliance Resource Partners', 'FXB  British Pound Sterling', 'SLP  Simulations Plus', 'ATER  Aterian', 'PBE  Power Shares Dynamic Biotechnology and Genome Portfolio', 'EQAL  PowerShares Exchange Traded Fund Trust II', 'AIA  iShares Asia 50', 'SPPI  Spectrum', 'RSP  Guggenheim S&P 500 Equal Weight', 'DRRX  Durect', 'MAR  Marriott', 'MRVL  Marvell', 'WPM  Wheaton Precious Metals', 'ARES  Ares Management', 'HSIC  Henry Schein', 'VRSK  Verisk Analytics', 'SLS  Sellas Life Sciences', 'ZY  Zymergen', 'CRHC  Cohn Robbins Holdings', 'MLM  Martin Marietta', 'GOGL  Golden Ocean', 'FHI  Federated Hermes ', 'UPW  ProShares Ultra Utilities', 'MOO  Market Vectors Agribusiness', 'SPGI  S&P Global', 'LABD  Direxion S&P Biotech Bear 3X', 'EEMA  iShares MSCI Emerging Markets Asia', 'CORZ  Core Scientific', 'LECO  Lincoln Electric', 'MTX  Minerals Technologies', 'KBWY  PowerShares KBW Premium Yield Equity REIT Portfolio', 'UWMC  UWM Holdings Corp', 'FRGI  Fiesta Restaurant', 'SDGR  Schrodinger', 'ARLO  Arlo Technologies', 'EATZ  Advisorshares Restaurant ETF', 'PSCC  PowerShares S&P SmallCap Consumer Staples Portfolio', 'PZA  PowerShares National AMT Free Municipal Bond Portfolio', 'VONV  Vanguard Russell 1000 Value', 'AB  AllianceBernstein', 'DFJ  WisdomTree Japan', 'BWXT  BWX Technologies', 'BIL  SPDR Barclays T Bill', 'TOLZ  ProShares DJ Brookfield Global Infrastructure', 'F  Ford', 'VIPS  Vipshop', 'MTCR  Metacrine', 'XHYE  XHYE ETF', 'CENX  Century Aluminum', 'IONQ  IONQ', 'FLR  Fluor', 'DNL  WisdomTree Global ex US Quality Dividend Growth Fund', 'DDD  3D Systems', 'CYTK  Cytokinetics', 'PLAN  Anaplan', 'BFH  Bread Financial Holdings Inc', 'ORGS  Orgenesis', 'SLB  Schlumberger', 'CGA  China Green', 'MHO  MI Homes', 'SDS  ProShares UltraShort S&P 500', 'GVA  Granite Construction', 'SNY  Sanofi', 'GLD  SPDR Gold Trust', 'BLRX  BioLineRx', 'ITRI  Itron', 'CVGI  Commercial Vehicle Group', 'VRT  Vertiv Holdings', 'J  Jacobs Engineering', 'LNG  Cheniere Energy', 'CDEV  Centennial Resource Development Inc (Class A Stock)', 'GII  SPDR S&P Global Infrastructure', 'DDOG  Datadog', 'MYGN  Myriad Genetics', 'BTU  Peabody Energy', 'CSTE  Caesarstone', 'BHLB  Berkshire Hills Bancorp', 'MTL  Mechel', 'KREF  KKR Real Estate Finance', 'LIN  Linde PLC', 'CRC  California Resources', 'LCNB  LCNB Corp.', 'QSR  Restaurant Brands', 'DBC  PowerShares Index', 'TMC  The Metals Company', 'SPOT  Spotify', 'SLGC  SomaLogic', 'BYFC  Broadway Financial', 'BBY  Best Buy', 'IEA  Infrastructure and Energy Alternatives', 'MNDY  monday.com', 'ADGI  Adagio Therapeutics', 'LYLT  Loyalty Ventures', 'FFBC  First Financial Bancorp', 'SKIL  Skillsoft', 'LFST  LifeStance Health Group', 'PSTX  Poseida Therapeutics', 'BWEN  Broadwind Energy', 'FLUX  Flux Power Holdings', 'HYG  iShares High Yield Corporate Bond ETF', 'SILX  ETFMG Prime 2x Daily Junior Silver Miners ETF', 'SBLK  Star Bulk Carriers', 'GXO  GXO Logistics', 'BIPC  Brookfield Infrastructure', 'ST  Sensata', 'TRTN  Triton International', 'DJIA  DJIA ETF', 'TURN  180 Degree Capital', 'SEE  Sealed Air', 'CHE  Chemed', 'BZH  Beazer Homes', 'SDC  SmileDirectClub', 'ABSI  AbSci Corp', 'AWI  Armstrong World', 'AVTX  Avalo Therapeutics', 'SPSM  SPDR Portfolio Small Cap ETF', 'ADIL  Adial Pharmaceuticals', 'USAC  USA Compression', 'CELH  Celsius Holdings', 'AIN  Albany International', 'SFIX  Stitch Fix', 'PATK  Patrick Industries', 'VHT  Vanguard Health Care', 'SVAL  iShares US Small Cap Value Factor ETF', 'LI  Li Auto', 'URTY  ProShares UltraPro Russell 2000', 'TSHA  Taysha Gene Therapies', 'QD  Qudian', 'XBIO  Xenetic Biosciences', 'UHT  Universal Health Realty', 'ULBI  Ultralife', 'WYY  WidePoint', 'VPG  Vishay Precision', 'RENN  Renren', 'QNGY  Quanergy Systems', 'SNPE  Xtrackers S&P 500 ESG ETF', 'VRA  Vera Bradley', 'TTSH  Tile Shop Holdings', 'THG  Hanover Insurance', 'SO  Southern Co', 'SKY  Skyline', 'XDNA  XDNA ETF', 'RPV  Guggenheim S&P 500', 'TALO  Talos Energy', 'TCBK  TriCo Bancshares', 'TNET  TriNet', 'VYM  Vanguard High Divdend Yield', 'VOX  Vanguard Telecom', 'ZBRA  Zebra Technologies', 'RAD  Rite Aid', 'QTRX  Quanterix', 'VIXM  ProShares VIX Mid Term Futures', 'TOUR  Tuniu', 'TTT  ProShares UltraPro 20', 'SWM  Schweitzer Mauduit International', 'WKHS  Workhorse Group', 'VRAY  ViewRay', 'SPNE  SeaSpine', 'XPOA  DPCM Capital', 'POWL  Powell', 'SANM  Sanmina', 'VCEL  Vericel', 'TNC  Tennant', 'SUSB  iShares Trust iShares ESG Aware 1 to 5 yr USD Corporate Bond', 'ZYME  Zymeworks', 'ZS  Zscaler', 'PTF  PowerShares DWA Technology Momentum Fund', 'TCRR  TCR2 Therapeutics', 'SOXQ  Invesco PHLX Semiconductor ETF', 'RJA  ELEMENTS Rogers Agriculture', 'YALA  Yalla Group Limited', 'SMBC  Southern Missouri Bancorp', 'VEON  VEON Ltd', 'TSE  Trinseo', 'VIRI  Virios Therapeutics', 'WTW  Willis Towers Watson', 'VHC  VirnetX', 'TIPT  Tiptree Financial', 'RDI  Reading International', 'VBK  Vanguard Small Cap Growth ETF', 'TRQ  Turquoise Hill', 'TECB  iShares USTech Breakthrough Multisector ETF', 'THTX  Theratechnologies', 'PSFE  Paysafe Ltd', 'YINN  Direxion Daily China Bull 3X Shares', 'SENT  Advisorshares Alpha DNA Equity Sentiment ETF', 'TWLO  Twilio', 'SNA  Snap on', 'SIGI  Selective Insurance', 'SRDX  SurModics', 'SCO  ProShares UltraShort DJ UBS Crude Oil', 'WRBY  Warby Parker', 'SPXU  ProShares UltraPro Short S&P 500', 'UONE  Urban One', 'VPL  Vanguard MSCI Pacific', 'YOU  Clear Secure', 'PTNQ  Pacer Trendpilot 100 ETF', 'TER  Teradyne', 'SDOG  ALPS Sector Dividend Dogs', 'USER  User Testing Inc', 'SOXL  Direxion Daily Semiconductor Bull 3X Shares', 'WWR  Westwater Resources', 'TCON  TRACON Pharmaceuticals', 'UST  ProShares Ultra 7 to 10', 'UAN  CVR Partners', 'TMUS  T Mobile', 'STVN  Stevanato Group', 'WEBL  Direxion Daily Dow Jones Internet Bull 3x', 'TTEK  Tetra Tech', 'TTE  TotalEnergies', 'TLIS  Talis Biomedical Corp', 'PWV  Powershares Dynamic Large Cap Value Portfolio', 'TWIN  Twin Disc', 'QEFA  SPDR MSCI EAFE Quality Mix', 'WRK  WestRock', 'VOC  Voc Energy Trust', 'STEP  StepStone Group', 'RTX  Raytheon Technologies', 'SMMD  iShares Russell 2500 ETF', 'XHYD  XHYD ETF', 'WWOW  Direxion World Without Waste ETF', 'VOT  Vanguard Mid Cap Growth', 'WSFS  WSFS Financial', 'STNE  StoneCo', 'TRVN  Trevena', 'TOWN  Towne Bank', 'TRIN  Trinity Capital', 'REYN  Reynolds Consumer Products', 'SIBN  SI BONE', 'USCI  United States Commodity Index Fund', 'PSTG  Pure Storage', 'VRAR  Glimpse Group', 'VT  Vanguard Total World Stock', 'SVC  Service Properties Trust', 'SSB  South State', 'SEV  Sono Group', 'VEL  Velocity Financial', 'REZ  iShares Real Estate', 'QQH  HCM Defender 100 Index ETF', 'SE  Sea Limited', 'RYU  Guggenheim S&P 500 Equal Weight Utilities', 'SWAV  ShockWave Medical', 'XGN  Exagen', 'TBI  TrueBlue', 'SHE  SPDR SSGA Gender Diversity Index ETF', 'SXT  Sensient', 'SCHZ  Schwab US Aggregate Bond', 'SDY  SPDR S&P Dividend', 'THRM  Gentherm', 'USFD  US Foods', 'WWE  WWE', 'SPYD  SPDR S&P 500 High Dividend ETF', 'VORB  Virgin Orbit', 'YELP  Yelp', 'TOST  Toast Inc', 'RAYS  Global X Solar ETF', 'TPG  TPG', 'SPGP  Invesco S&P 500 GARP ETF', 'RNW  ReNew Energy Global PLC', 'SDOW  ProShares UltraPro Short Dow 30', 'UMPQ  Umpqua Holdings', 'TIPX  SPDR Barclays 1 to 10 Year TIPS', 'PRA  ProAssurance', 'WEX  WEX Inc.', 'PSCD  PowerShares S&P SmallCap Consumer Discretionary Portfolio', 'SAIA  Saia', 'RDNT  RadNet', 'SLY  SPDR S&P 600 Small Cap', 'SCHP  Schwab US', 'WIT  Wipro', 'SPXL  Direxion Daily S&P Bull', 'VDE  Vanguard Energy ETF', 'TECK  Teck Resources', 'SPCX  S&P New Issue ETF', 'PTMN  Portman Ridge Finance', 'SOXS  Direxion Daily Semiconductor Bear 3X Shares', 'SLQD  iShares 0 5 Year Investment Grade Corporate Bond', 'RUSHA  Rush Enterprises', 'TSCO  Tractor Supply', 'XOS  Xos Inc', 'TGLS  Tecnoglass', 'Z  Zillow', 'TTNP  Titan Pharmaceuticals', 'TBPH  Theravance Biopharma', 'TDG  Transdigm', 'RCI  Rogers Communications', 'RDBX  Redbox Entertainment Inc', 'TFFP  TFF Pharmaceuticals', 'TISI  Team', 'RAMP  LiveRamp Holdings', 'SQ  Square', 'WE  WeWork', 'XMHQ  Invesco S&P MidCap Quality ETF', 'SYBX  Synlogic', 'RYE  Guggenheim S&P 500 Equal Weight Energy', 'UDN  PowerShares DB US Dollar Index Bearish Fund', 'UVIX  UVIX ETF', 'TROX  Tronox', 'USFR  WisdomTree Bloomberg Floating Rate Treasury Fund', 'TEX  Terex', 'STLA  Stellantis NV', 'UONEK  Urban One', 'RAPT  RAPT Therapeutics', 'XPEV  XPeng', 'VSH  Vishay', 'THO  Thor Industries', 'VCLT  Vanguard Long Term', 'STTK  Shattuck Labs', 'XIN  Xinyuan Real Estate', 'SYNL  Synalloy', 'WB  Weibo', 'USAK  USA Truck', 'TIL  Instill Bio Inc', 'THS  TreeHouse', 'WWD  Woodward', 'ROIV  Roivant Sciences', 'SFY  Sofi Select 500 ETF', 'SAFM  Sanderson Farms', 'QTWO  Q2 Holdings', 'SBRA  Sabra Healthcare', 'SRCL  Stericycle', 'SPLG  SPDR Portfolio Large Cap ETF', 'TENG  Direxion Daily 5G Communications Bull 2x ETF', 'PRTA  Prothena', 'TRMB  Trimble', 'SLGL  Sol Gel Technologies', 'XLI  Industrial Sector SPDR ETF', 'WLDN  Willdan Group', 'XPOF  Xponential Fitness', 'SQFT  Presidio Property Trust', 'VNT  Vontier Corp', 'TCPC  TCP Capital', 'SKYH  Sky Harbour Group', 'TTWO  TakeTwo Interactive Software', 'SIMS  SPDR S&P Kensho Intelligent Structures ETF', 'SEDG  SolarEdge', 'VECO  Veeco', 'VSAT  ViaSat', 'QTT  Qutoutiao', 'REET  iShares Global REIT', 'TPVG  TriplePoint Venture', 'VLY  Valley National', 'RSKD  Riskified', 'RAAS  Cloopen Group Holding Ltd', 'SLV  iShares Silver Trust', 'TFI  SPDR Nuveen Barclays', 'SURG  Surgepays Inc', 'WKEY  WISeKey International Holdings SA', 'UDMY  Udemy', 'PRLD  Prelude Therapeutics', 'VSTM  Verastem', 'RTL  Necessity Retail REIT', 'TGB  Taseko Mines', 'VCIT  Vanguard Intermediate', 'TBNK  Territorial Bancorp', 'SIEB  Siebert Financial', 'PNQI  PowerShares NASDAQ', 'PSCT  PowerShares S&P SmallCap Information Technology Portfolio', 'UPRO  ProShares UltraPro S&P 500', 'SJW  SJW Corp.', 'SNDL  Sundial Growers', 'TZOO  Travelzoo', 'SUPL  SUPL ETF', 'TNL  Travel plus Leisure', 'TSVT  2Seventy Bio Inc', 'ZIMV  ZimVie', 'TMCI  Treace Medical Concepts', 'SIRI  Sirius XM', 'SRAD  Sportradar Group', 'REXR  Rexford Industrial', 'SUNL  Sunlight Financial Holdings', 'PNR  Pentair', 'VST  Vistra Energy', 'PSIL  PSIL ETF', 'PXE  PowerShares Dynamic Energy Exploration and Production Portfolio', 'PRNT  The 3D Printing ETF', 'PSA  Public Storage', 'PSEC  Prospect Capital', 'TVTX  Travere Therapeutics', 'WAVD  Wavedancer Inc', 'STWD  Starwood Property', 'STBA  S&T Bancorp', 'WTI  W&T Offshore', 'PPH  Market Vectors Pharmaceutical', 'TNYA  Tenaya Therapeutics Inc', 'ZM  Zoom Video Communications', 'VZ  Verizon', 'RBLX  Roblox', 'VIVO  Meridian Bioscience', 'USLB  PowerShares Russell 1000 Low Beta Equal Weighted Portfolio', 'TRGP  Targa Resources', 'STON  StoneMor Partners', 'RCUS  Arcus Biosciences', 'SMR  Spring Valley Acquisition Corp', 'TYME  Tyme Technologies'"""

worldTickerRegex = re.compile(r" '([A-Za-z0-9.!@#$%^&*()]+)  [A-Za-z0-9 !@#$%^&*()]*")
tickerListComplete = worldTickerRegex.findall(tickerList)
# ~ print(tickerListComplete)
# -- END -- #






def closestTradingDate():
    # closest trading day
    from pandas.tseries.offsets import DateOffset

    today_date = pd.Timestamp(date.today())
    closest_date = today_date

    # Checks if today's date is SATURDAY
    if (today_date.isoweekday() == 6):
        closest_date = today_date - DateOffset(days = 1)
    # Checks if today_date is SUNDAY

    elif (today_date.isoweekday() == 7):
        closest_date = today_date - DateOffset(days = 2)

    # Check if the Friday is a holiday and subtract another day
    while(str(closest_date)[:10] in holidays2022):
        closest_date = closest_date - DateOffset(days = 1)
    
    # returns only date ('YYYY-MM-DD') without time
    closest_date = str(closest_date)[:10]
    
    return closest_date


def listOfDatesToCurrent():
# returns a list of active trading dates from the year 2022 (can change to be any year)
    dates = []

    for num in range(len(weekdays_year)):

        # Changing it from a 'pandas timestamp' to a string with only year-month-day
        dates.append(weekdays_year.date[num].strftime('%Y-%m-%d'))

    # Disclude holidays from active trading days
    dates = [date for date in dates if date not in holidays2022]
    try:
        # this slice will stop at today's date from first day
        # finds the index of today's date (date.today()) and slices list to stop at today
        dates_to_present = dates[:dates.index(closestTradingDate())+1]
    except:
        print("ERROR?! UNCOMMENT BELOW LINE TO GIVE STATIC AMOUNT OF DATES")
    #     dates_to_present = dates[:134]

    return dates_to_present


In [3]:
def closestTradingDate():
# will always return a day where the NYSE/NASDAQ are open.

    from pandas.tseries.offsets import DateOffset

    today_date = pd.Timestamp(date.today())
    closest_date = today_date

    # Checks if today's date is SATURDAY
    if (today_date.isoweekday() == 6):
        closest_date = today_date - DateOffset(days = 1)
    # Checks if today_date is SUNDAY

    elif (today_date.isoweekday() == 7):
        closest_date = today_date - DateOffset(days = 2)

    # Check if the Friday is a holiday and subtract another day
    while(str(closest_date)[:10] in holidays2022 or closest_date.isoweekday() == 6 or closest_date.isoweekday() == 7):
        closest_date = closest_date - DateOffset(days = 1)
    
    # returns only date ('YYYY-MM-DD') without time
    closest_date = str(closest_date)[:10]
    
    return closest_date
# will always return a day where the NYSE/NASDAQ are open.
# this allows you to run the program on any day of the week/holiday. 

def listOfDatesToCurrent():
# returns a list of active trading dates from the year 2022 (can change to be any year)

	dates = []	
	from datetime import date
	for num in range(len(weekdays_year)):
		
		# Changing it from a 'pandas timestamp' to a string with only year-month-day
		dates.append(weekdays_year.date[num].strftime('%Y-%m-%d'))

	# Disclude holidays from active trading days
	dates = [date for date in dates if date not in holidays2022]

	try:
		# this slice will stop at today's date from first day
		# finds the index of today's date (date.today()) and slices list to stop at today
		dates_to_present = dates[:dates.index(closestTradingDate())+1]
	except:
		print("ERROR?! UNCOMMENT BELOW LINE TO GIVE STATIC AMOUNT OF DATES")
	#     dates_to_present = dates[:134]
	return dates_to_present
# returns a list of active trading d
closestTradingDate()
listOfDatesToCurrent()

['2022-01-03',
 '2022-01-04',
 '2022-01-05',
 '2022-01-06',
 '2022-01-07',
 '2022-01-10',
 '2022-01-11',
 '2022-01-12',
 '2022-01-13',
 '2022-01-14',
 '2022-01-18',
 '2022-01-19',
 '2022-01-20',
 '2022-01-21',
 '2022-01-24',
 '2022-01-25',
 '2022-01-26',
 '2022-01-27',
 '2022-01-28',
 '2022-01-31',
 '2022-02-01',
 '2022-02-02',
 '2022-02-03',
 '2022-02-04',
 '2022-02-07',
 '2022-02-08',
 '2022-02-09',
 '2022-02-10',
 '2022-02-11',
 '2022-02-14',
 '2022-02-15',
 '2022-02-16',
 '2022-02-17',
 '2022-02-18',
 '2022-02-22',
 '2022-02-23',
 '2022-02-24',
 '2022-02-25',
 '2022-02-28',
 '2022-03-01',
 '2022-03-02',
 '2022-03-03',
 '2022-03-04',
 '2022-03-07',
 '2022-03-08',
 '2022-03-09',
 '2022-03-10',
 '2022-03-11',
 '2022-03-14',
 '2022-03-15',
 '2022-03-16',
 '2022-03-17',
 '2022-03-18',
 '2022-03-21',
 '2022-03-22',
 '2022-03-23',
 '2022-03-24',
 '2022-03-25',
 '2022-03-28',
 '2022-03-29',
 '2022-03-30',
 '2022-03-31',
 '2022-04-01',
 '2022-04-04',
 '2022-04-05',
 '2022-04-06',
 '2022-04-

In [4]:
x = [9,3,4,5
    ]
list(reversed(x))

[5, 4, 3, 9]

In [5]:
def dateRange_filepathList(date1, date2):
    datelist = list(reversed(listOfDatesToCurrent()))
    if datelist.index(date1) > datelist.index(date2):
        temp = date1
        date1 = date2
        date2 = temp
    return [# list comprehension that returns a list of file paths between 2 dates
            r"C:\Users\lorig\Desktop\PythonCode\Projects, Exercises, Programs\Trade Alert Journey\Data\{}\{}_intraday_volume\{}_intraday_volume_{}.csv"
            .format(ticker, ticker, ticker, datelist[i]) for i in range(datelist.index(date1), datelist.index(date2)+1)
           ]

In [6]:
def datetimeConverter(year=None, month=None, day=None, hour=None, minute=None):
    return datetime(year, month, day, hour, minute)



In [7]:
def printGraph(dataframe):
    
    # create graph
    fig = go.Figure()
    
#     dataframe.sum_calls = [i for i in reversed(list(dataframe.sum_calls))]
#     dataframe.sum_puts = [i for i in reversed(list(dataframe.sum_puts))]
    #update x-axis
    fig.add_trace(go.Scatter(x=dataframe.time, y=dataframe.sum_calls,
                                mode='lines', name='Sum Calls',
                                line=dict(color='lightgreen')))

    #update y-axis
    fig.add_trace(go.Scatter(x=dataframe.time, y=dataframe.sum_puts,
                            mode='lines', name='Sum Puts',
                            line=dict(color='firebrick')))
    avecalls = dataframe.sum_calls.sum()/len(dataframe.sum_calls)
    aveputs = dataframe.sum_puts.sum()/len(dataframe.sum_puts)
    
    fig.add_hline(y=aveputs, line=dict(color='orangered'))
    
    fig.add_hline(y=avecalls, line=dict(color='forestgreen'))

    
    fig.update_layout(xaxis=dict(type = "category", rangeslider=dict(visible=True)),
                     )
    
    
#     fig.update_layout(xaxis_range=[datetime(2022, 8, 5),datetime(2022, 9, 1)],
#                       yaxis_range=[0,1000])
    fig.show()
    


# Concatenated Option Volume Graph

In [9]:
# ---------------------- UPDATE ME ------------------------------- # 

ticker = "xle".upper()
date1 = "2022-07-05"
date2 = "2022-08-11"


# ---------------------------------------------------------------- # 
datelist = list(reversed(listOfDatesToCurrent()))
     
# START OF ACTUAL PROGRAM
printmd("**Volume between {} and {}**".format(date1, date2))

concatdf = DataFrame({})
num = datelist.index(date1)
xaxis_list = []

avesumputs = []
avesumcalls = []
for file in dateRange_filepathList(date1, date2):
#     year, month, day
    # read file
    df = pd.read_csv(file)
    
    # ----------------- CREATING X AXIS ------------------- #
    year = file[-14:-10].lstrip("0")
    month = file[-9:-7].lstrip("0")
    day = file[-6:-4].lstrip("0")

    for i in range(len(df)):
        hour = df.time.iloc[i][0:2].lstrip("0")
        minute = df.time.iloc[i][3:5]
#         print(hour)
#         if minute != "00":
#             minute = "0"
#         else:
#             pass
#             minute = df.time.iloc[i][3:5].lstrip("0")
        
        from datetime import datetime
        datetimeObject = datetime(int(year), int(month), int(day), int(hour), int(minute))
        xaxis_list.append(datetimeObject)
        
    # xaxis_list returns a list of datetime objects of every minute and every day of file
    # ----------------- X AXIS COMPLETED ------------------- #
    
    
    

    concatdf = pd.concat([concatdf, df])
#     print("Date:", file[-14:-4], len(concatdf))
# concatdf.time = [i for i in range(len(concatdf))]


concatdf.time = ((xaxis_list))

printGraph(concatdf)
# concatdf.to_csv(r"C:\Users\lorig\Desktop\PythonCode\Projects, Exercises, Programs\Trade Alert Journey\Data\XLE\XLE_intraday_volume\watttttttt.csv")

**Volume between 2022-07-05 and 2022-08-11**

In [291]:
testing = pd.read_csv(r"C:\Users\lorig\Desktop\PythonCode\Projects, Exercises, Programs\Trade Alert Journey\Data\XLE\XLE_intraday_volume\XLE_intraday_volume_2022-09-02.csv")
len(testing)

testing.sum_puts

0        180.0
1        303.0
2        623.0
3       1337.0
4       1434.0
        ...   
400    77505.0
401    77515.0
402    77536.0
403    78194.0
404    78723.0
Name: sum_puts, Length: 405, dtype: float64

In [358]:
from datetime import datetime, timezone

import plotly.graph_objects as go
import datetime

# x = [datetime.datetime(year=2013, month=10, day=4),
#      datetime.datetime(year=2013, month=11, day=5),
#      datetime.datetime(year=2013, month=12, day=6)]

fig = go.Figure(data=[go.Scatter(x=xaxis_list, y=[concatdf.sum_puts])])

# Use datetime objects to set xaxis range
# fig.update_layout(xaxis_range=[datetime.datetime(2013, 3, 17),
#                                datetime.datetime(2013, 11, 20)],
#                   yaxis_range=[0,3]
#                  )
# fig.show()


In [357]:
for file in dateRange_filepathList(date1, date2):
    df = pd.read_csv(file)


In [62]:
import plotly.express as px

df = px.data.stocks()
fig = px.line(df, x='date', y="GOOG")
fig.show()

In [90]:
read = pd.read_csv(r"C:\Users\lorig\Desktop\PythonCode\Projects, Exercises, Programs\Trade Alert Journey\Data\XLE\XLE_intraday_volume\watttttttt.csv")

In [91]:
printGraph(read)

In [103]:

import datetime
from datetime import datetime

In [109]:
# WORKING WITH DATETIME TO CREATE HOUR CHARTS.

testdate = datetime(2022, 9, 2, 9, 30)


datetime.datetime